In [1]:
import json
#import psycopg2
import pandas as pd

import requests
import urllib
import gzip

import codecs
        
from datetime import datetime

In [17]:
%%time
NERR = 100 # курсов
NARCH = 50 # тестов внутри курса
i= 0

# res = open('sheet.txt', 'w', encoding='utf-8' )

df = pd.DataFrame()

with open('log.txt', 'r', encoding='utf-8') as g:
    
    for line in g:  # -------------------------------------------------------------------------------------- архив
        j = 0
        i += 1
        fname = line.replace('https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1292/', '').replace('\n', '')
        # res.write(f'Handling {fname}\n')
        print(f'Обрабатывается {fname} ', datetime.now())
        
        with gzip.open(f'E:/Open_edu_zips/{fname}', mode='rt') as f:
            if j==0:
                print(f'    файл открыт ', datetime.now())
            k=0
            k_all=0
            str_cnt = 0
            for s in f: # --------------------------------------------------------------------------- строка архива
                str_cnt += 1
                if str_cnt%20000 == 0:
                    print(str_cnt, datetime.now(), k, k_all, d['time'])
                d = json.loads(s)
                if str_cnt==1:
                    print(f'    {datetime.now()} problem_check first ', d['time'])
                datetime_object = datetime.strptime(d['time'][:19],'%Y-%m-%dT%H:%M:%S')
                b = datetime_object > datetime.strptime("2020-04-10", '%Y-%m-%d')
                
                if d["event_type"] == "problem_check" and d["event_source"] == "server" and datetime_object > datetime.strptime("2010-04-10", '%Y-%m-%d'):
                    k_all += 1
                
                if d["event_type"] == "problem_check" and d["event_source"] == "server" and b and i <= NERR and k<=NARCH:
                    # print('problem_id: ', d['event']['problem_id'])
                    
                    if k==0:
                            print(f'    {datetime.now()} problem_check найден ', d['time'], str_cnt)
                    k += 1
                    
                    for key in d['event']['submission'].keys(): # ------------------------------------------- вопрос
                        
                        qdd = [] # контейнер для записей вопроса
                        
                        input_type = d['event']['submission'][key]['input_type']
                        # print('  question:', key, ' ', d['time'], ' ', d['username'] , ' ', input_type)
                        if input_type == 'checkboxgroup': # ******************************** checkboxgroup
                            
                            for choice_id in d['event']['submission'][key]['choices_map'].keys(): # --- опция вопроса
                                dd = {} # заготовка для "плоской" записи
                                dd['i'] = i
                                # print('      ', i)
                                dd['time'] = d['time']
                                dd['username'] = d['username']
                                dd['problem_id'] = d['event']['problem_id']
                                dd['max_grade'] = d['event']['max_grade']
                                dd['attempts'] = d['event']['attempts']
                                dd['grade'] = d['event']['grade']
                                dd['success'] = d['event']['success']
                                dd['question_id'] = key
                                dd['input_type'] =input_type
                                dd['question_txt'] = d['event']['submission'][key]['question']
                                dd['resp_correct'] = d['event']['submission'][key]['correct']
                                dd['choice_id'] = choice_id
                                dd['choice_txt'] = d['event']['submission'][key]['choices_map'][choice_id]
                                qdd.append(dd)
                                #print('-', dd)
                                #print('--', qdd)
                            
                            qdf = pd.DataFrame(qdd)
                            
                            
                            qdf['option_correct'] = False
                            qdf['option_selected'] = False
                            qdf['choice_txt2'] =''
                            qdf['choice_txt3'] =''
                            for n, ch_id in enumerate(d['event']['submission'][key]['choice_correct']):
                                # print('choice_correct', ch_id)
                                qdf['option_correct'] = qdf['option_correct'] | (qdf['choice_id'] == ch_id)
                                qdf.loc[qdf['choice_id']==ch_id, ['choice_txt2']] =\
                                        d['event']['submission'][key]['answer_correct'][n]
                                    
                            for n, ch_id in enumerate(d['event']['submission'][key]['choice']):
                                # print('choice', ch_id)
                                qdf['option_selected'] = qdf['option_selected'] | (qdf['choice_id'] == ch_id)
                                qdf.loc[qdf['choice_id']==ch_id, ['choice_txt3']] =\
                                        d['event']['submission'][key]['answer'][n]
           
                        elif input_type == 'choicegroup': # ******************************** choicegroup
                            
                            for choice_id in d['event']['submission'][key]['choices_map'].keys(): # --- опция вопроса
                                dd = {} # заготовка для "плоской" записи
                                dd['i'] = i
                                # print('      ', i)
                                dd['time'] = d['time']
                                dd['username'] = d['username']
                                dd['problem_id'] = d['event']['problem_id']
                                dd['max_grade'] = d['event']['max_grade']
                                dd['attempts'] = d['event']['attempts']
                                dd['grade'] = d['event']['grade']
                                dd['success'] = d['event']['success']
                                dd['question_id'] = key
                                dd['input_type'] =input_type
                                dd['question_txt'] = d['event']['submission'][key]['question'].replace('\n', ' ')
                                dd['resp_correct'] = d['event']['submission'][key]['correct']
                                dd['choice_id'] = choice_id
                                dd['choice_txt'] = d['event']['submission'][key]['choices_map'][choice_id]
                                qdd.append(dd)
                            
                            qdf = pd.DataFrame(qdd)
                            
                            qdf['option_correct'] = False
                            qdf['option_selected'] = False
                            qdf['choice_txt2'] =''
                            qdf['choice_txt3'] =''
                            for n, ch_id in enumerate(d['event']['submission'][key]['choice_correct']):
                                # print('choice_correct', ch_id)
                                qdf['option_correct'] = qdf['option_correct'] | (qdf['choice_id'] == ch_id)
                                qdf.loc[qdf['choice_id']==ch_id,['choice_txt2']] = \
                                        d['event']['submission'][key]['answer_correct'][n]
                                
                            ch_id = d['event']['submission'][key]['choice']
                            # print('choice', ch_id)
                            qdf['option_selected'] = qdf['option_selected'] | (qdf['choice_id'] == ch_id)
                            # print(d['event']['submission'][key]['answer'])
                            qdf.loc[qdf['choice_id']==ch_id, ['choice_txt3']] =\
                                        d['event']['submission'][key]['answer']
                
                        elif input_type == 'optioninput': # ********************************  option
                            
                            for choice_txt in d['event']['submission'][key]['answers_available']: # --- опция вопроса
                                dd = {} # заготовка для "плоской" записи
                                dd['i'] = i
                                # print('      ', i)
                                dd['time'] = d['time']
                                dd['username'] = d['username']
                                dd['problem_id'] = d['event']['problem_id']
                                dd['max_grade'] = d['event']['max_grade']
                                dd['attempts'] = d['event']['attempts']
                                dd['grade'] = d['event']['grade']
                                dd['success'] = d['event']['success']
                                dd['question_id'] = key
                                dd['input_type'] =input_type
                                dd['question_txt'] = d['event']['submission'][key]['question'].replace('\n', ' ')
                                dd['resp_correct'] = d['event']['submission'][key]['correct']
                                dd['choice_id'] = hash(choice_txt)
                                dd['choice_txt'] = choice_txt
                                qdd.append(dd)
                            
                            qdf = pd.DataFrame(qdd)
                            
                            qdf['option_correct'] = False
                            qdf['option_selected'] = False
                            qdf['choice_txt2'] =''
                            qdf['choice_txt3'] =''
                            for ch_txt in d['event']['submission'][key]['answer_correct']:
                                # print('choice_correct', ch_id)
                                qdf['option_correct'] = qdf['option_correct'] | (qdf['choice_txt'] == ch_txt)
                                
                            ch_txt = d['event']['submission'][key]['answer']
                            # print('choice', ch_id)
                            qdf['option_selected'] = qdf['option_selected'] | (qdf['choice_txt'] == ch_txt)
                            # print(d['event']['submission'][key]['answer'])

                        elif input_type == 'formulaequationinput': # ************************ formula
                            dd = {} # заготовка для "плоской" записи
                            dd['i'] = i
                            # print('      ', i)
                            dd['time'] = d['time']
                            dd['username'] = d['username']
                            dd['problem_id'] = d['event']['problem_id']
                            dd['max_grade'] = d['event']['max_grade']
                            dd['attempts'] = d['event']['attempts']
                            dd['grade'] = d['event']['grade']
                            dd['success'] = d['event']['success']
                            dd['question_id'] = key
                            dd['input_type'] =input_type
                            dd['question_txt'] = d['event']['submission'][key]['question'].replace('\n', ' ')
                            dd['resp_correct'] = d['event']['submission'][key]['correct']
                            dd['choice_id'] = None
                            dd['choice_txt'] = d['event']['submission'][key]['answer']
                            dd['option_correct'] = None
                            dd['option_selected'] = None
                            dd['choice_txt2'] = d['event']['submission'][key]['answer_correct']
                            dd['choice_txt3'] =''
                            
                            qdf = pd.DataFrame(dd)
                            
                        elif input_type == 'textline': # ************************************ text
                            dd = {} # заготовка для "плоской" записи
                            dd['i'] = i
                            # print('      ', i)
                            dd['time'] = d['time']
                            dd['username'] = d['username']
                            dd['problem_id'] = d['event']['problem_id']
                            dd['max_grade'] = d['event']['max_grade']
                            dd['attempts'] = d['event']['attempts']
                            dd['grade'] = d['event']['grade']
                            dd['success'] = d['event']['success']
                            dd['question_id'] = key
                            dd['input_type'] =input_type
                            dd['question_txt'] = d['event']['submission'][key]['question'].replace('\n', ' ')
                            dd['resp_correct'] = d['event']['submission'][key]['correct']
                            dd['choice_id'] = None
                            dd['choice_txt'] = d['event']['submission'][key]['answer']
                            dd['option_correct'] = d['event']['submission'][key]['answer'] in d['event']['submission'][key]['answer_correct']
                            dd['option_selected'] = None
                            dd['choice_txt2'] = d['event']['submission'][key]['answer_correct']
                            dd['choice_txt3'] =''
                            
                            qdf = pd.DataFrame(dd)
                            qdf['option_selected'] = (qdf['choice_txt']==qdf['choice_txt2'])
                           # if dd['time'] == '2020-04-10T04:10:56.174582+00:00':
                           #     print(dd)
                           #     print()
                           #     print(qdf)
                            
                        j += 1
                            
                        if qdd or dd:
                            df = pd.concat([df, qdf], ignore_index=True)
                        
                        if i>NERR or k>NARCH:
                            break

                        
                if i>NERR or k>NARCH:
                    break
                    
            print('    ', str_cnt, f'{k}/{k_all}', d['time'])
            
            if i>NERR:
                break
            print()
# res.close()

Обрабатывается hse_AGBASE_spring_2020.gz  2020-06-10 10:35:06.378609
    файл открыт  2020-06-10 10:35:06.378609
    2020-06-10 10:35:06.378609 problem_check first  2020-01-31T21:25:29.042698+00:00
20000 2020-06-10 10:35:09.053743 0 132 2020-02-20T09:45:09.378709+00:00
40000 2020-06-10 10:35:11.325765 0 261 2020-02-24T08:41:44.209440+00:00
60000 2020-06-10 10:35:13.756428 0 261 2020-02-29T03:20:13.997966+00:00
80000 2020-06-10 10:35:15.993329 0 277 2020-03-03T18:40:53.391098+00:00
100000 2020-06-10 10:35:18.569988 0 314 2020-03-09T18:58:43.555802+00:00
120000 2020-06-10 10:35:21.611910 0 365 2020-03-17T18:56:57.589721+00:00
140000 2020-06-10 10:35:23.896105 0 403 2020-03-27T17:04:20.588540+00:00
160000 2020-06-10 10:35:25.934278 0 444 2020-04-07T04:28:22.639654+00:00
    2020-06-10 10:35:26.628042 problem_check найден  2020-04-10T14:35:02.033591+00:00 166370
180000 2020-06-10 10:35:36.870557 43 490 2020-04-23T07:21:15.418277+00:00
     181783 51/498 2020-04-25T13:44:14.965722+00:00

Об

1420000 2020-06-10 10:39:41.078203 0 12277 2020-03-22T18:21:20.407047+00:00
1440000 2020-06-10 10:39:43.223955 0 12525 2020-03-22T19:37:04.424474+00:00
1460000 2020-06-10 10:39:45.486479 0 12836 2020-03-22T20:56:22.131690+00:00
1480000 2020-06-10 10:39:47.546895 0 13023 2020-03-23T10:11:30.139277+00:00
1500000 2020-06-10 10:39:49.965985 0 13218 2020-03-23T14:31:51.628778+00:00
1520000 2020-06-10 10:39:52.132621 0 13379 2020-03-23T18:38:51.214375+00:00
1540000 2020-06-10 10:39:54.243789 0 13602 2020-03-24T08:39:08.442860+00:00
1560000 2020-06-10 10:39:56.361655 0 13814 2020-03-24T13:48:40.661333+00:00
1580000 2020-06-10 10:39:58.518952 0 14009 2020-03-24T18:41:10.352557+00:00
1600000 2020-06-10 10:40:00.687637 0 14224 2020-03-25T08:41:25.896886+00:00
1620000 2020-06-10 10:40:03.331278 0 14441 2020-03-25T14:40:28.933409+00:00
1640000 2020-06-10 10:40:05.504018 0 14674 2020-03-25T20:50:28.517698+00:00
1660000 2020-06-10 10:40:07.793056 0 14875 2020-03-26T12:09:41.589498+00:00
1680000 2020

320000 2020-06-10 10:44:07.351818 3 1021 2020-04-10T14:55:06.462187+00:00
     331627 51/1069 2020-04-12T19:35:59.618026+00:00

Обрабатывается hse_COMPOL_spring_2020.gz  2020-06-10 10:44:35.748074
    файл открыт  2020-06-10 10:44:35.748074
    2020-06-10 10:44:35.748074 problem_check first  2020-01-31T21:26:05.820946+00:00
20000 2020-06-10 10:44:37.939113 0 32 2020-02-08T12:59:22.064577+00:00
40000 2020-06-10 10:44:40.101383 0 85 2020-02-13T16:04:46.989766+00:00
60000 2020-06-10 10:44:42.338033 0 189 2020-02-21T07:21:53.699874+00:00
80000 2020-06-10 10:44:45.779363 0 291 2020-03-15T20:06:01.197742+00:00
100000 2020-06-10 10:44:52.327973 0 391 2020-04-12T12:41:17.042865+00:00
    2020-06-10 10:44:52.468586 problem_check найден  2020-04-12T15:33:09.683686+00:00 101139
120000 2020-06-10 10:45:28.982824 40 431 2020-04-29T07:54:14.869688+00:00
     124888 40/431 2020-05-13T21:30:48.999230+00:00

Обрабатывается hse_CONTART_spring_2020.gz  2020-06-10 10:45:29.566704
    файл открыт  2020-06-

20000 2020-06-10 10:49:51.204097 0 81 2020-02-26T06:07:50.674832+00:00
40000 2020-06-10 10:49:53.494488 0 150 2020-03-06T10:54:11.387035+00:00
60000 2020-06-10 10:49:56.823999 0 224 2020-03-22T06:50:11.850616+00:00
    2020-06-10 10:49:59.770703 problem_check найден  2020-04-10T14:17:45.151032+00:00 79926
80000 2020-06-10 10:50:00.576309 1 295 2020-04-10T15:02:54.350724+00:00
     95184 51/345 2020-04-26T09:13:52.649160+00:00

Обрабатывается hse_CORPMN_spring_2020.gz  2020-06-10 10:50:42.384105
    файл открыт  2020-06-10 10:50:42.385103
    2020-06-10 10:50:42.386106 problem_check first  2020-01-31T21:19:13.870530+00:00
20000 2020-06-10 10:50:44.618821 0 17 2020-02-04T12:13:31.819856+00:00
40000 2020-06-10 10:50:46.731078 0 55 2020-02-07T21:06:39.366462+00:00
60000 2020-06-10 10:50:49.255131 0 129 2020-02-09T19:57:11.989354+00:00
80000 2020-06-10 10:50:51.236383 0 155 2020-02-13T14:13:15.314004+00:00
100000 2020-06-10 10:50:53.187910 0 189 2020-02-16T14:46:06.019791+00:00
120000 2020-

220000 2020-06-10 10:56:57.608045 0 2177 2020-02-29T15:09:15.830726+00:00
240000 2020-06-10 10:56:59.754172 0 2373 2020-03-01T19:51:24.725806+00:00
260000 2020-06-10 10:57:02.082097 0 2490 2020-03-04T12:00:12.367412+00:00
280000 2020-06-10 10:57:04.884395 0 2654 2020-03-07T14:48:31.907902+00:00
300000 2020-06-10 10:57:07.151951 0 2955 2020-03-09T04:24:08.466558+00:00
320000 2020-06-10 10:57:09.403363 0 3177 2020-03-10T12:47:43.457211+00:00
340000 2020-06-10 10:57:12.032094 0 3382 2020-03-12T10:26:20.225134+00:00
360000 2020-06-10 10:57:14.336302 0 3535 2020-03-13T19:08:16.108803+00:00
380000 2020-06-10 10:57:16.853545 0 3883 2020-03-15T13:04:48.722890+00:00
400000 2020-06-10 10:57:22.995769 0 4117 2020-03-17T13:43:57.344567+00:00
420000 2020-06-10 10:57:25.421228 0 4254 2020-03-19T13:31:23.098602+00:00
440000 2020-06-10 10:57:27.708794 0 4389 2020-03-21T08:07:09.835584+00:00
460000 2020-06-10 10:57:30.054332 0 4610 2020-03-22T10:56:35.021465+00:00
480000 2020-06-10 10:57:32.400076 0 48

...

420000 2020-06-10 11:55:01.073777 0 31000 2020-03-10T09:55:28.044754+00:00
440000 2020-06-10 11:55:03.338975 0 31629 2020-03-11T15:39:51.667143+00:00
460000 2020-06-10 11:55:05.795950 0 32678 2020-03-13T20:25:49.938150+00:00
480000 2020-06-10 11:55:08.235260 0 33839 2020-03-15T10:23:11.439403+00:00
500000 2020-06-10 11:55:10.971635 0 35784 2020-03-15T17:02:52.151536+00:00
520000 2020-06-10 11:55:13.822080 0 37953 2020-03-15T20:17:53.220162+00:00
540000 2020-06-10 11:55:17.329940 0 38951 2020-03-17T13:17:20.776021+00:00
560000 2020-06-10 11:55:19.603886 0 39428 2020-03-19T13:09:14.553847+00:00
580000 2020-06-10 11:55:22.170875 0 40624 2020-03-20T22:11:06.467127+00:00
600000 2020-06-10 11:55:24.735543 0 41980 2020-03-22T09:44:44.187262+00:00
620000 2020-06-10 11:55:27.753334 0 43953 2020-03-22T18:02:31.180362+00:00
640000 2020-06-10 11:55:30.693029 0 46036 2020-03-23T11:46:10.957673+00:00
660000 2020-06-10 11:55:33.282605 0 46733 2020-03-25T20:18:38.163587+00:00
680000 2020-06-10 11:55:3

140000 2020-06-10 12:04:05.962957 0 335 2020-02-19T12:13:43.115306+00:00
160000 2020-06-10 12:04:08.045332 0 382 2020-02-20T21:26:08.726507+00:00
180000 2020-06-10 12:04:10.253070 0 432 2020-02-22T14:03:39.300311+00:00
200000 2020-06-10 12:04:12.371747 0 526 2020-02-23T16:15:43.119249+00:00
220000 2020-06-10 12:04:14.475092 0 659 2020-02-24T17:52:55.513053+00:00
240000 2020-06-10 12:04:16.485448 0 712 2020-02-26T09:19:58.930786+00:00
260000 2020-06-10 12:04:18.839876 0 996 2020-02-27T19:20:11.093392+00:00
280000 2020-06-10 12:04:20.847978 0 1080 2020-03-01T07:53:16.463394+00:00
300000 2020-06-10 12:04:22.959547 0 1196 2020-03-02T05:40:04.540934+00:00
320000 2020-06-10 12:04:25.070245 0 1230 2020-03-03T16:36:07.433709+00:00
340000 2020-06-10 12:04:27.138966 0 1260 2020-03-05T21:26:02.181154+00:00
360000 2020-06-10 12:04:29.469150 0 1326 2020-03-08T09:27:26.078815+00:00
380000 2020-06-10 12:04:31.503013 0 1430 2020-03-09T11:34:08.386237+00:00
400000 2020-06-10 12:04:33.499194 0 1468 2020

320000 2020-06-10 12:13:36.025695 0 1102 2020-02-19T16:34:28.554066+00:00
340000 2020-06-10 12:13:38.120985 0 1160 2020-02-20T13:27:29.747098+00:00
360000 2020-06-10 12:13:40.297798 0 1224 2020-02-21T12:27:50.354289+00:00
380000 2020-06-10 12:13:42.588761 0 1300 2020-02-22T12:59:30.915158+00:00
400000 2020-06-10 12:13:45.708911 0 1424 2020-02-23T08:10:48.752600+00:00
420000 2020-06-10 12:13:47.962117 0 1571 2020-02-23T15:33:15.278032+00:00
440000 2020-06-10 12:13:50.236742 0 1943 2020-02-23T20:47:10.768643+00:00
460000 2020-06-10 12:13:52.257404 0 2016 2020-02-24T17:39:58.719117+00:00
480000 2020-06-10 12:13:54.270804 0 2089 2020-02-26T08:10:10.344837+00:00
500000 2020-06-10 12:13:56.950859 0 2171 2020-02-27T20:23:37.114484+00:00
520000 2020-06-10 12:13:59.256194 0 2305 2020-02-29T17:25:18.061016+00:00
540000 2020-06-10 12:14:01.552168 0 2632 2020-03-01T18:57:40.053275+00:00
560000 2020-06-10 12:14:03.647010 0 2834 2020-03-03T11:03:40.248403+00:00
580000 2020-06-10 12:14:06.169813 0 29

200000 2020-06-10 12:27:21.936029 0 578 2020-03-29T17:18:32.862714+00:00
220000 2020-06-10 12:27:23.827877 0 604 2020-04-02T21:18:15.974859+00:00
240000 2020-06-10 12:27:27.079636 0 674 2020-04-09T05:20:51.116148+00:00
    2020-06-10 12:27:27.620075 problem_check найден  2020-04-10T07:46:16.610665+00:00 243567
     250777 51/730 2020-04-12T18:36:26.699004+00:00

Обрабатывается hse_RLING_spring_2020.gz  2020-06-10 12:30:40.275808
    файл открыт  2020-06-10 12:30:40.276808
    2020-06-10 12:30:40.277808 problem_check first  2020-01-31T21:28:22.044796+00:00
20000 2020-06-10 12:30:42.418081 0 211 2020-02-16T07:19:23.711783+00:00
40000 2020-06-10 12:30:44.615533 0 501 2020-02-24T11:49:27.959274+00:00
60000 2020-06-10 12:30:48.499216 0 870 2020-03-16T15:10:47.046333+00:00
    2020-06-10 12:30:50.634117 problem_check найден  2020-04-10T11:01:29.938980+00:00 74784
80000 2020-06-10 12:31:01.393606 36 1096 2020-04-15T11:54:35.785431+00:00
     83330 51/1111 2020-04-18T22:48:26.607476+00:00

Обр

360000 2020-06-10 12:39:39.161428 0 12217 2020-04-08T07:53:06.150477+00:00
    2020-06-10 12:39:40.292343 problem_check найден  2020-04-10T01:30:08.797321+00:00 368255
     373185 51/12430 2020-04-11T20:21:37.222901+00:00

Обрабатывается hse_TIGR_spring_2020.gz  2020-06-10 12:41:37.368484
    файл открыт  2020-06-10 12:41:37.368484
    2020-06-10 12:41:37.368484 problem_check first  2020-01-31T21:26:56.729776+00:00
20000 2020-06-10 12:41:40.729695 0 66 2020-03-03T15:43:52.402233+00:00
40000 2020-06-10 12:41:43.385513 0 171 2020-03-06T11:39:09.286788+00:00
60000 2020-06-10 12:41:45.630608 0 334 2020-03-09T10:09:00.578249+00:00
80000 2020-06-10 12:41:47.874611 0 406 2020-03-12T08:19:21.665041+00:00
100000 2020-06-10 12:41:50.211244 0 493 2020-03-15T15:14:29.336054+00:00
120000 2020-06-10 12:41:55.458627 0 586 2020-03-18T21:57:26.647381+00:00
140000 2020-06-10 12:41:57.914170 0 732 2020-03-23T20:02:27.945448+00:00
160000 2020-06-10 12:42:00.478957 0 858 2020-03-29T20:42:17.114398+00:00
18

In [2]:
# загрузка gz-архива по ссылке
def load_from_url(url):
    fname = url[url.rfind('/')+1:].strip()
    gz = urllib.request.urlopen(url).read()
    
    f = open(f'E:/Open_edu_zips/{fname}', "wb")
    f.write(gz)
    f.close()
    
    return fname



In [ ]:
with open('log.txt', 'r', encoding='utf-8') as g:
    for line in g:
        print(f'Обрабатывается {line}')
        fname = load_from_url(line)

In [48]:
%%time
import os
with open('log.txt', 'r', encoding='utf-8') as g:
    for line in g:
        fname = line.replace('https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1292/', '').replace('\n', '')
        print(f'Обрабатывается {fname}')
        print(os.path.getsize(f'E:/Open_edu_zips/{fname}')) 


Обрабатывается hse_AGBASE_spring_2020.gz
11373401
Обрабатывается hse_AIPP_spring_2020.gz
32752692
Обрабатывается hse_ART_spring_2020.gz
303945367
Обрабатывается hse_BASENTREP_spring_2020.gz
11661622
Обрабатывается hse_BRAPS_spring_2020.gz
30054128
Обрабатывается hse_CINCULT_spring_2020.gz
24765993
Обрабатывается hse_COMPOL_spring_2020.gz
14545790
Обрабатывается hse_CONTART_spring_2020.gz
147664837
Обрабатывается hse_CorFin_spring_2020.gz
43758662
Обрабатывается hse_CORPIS_spring_2020.gz
7696705
Обрабатывается hse_CORPMN_spring_2020.gz
16768788
Обрабатывается hse_CORPUS_spring_2020.gz
17636656
Обрабатывается hse_CULT_spring_2020.gz
31160231
Обрабатывается hse_DATPRO_spring_2020.gz
43604844
Обрабатывается hse_DEMO_spring_2020.gz
26116272
Обрабатывается hse_DIGHIST_spring_2020.gz
20971519
Обрабатывается hse_ECONOM_spring_2020.gz
76475171
Обрабатывается hse_ECPSY_spring_2020.gz
11580039
Обрабатывается hse_ECSOC_spring_2020.gz
11991417
Обрабатывается hse_EXACTS_spring_2020.gz
22837529
Обраб

In [15]:
x='{"input_type": "textline", "question": "\u0417\u0430\u043f\u0438\u0448\u0438\u0442\u0435 \u0432 \u0434\u0432\u043e\u0438\u0447\u043d\u043e\u0439 \u0441\u0438\u0441\u0442\u0435\u043c\u0435 \u0447\u0438\u0441\u043b\u043e 1001.", "group_label": "", "response_type": "stringresponse", "answer": "1111101001", "variant": "", "correct": true}'
x.encode('cp1251').decode('cp1251')

'{"input_type": "textline", "question": "Запишите в двоичной системе число 1001.", "group_label": "", "response_type": "stringresponse", "answer": "1111101001", "variant": "", "correct": true}'

In [33]:
res.close()

In [39]:
tps

{'checkboxgroup|choiceresponse',
 'choicegroup|multiplechoiceresponse',
 'formulaequationinput|numericalresponse',
 'optioninput|optionresponse',
 'textline|stringresponse'}

In [11]:
34%10


4

In [200]:
hash('1.Как Евриклея узнает Одиссея?')

2672461483590179972

In [211]:
df.to_csv('qw.csv', sep=';', encoding='cp1251') #, encoding='cp1251'

In [19]:
df.shape

(149103, 18)

In [18]:
df.to_json('df.json', orient='table')

In [15]:
df[df['question_txt'].apply(lambda x: x.strip()=='')]
#[df['time']=='2020-04-10T14:35:02.033591+00:00']

,i,time,username,problem_id,max_grade,attempts,grade,success,question_id,input_type,question_txt,resp_correct,choice_id,choice_txt,option_correct,option_selected,choice_txt2,choice_txt3


In [22]:
%%time

with open('publish.txt', 'r', encoding='utf-8') as g:
    for line in g:
        fname = line.replace('https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1292/', '').replace('\n', '')
        print(f'Обрабатывается {fname}')
        with gzip.open(f'E:/Open_edu_zips/{fname}', mode='rt') as f:
            i=0
            for s in f:
                d = json.loads(s)
                if d["event_type"] == "problem_publish": # and d["event_source"] == "server":
                    #print(d['event']['submission'].keys()) "problem_check"
                    print(d['time'])
                    i += 1
                    #if i>1:
                    #    break
            print(i, '\n')

Обрабатывается hse_BRAPS_spring_2020.gz
2020-04-28T12:31:35.260710+00:00
1 

Обрабатывается hse_CINCULT_spring_2020.gz
2020-04-15T09:17:48.145534+00:00
1 

Обрабатывается hse_COMPOL_spring_2020.gz
2020-04-20T08:29:40.139048+00:00
2020-04-20T08:30:20.568854+00:00
2 

Обрабатывается hse_CONTART_spring_2020.gz
2020-04-20T09:54:44.626763+00:00
2020-04-20T09:55:57.171760+00:00
2020-04-20T09:56:45.339402+00:00
3 

Обрабатывается hse_CORPIS_spring_2020.gz
2020-04-11T21:45:16.072024+00:00
2020-04-12T10:08:48.374845+00:00
2020-04-12T10:10:13.902162+00:00
2020-04-15T09:01:43.712329+00:00
4 

Обрабатывается hse_INTMODPOL_spring_2020.gz
2020-04-13T00:45:03.354629+00:00
1 

Обрабатывается hse_INTRAI_spring_2020.gz
2020-04-13T18:48:01.067762+00:00
2020-05-08T22:58:49.438129+00:00
2 

Обрабатывается hse_INTRML_spring_2020.gz
2020-04-11T17:24:39.343353+00:00
2020-04-12T21:15:29.228436+00:00
2020-04-12T21:44:44.629734+00:00
2020-04-13T15:28:32.806210+00:00
2020-04-18T10:15:41.244386+00:00
2020-04-21T10

KeyboardInterrupt: 

In [40]:
%%time
fname = 'hse_CONTART_spring_2020.gz'
with gzip.open(f'E:/Open_edu_zips/{fname}', mode='rt') as f:
    i=0
    for s in f:
        d = json.loads(s)
        if d["event_type"] == "problem_check" and d["event_source"] == "server":
            #print(d['event']['submission'].keys())
            #print(d, '\n')
            i += 1
            #if i>1:
            #    break
    print(i)    
    # s = f.readline()

            #df = pd.read_csv(f, sep=';', header=None)

13313
Wall time: 1min 53s


In [217]:
s = r'{"username": "osen200606", "event_type": "problem_check", "ip": "176.117.22.210", "agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 YaBrowser/20.3.2.242 Yowser/2.5 Safari/537.36", "host": "courses.openedu.ru", "referer": "https://courses.openedu.ru/courses/course-v1:hse+DIGHIST+spring_2020/courseware/af2b5624a78e4260a68e4992ede84603/d26ec9a177da433aa2e8a9196a11ce26/", "accept_language": "ru;q=1.0", "event": {"submission": {"e4116937a01a4c87894309308dacfa4e_8_1": {"choices_map": {"choice_0": "\u042d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "choice_1": "\u041d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u044b\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"}, "answers_available": ["\u042d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "\u041d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u044b\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"], "question": "7. \u041f\u0440\u043e\u0447\u0438\u0442\u0430\u0439\u0442\u0435 \u043e\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u043f\u0440\u043e\u0435\u043a\u0442\u0430 \u0438 \u043e\u0442\u0432\u0435\u0442\u044c\u0442\u0435, \u043e \u043a\u0430\u043a\u043e\u043c \u0442\u0438\u043f\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0445 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0439 \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.\n\n\u2026 \u0412\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u0430\u044f \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u044f \u043f\u0440\u0438\u043b\u0435\u0433\u0430\u044e\u0449\u0435\u0439 \u0442\u0435\u0440\u0440\u0438\u0442\u043e\u0440\u0438\u0438 \u0438 \u0437\u0430\u0441\u0442\u0440\u043e\u0439\u043a\u0438 \u0431\u044b\u043b\u0430 \u0441\u043e\u0437\u0434\u0430\u043d\u0430 \u043f\u043e \u0437\u0430\u043a\u0430\u0437\u0443 \u043c\u0443\u0437\u0435\u044f-\u0437\u0430\u043f\u043e\u0432\u0435\u0434\u043d\u0438\u043a\u0430. \u0420\u0430\u0437\u0440\u0430\u0431\u043e\u0442\u0447\u0438\u043a\u0438, \u043f\u0440\u0438\u0432\u043b\u0435\u0447\u0435\u043d\u043d\u044b\u0435 \u043c\u0443\u0437\u0435\u0435\u043c, \u0441\u043e\u0437\u0434\u0430\u043b\u0438 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u044e, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u0434\u0430\u0435\u0442 \u043f\u0440\u0438\u0431\u043b\u0438\u0437\u0438\u0442\u0435\u043b\u044c\u043d\u043e\u0435 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u0438\u0435 \u043e \u0440\u0430\u0441\u0441\u043c\u0430\u0442\u0440\u0438\u0432\u0430\u0435\u043c\u043e\u043c \u043e\u0431\u044a\u0435\u043a\u0442\u0435, \u043f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0441\u043e\u0437\u0434\u0430\u0432\u0430\u0442\u044c \u043d\u0430 \u043e\u0441\u043d\u043e\u0432\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0442\u0443\u0440\u044b, \u043e\u043d\u043b\u0430\u0439\u043d-\u0438\u0433\u0440\u044b \u0438 \u0442.\u043f. \u2026", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u042d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"], "answer": "\u042d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "correct": true}, "e4116937a01a4c87894309308dacfa4e_15_1": {"choices_map": {"choice_0": "\u041f. \u0420\u0435\u0439\u043b\u0438", "choice_1": "\u0420. \u0424\u043e\u0433\u0435\u043b\u044c", "choice_2": "\u041c. \u0422\u0430\u043b\u043b\u0435\u0440", "choice_3": "\u041b. \u042d\u0439\u043b\u0435\u0440"}, "answers_available": ["\u041f. \u0420\u0435\u0439\u043b\u0438", "\u0420. \u0424\u043e\u0433\u0435\u043b\u044c", "\u041c. \u0422\u0430\u043b\u043b\u0435\u0440", "\u041b. \u042d\u0439\u043b\u0435\u0440"], "question": "14. \u041a\u0442\u043e \u044f\u0432\u043b\u044f\u0435\u0442\u0441\u044f \u043e\u0441\u043d\u043e\u0432\u0430\u0442\u0435\u043b\u0435\u043c \u043d\u0430\u043f\u0440\u0430\u0432\u043b\u0435\u043d\u0438\u044f \u00ab\u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u0430\u044f \u0430\u0440\u0445\u0435\u043e\u043b\u043e\u0433\u0438\u044f\u00bb? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041f. \u0420\u0435\u0439\u043b\u0438"], "answer": "\u041c. \u0422\u0430\u043b\u043b\u0435\u0440", "correct": false}, "e4116937a01a4c87894309308dacfa4e_5_1": {"choices_map": {"choice_0": "\u00ab\u0412\u043d\u0443\u0442\u0440\u0438 \u0411\u0440\u0435\u0439\u0433\u0435\u043b\u044f\u00bb, www.insidebruegel.net", "choice_1": "\u0418\u0441\u0442\u043e\u0440\u0438\u044f \u0420\u043e\u0441\u0441\u0438\u0438 \u0432 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u044f\u0445, russiainphoto.ru", "choice_2": "Gesture Atlas, biblhertz.github.io/atlas", "choice_3": "Recogito, recogito.pelagios.org"}, "answers_available": ["\u00ab\u0412\u043d\u0443\u0442\u0440\u0438 \u0411\u0440\u0435\u0439\u0433\u0435\u043b\u044f\u00bb, www.insidebruegel.net", "\u0418\u0441\u0442\u043e\u0440\u0438\u044f \u0420\u043e\u0441\u0441\u0438\u0438 \u0432 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u044f\u0445, russiainphoto.ru", "Gesture Atlas, biblhertz.github.io/atlas", "Recogito, recogito.pelagios.org"], "question": "4. \u0412 \u043a\u0430\u043a\u043e\u043c \u043f\u0440\u043e\u0435\u043a\u0442\u0435 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0430 \u0442\u0435\u0445\u043d\u043e\u043b\u043e\u0433\u0438\u044f \u043c\u0430\u043a\u0440\u043e\u0444\u043e\u0442\u043e\u0441\u044a\u0435\u043c\u043a\u0438? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u00ab\u0412\u043d\u0443\u0442\u0440\u0438 \u0411\u0440\u0435\u0439\u0433\u0435\u043b\u044f\u00bb, www.insidebruegel.net"], "answer": "Gesture Atlas, biblhertz.github.io/atlas", "correct": false}, "e4116937a01a4c87894309308dacfa4e_10_1": {"choices_map": {"choice_0": "\u0413\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "choice_1": "\u041e\u043f\u0438\u0441\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "choice_2": "\u041a\u0430\u0440\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "choice_3": "\u0421\u0442\u0430\u0442\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438"}, "answers_available": ["\u0413\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u041e\u043f\u0438\u0441\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u041a\u0430\u0440\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u0421\u0442\u0430\u0442\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438"], "question": "9. \u041a\u0430\u043a\u0438\u0435 \u0442\u0438\u043f\u044b \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u043e\u0432 \u043c\u043e\u0436\u043d\u043e \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u0442\u044c \u043f\u0440\u0438 \u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0438 \u0431\u0430\u0437\u044b \u0434\u043b\u044f \u043f\u043e\u0441\u0442\u0440\u043e\u0435\u043d\u0438\u044f \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0412\u0421\u0415 \u043f\u0440\u0430\u0432\u0438\u043b\u044c\u043d\u044b\u0435 \u043e\u0442\u0432\u0435\u0442\u044b.", "input_type": "checkboxgroup", "variant": "", "group_label": "", "choice": ["choice_0", "choice_2"], "choice_correct": ["choice_0", "choice_1", "choice_2"], "response_type": "choiceresponse", "answer_correct": ["\u0413\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u041e\u043f\u0438\u0441\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u041a\u0430\u0440\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438"], "answer": ["\u0413\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438", "\u041a\u0430\u0440\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u0438"], "correct": false}, "e4116937a01a4c87894309308dacfa4e_13_1": {"choices_map": {"choice_0": "\u044d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u043e\u0439", "choice_1": "\u043d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u043e\u0439"}, "answers_available": ["\u044d\u043a\u0441\u043a\u0443\u0440\u0441\u0438\u043e\u043d\u043d\u043e-\u0442\u0443\u0440\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u043e\u0439", "\u043d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u043e\u0439"], "question": "12. \u0423 \u0432\u0430\u0441 \u0438\u043c\u0435\u044e\u0442\u0441\u044f \u0447\u0435\u0440\u0442\u0435\u0436\u0438, \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u044f \u0438 \u043e\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u0444\u0443\u043d\u043a\u0446\u0438\u043e\u043d\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u044f \u043d\u0435\u043a\u043e\u0435\u0433\u043e \u0438\u0441\u0442\u043e\u0440\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u043e\u0431\u044a\u0435\u043a\u0442\u0430, \u043d\u0435\u0441\u043e\u0445\u0440\u0430\u043d\u0438\u0432\u0448\u0435\u0433\u043e\u0441\u044f \u0434\u043e \u043d\u0430\u0448\u0438\u0445 \u0434\u043d\u0435\u0439. \u0412\u0430\u043c \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u044d\u0442\u043e\u0442 \u043e\u0431\u044a\u0435\u043a\u0442, \u0430 \u0442\u0430\u043a\u0436\u0435 \u043f\u043e\u0434\u0442\u0432\u0435\u0440\u0434\u0438\u0442\u044c \u0438\u043b\u0438 \u043e\u043f\u0440\u043e\u0432\u0435\u0440\u0433\u043d\u0443\u0442\u044c \u0442\u0443 \u0438\u043b\u0438 \u0438\u043d\u0443\u044e \u043d\u0430\u0443\u0447\u043d\u0443\u044e \u0433\u0438\u043f\u043e\u0442\u0435\u0437\u0443: \u043e\u0431 \u043e\u0441\u043e\u0431\u0435\u043d\u043d\u043e\u0441\u0442\u044f\u0445 \u0441\u0442\u0440\u0443\u043a\u0442\u0443\u0440\u044b \u043e\u0431\u044a\u0435\u043a\u0442\u0430, \u0435\u0433\u043e \u0444\u0443\u043d\u043a\u0446\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u043e\u043c \u043d\u0430\u0437\u043d\u0430\u0447\u0435\u043d\u0438\u0438 \u0438 \u0442.\u043f. \u0412 \u0434\u0430\u043d\u043d\u043e\u043c \u0441\u043b\u0443\u0447\u0430\u0435, \u0432\u044b \u0431\u0443\u0434\u0435\u0442\u0435 \u0440\u0430\u0431\u043e\u0442\u0430\u0442\u044c \u0441 \u2026 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0435\u0439. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_1", "choice_correct": ["choice_1"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u043d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u043e\u0439"], "answer": "\u043d\u0430\u0443\u0447\u043d\u043e-\u043e\u0431\u043e\u0441\u043d\u043e\u0432\u0430\u043d\u043d\u043e\u0439", "correct": true}, "e4116937a01a4c87894309308dacfa4e_7_1": {"choices_map": {"choice_0": "\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u043e\u0442\u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u0442\u044c QR-\u043a\u043e\u0434 \u0438 \u043c\u043e\u043c\u0435\u043d\u0442\u0430\u043b\u044c\u043d\u043e \u043f\u043e\u043b\u0443\u0447\u0438\u0442\u044c \u0434\u043e\u043f\u043e\u043b\u043d\u0438\u0442\u0435\u043b\u044c\u043d\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e (\u0430\u043d\u043d\u043e\u0442\u0430\u0446\u0438\u0438 \u0438 \u0430\u0443\u0434\u0438\u043e\u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b)", "choice_1": "\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0441\u043a\u0430\u0447\u0430\u0442\u044c \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0435 \u0441 \u0430\u043b\u0444\u0430\u0432\u0438\u0442\u043d\u044b\u043c \u043a\u0430\u0442\u0430\u043b\u043e\u0433\u043e\u043c \u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0435\u043d\u0438\u0439 \u0438 \u0445\u0443\u0434\u043e\u0436\u043d\u0438\u043a\u043e\u0432", "choice_2": "\u0421\u043e\u0434\u0435\u0440\u0436\u0438\u0442 \u043a\u0440\u0430\u0442\u043a\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043c\u0443\u0437\u0435\u0439\u043d\u043e\u043c \u043a\u043e\u043c\u043f\u043b\u0435\u043a\u0441\u0435, \u043a\u043e\u043d\u0442\u0430\u043a\u0442\u044b, \u0447\u0430\u0441\u044b \u0440\u0430\u0431\u043e\u0442\u044b \u0438 \u0438\u043d\u0442\u0435\u0440\u0430\u043a\u0442\u0438\u0432\u043d\u0443\u044e \u043a\u0430\u0440\u0442\u0443 \u0441 \u0443\u043a\u0430\u0437\u0430\u043d\u0438\u0435\u043c \u043e\u0431\u044a\u0435\u043a\u0442\u043e\u0432 \u0420\u0443\u0441\u0441\u043a\u043e\u0433\u043e \u043c\u0443\u0437\u0435\u044f"}, "answers_available": ["\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u043e\u0442\u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u0442\u044c QR-\u043a\u043e\u0434 \u0438 \u043c\u043e\u043c\u0435\u043d\u0442\u0430\u043b\u044c\u043d\u043e \u043f\u043e\u043b\u0443\u0447\u0438\u0442\u044c \u0434\u043e\u043f\u043e\u043b\u043d\u0438\u0442\u0435\u043b\u044c\u043d\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e (\u0430\u043d\u043d\u043e\u0442\u0430\u0446\u0438\u0438 \u0438 \u0430\u0443\u0434\u0438\u043e\u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b)", "\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0441\u043a\u0430\u0447\u0430\u0442\u044c \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0435 \u0441 \u0430\u043b\u0444\u0430\u0432\u0438\u0442\u043d\u044b\u043c \u043a\u0430\u0442\u0430\u043b\u043e\u0433\u043e\u043c \u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0435\u043d\u0438\u0439 \u0438 \u0445\u0443\u0434\u043e\u0436\u043d\u0438\u043a\u043e\u0432", "\u0421\u043e\u0434\u0435\u0440\u0436\u0438\u0442 \u043a\u0440\u0430\u0442\u043a\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043c\u0443\u0437\u0435\u0439\u043d\u043e\u043c \u043a\u043e\u043c\u043f\u043b\u0435\u043a\u0441\u0435, \u043a\u043e\u043d\u0442\u0430\u043a\u0442\u044b, \u0447\u0430\u0441\u044b \u0440\u0430\u0431\u043e\u0442\u044b \u0438 \u0438\u043d\u0442\u0435\u0440\u0430\u043a\u0442\u0438\u0432\u043d\u0443\u044e \u043a\u0430\u0440\u0442\u0443 \u0441 \u0443\u043a\u0430\u0437\u0430\u043d\u0438\u0435\u043c \u043e\u0431\u044a\u0435\u043a\u0442\u043e\u0432 \u0420\u0443\u0441\u0441\u043a\u043e\u0433\u043e \u043c\u0443\u0437\u0435\u044f"], "question": "6. \u041a\u0430\u043a \u0440\u0430\u0431\u043e\u0442\u0430\u0435\u0442 \u0441\u0430\u0439\u0442 \u0434\u043b\u044f \u043c\u043e\u0431\u0438\u043b\u044c\u043d\u044b\u0445 \u0443\u0441\u0442\u0440\u043e\u0439\u0441\u0442\u0432 \"\u0420\u0443\u0441\u0441\u043a\u0438\u0439 \u043c\u0443\u0437\u0435\u0439. \u0414\u043e\u043f\u043e\u043b\u043d\u0435\u043d\u043d\u0430\u044f \u0440\u0435\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c\"? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u043e\u0442\u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u0442\u044c QR-\u043a\u043e\u0434 \u0438 \u043c\u043e\u043c\u0435\u043d\u0442\u0430\u043b\u044c\u043d\u043e \u043f\u043e\u043b\u0443\u0447\u0438\u0442\u044c \u0434\u043e\u043f\u043e\u043b\u043d\u0438\u0442\u0435\u043b\u044c\u043d\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e (\u0430\u043d\u043d\u043e\u0442\u0430\u0446\u0438\u0438 \u0438 \u0430\u0443\u0434\u0438\u043e\u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b)"], "answer": "\u041f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u043e\u0442\u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u0442\u044c QR-\u043a\u043e\u0434 \u0438 \u043c\u043e\u043c\u0435\u043d\u0442\u0430\u043b\u044c\u043d\u043e \u043f\u043e\u043b\u0443\u0447\u0438\u0442\u044c \u0434\u043e\u043f\u043e\u043b\u043d\u0438\u0442\u0435\u043b\u044c\u043d\u0443\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e (\u0430\u043d\u043d\u043e\u0442\u0430\u0446\u0438\u0438 \u0438 \u0430\u0443\u0434\u0438\u043e\u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b)", "correct": true}, "e4116937a01a4c87894309308dacfa4e_4_1": {"choices_map": {"choice_0": "\u041f\u0440\u0438 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u043c\u043c\u0435\u0442\u0440\u0438\u0438", "choice_1": "\u041f\u0440\u0438 \u043b\u0430\u0437\u0435\u0440\u043d\u043e\u043c \u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0438", "choice_2": "\u041f\u0440\u0438 \u0430\u044d\u0440\u043e\u0444\u043e\u0442\u043e\u0441\u044a\u0435\u043c\u043a\u0435"}, "answers_available": ["\u041f\u0440\u0438 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u043c\u043c\u0435\u0442\u0440\u0438\u0438", "\u041f\u0440\u0438 \u043b\u0430\u0437\u0435\u0440\u043d\u043e\u043c \u0441\u043a\u0430\u043d\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0438", "\u041f\u0440\u0438 \u0430\u044d\u0440\u043e\u0444\u043e\u0442\u043e\u0441\u044a\u0435\u043c\u043a\u0435"], "question": "3. \u041f\u0440\u0438 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u043a\u0430\u043a\u043e\u0439 \u0442\u0435\u0445\u043d\u043e\u043b\u043e\u0433\u0438\u0438 \u0441\u043e\u0437\u0434\u0430\u043d\u0438\u044f \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0445 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0439, \u043e\u0441\u043d\u043e\u0432\u043d\u044b\u043c \u0438\u0441\u0442\u043e\u0447\u043d\u0438\u043a\u043e\u043c \u044f\u0432\u043b\u044f\u044e\u0442\u0441\u044f \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0438? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041f\u0440\u0438 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u043c\u043c\u0435\u0442\u0440\u0438\u0438"], "answer": "\u041f\u0440\u0438 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u043c\u043c\u0435\u0442\u0440\u0438\u0438", "correct": true}, "e4116937a01a4c87894309308dacfa4e_11_1": {"choices_map": {"choice_0": "3DF Zephyr", "choice_1": "SketchUp", "choice_2": "Blender", "choice_3": "3ds Max"}, "answers_available": ["3DF Zephyr", "SketchUp", "Blender", "3ds Max"], "question": "10. \u041a\u0430\u043a\u0438\u0435 \u0438\u0437 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u043d\u044b\u0445 \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c \u043f\u0440\u0435\u0434\u043d\u0430\u0437\u043d\u0430\u0447\u0435\u043d\u044b \u0434\u043b\u044f \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u043c\u043c\u0435\u0442\u0440\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u043e\u0431\u0440\u0430\u0431\u043e\u0442\u043a\u0438 \u0446\u0438\u0444\u0440\u043e\u0432\u044b\u0445 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u044b \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["3DF Zephyr"], "answer": "3DF Zephyr", "correct": true}, "e4116937a01a4c87894309308dacfa4e_12_1": {"choices_map": {"choice_0": "Recogito", "choice_1": "Autocad", "choice_2": "Deep dream generator", "choice_3": "Google \u041a\u0430\u0440\u0442\u0438\u043d\u043a\u0438"}, "answers_available": ["Recogito", "Autocad", "Deep dream generator", "Google \u041a\u0430\u0440\u0442\u0438\u043d\u043a\u0438"], "question": "11. \u0412\u0430\u043c \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e \u043f\u0440\u043e\u0432\u0435\u0441\u0442\u0438 \u0441\u0435\u043c\u0430\u043d\u0442\u0438\u0447\u0435\u0441\u043a\u0443\u044e \u0430\u043d\u043d\u043e\u0442\u0430\u0446\u0438\u044e \u043a\u0430\u0440\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0445 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439 \u2013 \u043e\u043f\u0440\u0435\u0434\u0435\u043b\u0438\u0442\u044c \u043d\u0430 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0438 \u0438 \u043f\u043e\u043c\u0435\u0442\u0438\u0442\u044c \u0433\u0435\u043e\u0433\u0440\u0430\u0444\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u043e\u0431\u044a\u0435\u043a\u0442\u044b. \u041a\u0430\u043a\u043e\u0439 \u0438\u0437 \u043f\u0440\u0438\u0432\u0435\u0434\u0435\u043d\u043d\u044b\u0445 \u0438\u043d\u0441\u0442\u0440\u0443\u043c\u0435\u043d\u0442\u043e\u0432 \u043f\u043e\u0437\u0432\u043e\u043b\u0438\u0442 \u0441\u0434\u0435\u043b\u0430\u0442\u044c \u044d\u0442\u043e \u0431\u0435\u0437 \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e\u0441\u0442\u0438 \u0438\u0437\u0443\u0447\u0430\u0442\u044c \u043a\u043e\u0434? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["Recogito"], "answer": "Recogito", "correct": true}, "e4116937a01a4c87894309308dacfa4e_6_1": {"choices_map": {"choice_0": "\u0412 \u0434\u043e\u0431\u0430\u0432\u043b\u0435\u043d\u0438\u0438 \u0438 \u0432\u043d\u0435\u0434\u0440\u0435\u043d\u0438\u0438 \u0432 \u0440\u0435\u0430\u043b\u044c\u043d\u0443\u044e \u0436\u0438\u0437\u043d\u044c, \u0432 \u0432\u043e\u0441\u043f\u0440\u0438\u044f\u0442\u0438\u0438 \u0447\u0435\u043b\u043e\u0432\u0435\u043a\u0430 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0438", "choice_1": "\u0412 \u0441\u043e\u0437\u0434\u0430\u043d\u0438\u0438 \u043d\u043e\u0432\u043e\u0433\u043e \u043d\u0435\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u044e\u0449\u0435\u0433\u043e \u043f\u0440\u043e\u0441\u0442\u0440\u0430\u043d\u0441\u0442\u0432\u0430, \u043c\u0438\u0440\u0430", "choice_2": "\u0412 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u0441\u043f\u0435\u0446\u0438\u0430\u043b\u044c\u043d\u044b\u0445 VR-\u043e\u0447\u043a\u043e\u0432", "choice_3": "\u0412 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u0432 \u043e\u0431\u0440\u0430\u0437\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0445 \u0446\u0435\u043b\u044f\u0445 \u0438 \u043d\u0430\u0443\u043a\u0435"}, "answers_available": ["\u0412 \u0434\u043e\u0431\u0430\u0432\u043b\u0435\u043d\u0438\u0438 \u0438 \u0432\u043d\u0435\u0434\u0440\u0435\u043d\u0438\u0438 \u0432 \u0440\u0435\u0430\u043b\u044c\u043d\u0443\u044e \u0436\u0438\u0437\u043d\u044c, \u0432 \u0432\u043e\u0441\u043f\u0440\u0438\u044f\u0442\u0438\u0438 \u0447\u0435\u043b\u043e\u0432\u0435\u043a\u0430 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0438", "\u0412 \u0441\u043e\u0437\u0434\u0430\u043d\u0438\u0438 \u043d\u043e\u0432\u043e\u0433\u043e \u043d\u0435\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u044e\u0449\u0435\u0433\u043e \u043f\u0440\u043e\u0441\u0442\u0440\u0430\u043d\u0441\u0442\u0432\u0430, \u043c\u0438\u0440\u0430", "\u0412 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u0441\u043f\u0435\u0446\u0438\u0430\u043b\u044c\u043d\u044b\u0445 VR-\u043e\u0447\u043a\u043e\u0432", "\u0412 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u0432 \u043e\u0431\u0440\u0430\u0437\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0445 \u0446\u0435\u043b\u044f\u0445 \u0438 \u043d\u0430\u0443\u043a\u0435"], "question": "5. \u0412 \u0447\u0435\u043c \u043e\u0442\u043b\u0438\u0447\u0438\u0435 \u0434\u043e\u043f\u043e\u043b\u043d\u0435\u043d\u043d\u043e\u0439 \u0440\u0435\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u0438 \u043e\u0442 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_3", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0412 \u0434\u043e\u0431\u0430\u0432\u043b\u0435\u043d\u0438\u0438 \u0438 \u0432\u043d\u0435\u0434\u0440\u0435\u043d\u0438\u0438 \u0432 \u0440\u0435\u0430\u043b\u044c\u043d\u0443\u044e \u0436\u0438\u0437\u043d\u044c, \u0432 \u0432\u043e\u0441\u043f\u0440\u0438\u044f\u0442\u0438\u0438 \u0447\u0435\u043b\u043e\u0432\u0435\u043a\u0430 \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0438"], "answer": "\u0412 \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043d\u0438\u0438 \u0432 \u043e\u0431\u0440\u0430\u0437\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u043d\u044b\u0445 \u0446\u0435\u043b\u044f\u0445 \u0438 \u043d\u0430\u0443\u043a\u0435", "correct": false}, "e4116937a01a4c87894309308dacfa4e_16_1": {"choices_map": {"choice_0": "\u041c\u0435\u0442\u043e\u0434\u044b \u043c\u0430\u0448\u0438\u043d\u043d\u043e\u0433\u043e \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u044f \u0438 \u0438\u043d\u0442\u0435\u043b\u043b\u0435\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0433\u043e \u0430\u043d\u0430\u043b\u0438\u0437\u0430 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439", "choice_1": "\u041f\u043e\u0438\u0441\u043a \u043f\u043e \u043a\u0430\u0440\u0442\u0438\u043d\u043a\u0435 Google", "choice_2": "\u041a\u043e\u043d\u0442\u0435\u043d\u0442-\u0430\u043d\u0430\u043b\u0438\u0437 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439", "choice_3": "\u041b\u0438\u043d\u0435\u0439\u043d\u044b\u0439 \u0434\u0438\u0441\u043a\u0440\u0438\u043c\u0438\u043d\u0430\u043d\u0442\u043d\u044b\u0439 \u0430\u043d\u0430\u043b\u0438\u0437"}, "answers_available": ["\u041c\u0435\u0442\u043e\u0434\u044b \u043c\u0430\u0448\u0438\u043d\u043d\u043e\u0433\u043e \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u044f \u0438 \u0438\u043d\u0442\u0435\u043b\u043b\u0435\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0433\u043e \u0430\u043d\u0430\u043b\u0438\u0437\u0430 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439", "\u041f\u043e\u0438\u0441\u043a \u043f\u043e \u043a\u0430\u0440\u0442\u0438\u043d\u043a\u0435 Google", "\u041a\u043e\u043d\u0442\u0435\u043d\u0442-\u0430\u043d\u0430\u043b\u0438\u0437 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439", "\u041b\u0438\u043d\u0435\u0439\u043d\u044b\u0439 \u0434\u0438\u0441\u043a\u0440\u0438\u043c\u0438\u043d\u0430\u043d\u0442\u043d\u044b\u0439 \u0430\u043d\u0430\u043b\u0438\u0437"], "question": "15. \u0412\u0430\u043c \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e \u043f\u0440\u043e\u0430\u043d\u0430\u043b\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0438 \u043a\u043b\u0430\u0441\u0441\u0438\u0444\u0438\u0446\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043d\u0435\u0441\u043a\u043e\u043b\u044c\u043a\u043e \u0441\u043e\u0442\u0435\u043d \u0442\u044b\u0441\u044f\u0447 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439 \u043f\u043e \u0441\u0442\u0438\u043b\u044e. \u041a\u0430\u043a\u043e\u0439 \u043c\u0435\u0442\u043e\u0434 \u0432\u044b \u0431\u0443\u0434\u0435\u0442\u0435  \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u0442\u044c \u0434\u043b\u044f \u0440\u0435\u0448\u0435\u043d\u0438\u044f \u0434\u0430\u043d\u043d\u043e\u0439 \u0437\u0430\u0434\u0430\u0447\u0438? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041c\u0435\u0442\u043e\u0434\u044b \u043c\u0430\u0448\u0438\u043d\u043d\u043e\u0433\u043e \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u044f \u0438 \u0438\u043d\u0442\u0435\u043b\u043b\u0435\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0433\u043e \u0430\u043d\u0430\u043b\u0438\u0437\u0430 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439"], "answer": "\u041a\u043e\u043d\u0442\u0435\u043d\u0442-\u0430\u043d\u0430\u043b\u0438\u0437 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0439", "correct": false}, "e4116937a01a4c87894309308dacfa4e_3_1": {"choices_map": {"choice_0": "\u043d\u0430\u0431\u043e\u0440\u0430 \u0442\u043e\u0447\u0435\u043a", "choice_1": "\u0447\u0435\u0440\u0442\u0435\u0436\u0430", "choice_2": "\u043c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u0444\u043e\u0440\u043c\u0443\u043b\u044b", "choice_3": "\u043d\u0430\u0431\u043e\u0440 \u0433\u0435\u043e\u043c\u0435\u0442\u0440\u0438\u0447\u0435\u0441\u043a\u0438\u0445 \u043f\u0440\u0438\u043c\u0438\u0442\u0438\u0432\u043e\u0432"}, "answers_available": ["\u043d\u0430\u0431\u043e\u0440\u0430 \u0442\u043e\u0447\u0435\u043a", "\u0447\u0435\u0440\u0442\u0435\u0436\u0430", "\u043c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u0444\u043e\u0440\u043c\u0443\u043b\u044b", "\u043d\u0430\u0431\u043e\u0440 \u0433\u0435\u043e\u043c\u0435\u0442\u0440\u0438\u0447\u0435\u0441\u043a\u0438\u0445 \u043f\u0440\u0438\u043c\u0438\u0442\u0438\u0432\u043e\u0432"], "question": "2. \u0420\u0430\u0441\u0442\u0440\u043e\u0432\u0430\u044f \u0433\u0440\u0430\u0444\u0438\u043a\u0430 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u044f\u0435\u0442 \u0438\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u044f \u0432 \u0432\u0438\u0434\u0435 \u2026. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u043d\u0430\u0431\u043e\u0440\u0430 \u0442\u043e\u0447\u0435\u043a"], "answer": "\u043d\u0430\u0431\u043e\u0440\u0430 \u0442\u043e\u0447\u0435\u043a", "correct": true}, "e4116937a01a4c87894309308dacfa4e_9_1": {"choices_map": {"choice_0": "4D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "choice_1": "3D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"}, "answers_available": ["4D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "3D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"], "question": "8. \u041f\u0440\u0438\u043c\u0435\u0440\u043e\u043c \u043a\u0430\u043a\u043e\u0433\u043e \u0432\u0438\u0434\u0430 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438 \u044f\u0432\u043b\u044f\u0435\u0442\u0441\u044f \u043f\u0440\u043e\u0435\u043a\u0442 Rome Reborn (www.romereborn.org)? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_1", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["4D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438"], "answer": "3D \u0432\u0438\u0440\u0442\u0443\u0430\u043b\u044c\u043d\u043e\u0439 \u0440\u0435\u043a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0438\u0438", "correct": false}, "e4116937a01a4c87894309308dacfa4e_14_1": {"choices_map": {"choice_0": "\u041c\u0443\u043b\u044c\u0442\u0438\u043c\u0435\u0434\u0438\u0439\u043d\u044b\u0439 \u043a\u043e\u0440\u043f\u0443\u0441 \u041d\u041a\u0420\u042f, www.ruscorpora.ru", "choice_1": "\u0418\u0441\u0442\u043e\u0440\u0438\u044f \u0420\u043e\u0441\u0441\u0438\u0438 \u0432 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u044f\u0445, russiainphoto.ru", "choice_2": "Gesture Atlas, biblhertz.github.io/atlas", "choice_3": "\u041a\u0438\u043d\u043e\u041f\u043e\u0438\u0441\u043a, www.kinopoisk.ru"}, "answers_available": ["\u041c\u0443\u043b\u044c\u0442\u0438\u043c\u0435\u0434\u0438\u0439\u043d\u044b\u0439 \u043a\u043e\u0440\u043f\u0443\u0441 \u041d\u041a\u0420\u042f, www.ruscorpora.ru", "\u0418\u0441\u0442\u043e\u0440\u0438\u044f \u0420\u043e\u0441\u0441\u0438\u0438 \u0432 \u0444\u043e\u0442\u043e\u0433\u0440\u0430\u0444\u0438\u044f\u0445, russiainphoto.ru", "Gesture Atlas, biblhertz.github.io/atlas", "\u041a\u0438\u043d\u043e\u041f\u043e\u0438\u0441\u043a, www.kinopoisk.ru"], "question": "13. \u041a\u0430\u043a\u043e\u0439 \u0438\u0437 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u043d\u044b\u0445 \u0440\u0435\u0441\u0443\u0440\u0441\u043e\u0432 \u0441\u043e\u0434\u0435\u0440\u0436\u0438\u0442 \u0444\u0440\u0430\u0433\u043c\u0435\u043d\u0442\u044b \u043a\u0438\u043d\u043e\u0444\u0438\u043b\u044c\u043c\u043e\u0432 \u0437\u0430 \u043d\u0435\u0441\u043a\u043e\u043b\u044c\u043a\u043e \u0434\u0435\u0441\u044f\u0442\u0438\u043b\u0435\u0442\u0438\u0439 \u0441 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u044c\u044e \u043f\u043e\u0438\u0441\u043a\u0430 \u043d\u0435 \u0442\u043e\u043b\u044c\u043a\u043e \u043f\u043e \u043f\u0440\u043e\u0438\u0437\u043d\u043e\u0441\u0438\u043c\u043e\u043c\u0443 \u0442\u0435\u043a\u0441\u0442\u0443, \u043d\u043e \u0438 \u043f\u043e \u0436\u0435\u0441\u0442\u0430\u043c, \u0442\u0438\u043f\u0443 \u0440\u0435\u0447\u0435\u0432\u043e\u0433\u043e \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044f \u0438 \u0442.\u043f. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430 \u0438\u0437 \u043f\u0435\u0440\u0435\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044b\u0445.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041c\u0443\u043b\u044c\u0442\u0438\u043c\u0435\u0434\u0438\u0439\u043d\u044b\u0439 \u043a\u043e\u0440\u043f\u0443\u0441 \u041d\u041a\u0420\u042f, www.ruscorpora.ru"], "answer": "\u041c\u0443\u043b\u044c\u0442\u0438\u043c\u0435\u0434\u0438\u0439\u043d\u044b\u0439 \u043a\u043e\u0440\u043f\u0443\u0441 \u041d\u041a\u0420\u042f, www.ruscorpora.ru", "correct": true}, "e4116937a01a4c87894309308dacfa4e_2_1": {"choices_map": {}, "answers_available": [], "question": "1. \u0418\u0441\u043f\u043e\u043b\u044c\u0437\u0443\u044f \u043f\u043e\u0438\u0441\u043a\u043e\u0432\u044b\u0435 \u0441\u0438\u0441\u0442\u0435\u043c\u044b, \u043d\u0430\u0439\u0434\u0438\u0442\u0435 \u0430\u0432\u0442\u043e\u0440\u0430 \u0438 \u043d\u0430\u0437\u0432\u0430\u043d\u0438\u0435 \u043a\u0430\u0440\u0442\u0438\u043d\u044b. \u0412 \u043e\u0442\u0432\u0435\u0442 \u0437\u0430\u043f\u0438\u0448\u0438\u0442\u0435 \u0444\u0430\u043c\u0438\u043b\u0438\u044e \u0445\u0443\u0434\u043e\u0436\u043d\u0438\u043a\u0430 \u0438 \u043d\u0430\u0437\u0432\u0430\u043d\u0438\u0435 \u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0435\u043d\u0438\u044f \u0447\u0435\u0440\u0435\u0437 \u0437\u0430\u043f\u044f\u0442\u0443\u044e, \u043d\u0430\u043f\u0440\u0438\u043c\u0435\u0440, \u0412\u0430\u0441\u043d\u0435\u0446\u043e\u0432, \u0411\u043e\u0433\u0430\u0442\u044b\u0440\u0438.\n  <p/>\n  <p><img src=\"/static/test_9_1.jpg\" width=\"600\"/></p>\n  <p/>\n  <p>\u0418\u0437\u043e\u0431\u0440\u0430\u0436\u0435\u043d\u0438\u0435 \u0438\u0437 \u0412\u0438\u043a\u0438\u043f\u0435\u0434\u0438\u0438</p>", "input_type": "textline", "variant": "", "group_label": "", "choice": [], "choice_correct": [], "response_type": "stringresponse", "answer_correct": ["\u0421\u0430\u0432\u0440\u0430\u0441\u043e\u0432, \u0421\u0435\u043b\u044c\u0441\u043a\u0438\u0439 \u0432\u0438\u0434", "\u0421\u0410\u0412\u0420\u0410\u0421\u041e\u0412, \u0421\u0415\u041b\u042c\u0421\u041a\u0418\u0419 \u0412\u0418\u0414", "\u0441\u0430\u0432\u0440\u0430\u0441\u043e\u0432, \u0441\u0435\u043b\u044c\u0441\u043a\u0438\u0439 \u0432\u0438\u0434"], "answer": "\u0421\u0430\u0432\u0440\u0430\u0441\u043e\u0432, \u0421\u0435\u043b\u044c\u0441\u043a\u0438\u0439 \u0432\u0438\u0434", "correct": true}}, "success": "incorrect", "grade": 9, "correct_map": {"e4116937a01a4c87894309308dacfa4e_8_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_15_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_5_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_10_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_7_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_4_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_11_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_12_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_6_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_16_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_2_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_3_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_13_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_14_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "e4116937a01a4c87894309308dacfa4e_9_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}}, "state": {"correct_map": {}, "input_state": {"e4116937a01a4c87894309308dacfa4e_8_1": {}, "e4116937a01a4c87894309308dacfa4e_15_1": {}, "e4116937a01a4c87894309308dacfa4e_5_1": {}, "e4116937a01a4c87894309308dacfa4e_10_1": {}, "e4116937a01a4c87894309308dacfa4e_9_1": {}, "e4116937a01a4c87894309308dacfa4e_7_1": {}, "e4116937a01a4c87894309308dacfa4e_4_1": {}, "e4116937a01a4c87894309308dacfa4e_11_1": {}, "e4116937a01a4c87894309308dacfa4e_12_1": {}, "e4116937a01a4c87894309308dacfa4e_6_1": {}, "e4116937a01a4c87894309308dacfa4e_16_1": {}, "e4116937a01a4c87894309308dacfa4e_3_1": {}, "e4116937a01a4c87894309308dacfa4e_13_1": {}, "e4116937a01a4c87894309308dacfa4e_14_1": {}, "e4116937a01a4c87894309308dacfa4e_2_1": {}}, "has_saved_answers": null, "seed": 1, "done": null, "student_answers": {}}, "answers": {"e4116937a01a4c87894309308dacfa4e_8_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_15_1": "choice_2", "e4116937a01a4c87894309308dacfa4e_5_1": "choice_2", "e4116937a01a4c87894309308dacfa4e_10_1": ["choice_0", "choice_2"], "e4116937a01a4c87894309308dacfa4e_9_1": "choice_1", "e4116937a01a4c87894309308dacfa4e_7_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_4_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_11_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_12_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_6_1": "choice_3", "e4116937a01a4c87894309308dacfa4e_16_1": "choice_2", "e4116937a01a4c87894309308dacfa4e_3_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_13_1": "choice_1", "e4116937a01a4c87894309308dacfa4e_14_1": "choice_0", "e4116937a01a4c87894309308dacfa4e_2_1": "\u0421\u0430\u0432\u0440\u0430\u0441\u043e\u0432, \u0421\u0435\u043b\u044c\u0441\u043a\u0438\u0439 \u0432\u0438\u0434"}, "attempts": 1, "max_grade": 15, "problem_id": "block-v1:hse+DIGHIST+spring_2020+type@problem+block@e4116937a01a4c87894309308dacfa4e"}, "event_source": "server", "context": {"course_user_tags": {}, "user_id": 1818869, "org_id": "hse", "asides": {}, "module": {"usage_key": "block-v1:hse+DIGHIST+spring_2020+type@problem+block@e4116937a01a4c87894309308dacfa4e", "display_name": "\u0422\u0435\u0441\u0442"}, "course_id": "course-v1:hse+DIGHIST+spring_2020", "path": "/courses/course-v1:hse+DIGHIST+spring_2020/xblock/block-v1:hse+DIGHIST+spring_2020+type@problem+block@e4116937a01a4c87894309308dacfa4e/handler/xmodule_handler/problem_check"}, "time": "2020-04-10T04:10:56.174582+00:00", "page": "x_module"}'
d = json.loads(s)
for k in d['choices_map'].keys():
    print(k)

KeyError: 'choices_map'

In [23]:
# This is a pretty print to file code
s = r'{"username": "asklemeshov", "event_source": "server", "name": "problem_publish", "accept_language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7", "time": "2020-04-15T09:17:48.145534+00:00", "agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36", "page": null, "host": "studio.openedu.ru", "session": "b378801b19c65516eb519c84b00f989b", "referer": "https://studio.openedu.ru/container/block-v1:hse+CINCULT+spring_2020+type@vertical+block@164bfcf3bb4e4a37a804378e4dca55b5", "context": {"user_id": 700336, "org_id": "", "course_id": "", "path": "/xblock/block-v1:hse+CINCULT+spring_2020+type@vertical+block@164bfcf3bb4e4a37a804378e4dca55b5"}, "ip": "46.39.231.209", "event": {"markdown": ">>1. \u041a\u0430\u043a\u0438\u0435 \u0441\u0442\u0440\u0430\u043d\u044b \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u043d\u0430\u0441\u0442\u043e\u044f\u0449\u0435\u0433\u043e \u043a\u0443\u0440\u0441\u0430 \u0440\u0430\u0441\u0441\u043c\u0430\u0442\u0440\u0438\u0432\u0430\u044e\u0442\u0441\u044f \u0432 \u0431\u043b\u043e\u043a\u0435, \u043f\u043e\u0441\u0432\u044f\u0449\u0435\u043d\u043d\u043e\u043c\u0443 \u0417\u0430\u043f\u0430\u0434\u043d\u043e\u0439 \u0415\u0432\u0440\u043e\u043f\u0435? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0434\u0432\u0430 \u043f\u0440\u0430\u0432\u0438\u043b\u044c\u043d\u044b\u0445 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u0430 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n[x] \u0411\u0435\u043d\u0438\u043b\u044e\u043a\u0441\u0430\n[] \u0410\u0432\u0441\u0442\u0440\u0438\u0438\n[x] \u041f\u0438\u0440\u0435\u043d\u0435\u0439\u0441\u043a\u043e\u0433\u043e \u043f\u043e\u043b\u0443\u043e\u0441\u0442\u0440\u043e\u0432\u0430\n[] \u0412\u0435\u043d\u0433\u0440\u0438\u0438\n\n>>2. \u041a\u0430\u043a\u0438\u0435 \u0438\u0437 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u043d\u044b\u0445 \u0432\u0435\u0445 \u0432 \u043a\u0438\u043d\u043e \u0418\u0441\u043f\u0430\u043d\u0438\u0438 \u041d\u0415 \u043e\u0442\u043d\u043e\u0441\u044f\u0442\u0441\u044f \u043a \u044d\u0442\u043e\u0439 \u0441\u0442\u0440\u0430\u043d\u0435? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0434\u0432\u0430 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u0430 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n[x] \u041e\u0442 \u0421\u0442\u043e\u0440\u043a\u0430 \u0434\u043e \u0431\u0440\u0430\u0442\u044c\u0435\u0432 \u0414\u0430\u0440\u0434\u0435\u043d\u043d\n[] \u0410\u0432\u0430\u043d\u0433\u0430\u0440\u0434 (\u0411\u0443\u043d\u044e\u044d\u043b\u044c) \n[] \u041f\u043e\u0441\u0442\u0444\u0440\u0430\u043d\u043a\u0438\u0441\u0442\u0441\u043a\u043e\u0435 \u043a\u0438\u043d\u043e\n[x] \u041e\u0442 \u0420\u0438\u0431\u0435\u0439\u0440\u044b \u0434\u043e \u041e\u043b\u0438\u0432\u0435\u0439\u0440\u044b\n\n>>3. \u0412 \u043a\u0430\u043a\u0438\u0435 \u0433\u043e\u0434\u044b \u0432 \u041d\u0438\u0434\u0435\u0440\u043b\u0430\u043d\u0434\u0430\u0445 \u043d\u0430\u0447\u0438\u043d\u0430\u044e\u0442 \u043c\u0430\u0441\u0441\u043e\u0432\u043e \u043e\u0442\u043a\u0440\u044b\u0432\u0430\u0442\u044c\u0441\u044f \u043a\u0438\u043d\u043e\u0442\u0435\u0430\u0442\u0440\u044b? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() 1890-\u0435\n() 1900-\u0435\n() 1910-\u0435\n(x) 1920-\u0435 \n\n>>4. \u041a\u0430\u043a\u0438\u043c \u0444\u0438\u043b\u044c\u043c\u043e\u043c \u0412\u0441\u0435\u0432\u043e\u043b\u043e\u0434\u0430 \u041f\u0443\u0434\u043e\u0432\u043a\u0438\u043d\u0430 \u0432 1926 \u0433\u043e\u0434\u0443 \u0431\u044b\u043b \u0432\u0434\u043e\u0445\u043d\u043e\u0432\u043b\u0435\u043d \u0419\u043e\u0440\u0438\u0441 \u0418\u0432\u0435\u043d\u0441, \u0447\u0442\u043e \u0440\u0435\u0448\u0438\u043b \u043e\u0441\u043d\u043e\u0432\u0430\u0442\u044c \u043e\u0431\u0449\u0435\u0441\u0442\u0432\u043e \u043a\u0438\u043d\u043e\u043b\u044e\u0431\u0438\u0442\u0435\u043b\u0435\u0439 \u0430\u0432\u0430\u043d\u0433\u0430\u0440\u0434\u0430? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n(x) \u041c\u0430\u0442\u044c\n() \u041a\u043e\u043d\u0435\u0446 \u0421\u0430\u043d\u043a\u0442-\u041f\u0435\u0442\u0435\u0440\u0431\u0443\u0440\u0433\u0430\n() \u041c\u0435\u0445\u0430\u043d\u0438\u043a\u0430 \u0433\u043e\u043b\u043e\u0432\u043d\u043e\u0433\u043e \u043c\u043e\u0437\u0433\u0430\n() \u0428\u0430\u0445\u043c\u0430\u0442\u043d\u0430\u044f \u0433\u043e\u0440\u044f\u0447\u043a\u0430\n\n>>5. \u041a\u0430\u043a \u043d\u0430\u0437\u044b\u0432\u0430\u0435\u0442\u0441\u044f \u0444\u0438\u043b\u044c\u043c \u0419\u043e\u0440\u0438\u0441\u0430 \u0418\u0432\u0435\u043d\u0441\u0430, \u0433\u0434\u0435 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0440\u0430\u0441\u043a\u0440\u044b\u043b\u0441\u044f \u043a\u0430\u043a \u043f\u0440\u043e\u0434\u043e\u043b\u0436\u0430\u0442\u0435\u043b\u044c \u0441\u0442\u0438\u043b\u0438\u0441\u0442\u0438\u043a\u0438 \u0412\u0430\u043b\u044c\u0442\u0435\u0440\u0430 \u0420\u0443\u0442\u0442\u043c\u0430\u043d\u0430 \u0438 \u0414\u0437\u0438\u0433\u0438 \u0412\u0435\u0440\u0442\u043e\u0432\u0430? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() \u0412\u0438\u0433\u0432\u0430\u043c (1911)\n(x) \u0414\u043e\u0436\u0434\u044c (1929)\n() \u041f\u0435\u0441\u043d\u044c \u043e \u0433\u0435\u0440\u043e\u044f\u0445 (1932)\n() \u0418\u0441\u043f\u0430\u043d\u0441\u043a\u0430\u044f \u0437\u0435\u043c\u043b\u044f (1937) \n\n>>6. \u041e \u043a\u0430\u043a\u043e\u043c \u0444\u0438\u043b\u044c\u043c\u0435 \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab\u0412 \u0441\u0442\u0430\u0440\u043e\u0439 \u0447\u0430\u0441\u0442\u0438 \u043d\u0438\u0434\u0435\u0440\u043b\u0430\u043d\u0434\u0441\u043a\u043e\u0439 \u0441\u0442\u043e\u043b\u0438\u0446\u044b \u043e\u0440\u0443\u0434\u0443\u0435\u0442 \u043c\u0430\u043d\u044c\u044f\u043a, \u0441\u0443\u0434\u044f \u043f\u043e \u043f\u043e\u0447\u0435\u0440\u043a\u0443, \u0432\u043e\u0434\u043e\u043b\u0430\u0437. \u0415\u043c\u0443 \u0443\u0434\u0430\u0435\u0442\u0441\u044f \u0443\u0445\u043e\u0434\u0438\u0442\u044c \u043b\u0438\u0448\u044c \u043e\u0434\u043d\u043e\u043c\u0443 \u0435\u043c\u0443 \u0438\u0437\u0432\u0435\u0441\u0442\u043d\u044b\u043c\u0438 \u0432\u043e\u0434\u043d\u044b\u043c\u0438 \u0430\u0440\u0442\u0435\u0440\u0438\u044f\u043c\u0438. \u041e\u0434\u043d\u0430 \u0438\u0437 \u043a\u0443\u043b\u044c\u043c\u0438\u043d\u0430\u0446\u0438\u043e\u043d\u043d\u044b\u0445 \u0441\u0446\u0435\u043d \u0444\u0438\u043b\u044c\u043c\u0430 \u2014 \u043f\u044f\u0442\u0438\u043c\u0438\u043d\u0443\u0442\u043d\u0430\u044f \u0441\u044a\u0435\u043c\u043a\u0430 \u043f\u043e\u0433\u043e\u043d\u0438 \u043d\u0430 \u0441\u043a\u0443\u0442\u0435\u0440\u0430\u0445 \u043d\u0430 100% \u0443\u043d\u0438\u043a\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043a\u0442\u0443\u0440\u0435 \u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c\u0430\u2026 \u0414\u0430\u0436\u0435 \u0441\u043f\u0443\u0441\u0442\u044f 30 \u043b\u0435\u0442, \u044d\u0442\u0443 \u043f\u043e\u0433\u043e\u043d\u044e \u0432\u043a\u043b\u044e\u0447\u0430\u044e\u0442 \u0432\u043e \u0432\u0441\u0435\u0432\u043e\u0437\u043c\u043e\u0436\u043d\u044b\u0435 \u0442\u043e\u043f\u044b \u0438 \u0443\u043f\u043e\u043c\u0438\u043d\u0430\u044e\u0442 \u043a\u0430\u043a \u043e\u0431\u0440\u0430\u0437\u0446\u043e\u0432\u044b\u0439 \u043f\u0440\u0438\u043c\u0435\u0440 \u0440\u0430\u0431\u043e\u0442\u044b \u0441 \u0435\u0441\u0442\u0435\u0441\u0442\u0432\u0435\u043d\u043d\u044b\u043c\u0438 \u0434\u0435\u043a\u043e\u0440\u0430\u0446\u0438\u044f\u043c\u0438\u00bb.  \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() \u041c\u0435\u043b\u044c\u043d\u0438\u0446\u0430 \u043a\u0430\u043c\u0435\u043d\u043d\u044b\u0445 \u0436\u0435\u043d\u0449\u0438\u043d (1960)\n() \u041f\u0440\u043e\u0438\u0441\u0448\u0435\u0441\u0442\u0432\u0438\u0435 \u0432 \u0413\u043b\u044f\u0439\u0432\u0438\u0446\u0435 (1961)\n(x) \u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c\u0441\u043a\u0438\u0439 \u043a\u043e\u0448\u043c\u0430\u0440/\u041f\u0440\u043e\u043a\u043b\u044f\u0442\u044b\u0439 \u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c (1987)\n() \u0413\u043e\u043b\u043b\u0430\u043d\u0434\u0441\u043a\u0438\u0439 \u0441\u0432\u0435\u0442 (2003) \n\n>>7. \u041f\u043e\u0441\u043b\u0435 \u0432\u044b\u0445\u043e\u0434\u0430 \u043a\u0430\u043a\u043e\u0433\u043e \u0444\u0438\u043b\u044c\u043c\u0430 \u0431\u044b\u043b \u0443\u0431\u0438\u0442 \u0435\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u2014  \u0422\u0435\u043e \u0432\u0430\u043d \u0413\u043e\u0433? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430. <<\n() \u0421\u0435\u0432\u0435\u0440\u044f\u043d\u0435 (1992)\n() \u0414\u0435\u0441\u044f\u0442\u044c \u043c\u043e\u043d\u043e\u043b\u043e\u0433\u043e\u0432 \u0438\u0437 \u0436\u0438\u0437\u043d\u0438 \u0441\u0435\u0440\u0438\u0439\u043d\u044b\u0445 \u0443\u0431\u0438\u0439\u0446 (1994)\n() \u0424\u0440\u0430\u043d\u043a \u0438 \u0415\u0432\u0430 (1973)\n(x) \u041f\u043e\u043a\u043e\u0440\u043d\u043e\u0441\u0442\u044c (2004)\n\n>>8. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u00ab\u0431\u0435\u043b\u044c\u0433\u0438\u0439\u0441\u043a\u043e\u0433\u043e \u0419\u043e\u0440\u0438\u0441\u0430 \u0418\u0432\u0435\u043d\u0441\u0430\u00bb, \u0441\u043d\u0438\u043c\u0430\u0432\u0448\u0435\u0433\u043e \u0443\u0436\u0430\u0441\u0430\u044e\u0449\u0438\u0435 \u0443\u0441\u043b\u043e\u0432\u0438\u044f \u0436\u0438\u0437\u043d\u0438 \u0448\u0430\u0445\u0442\u0435\u0440\u043e\u0432? \u0412 \u0440\u0435\u0437\u0443\u043b\u044c\u0442\u0430\u0442\u0435 \u044d\u0442\u0438\u0445 \u0441\u044a\u0435\u043c\u043e\u043a \u0432\u044b\u0448\u0435\u043b \u0444\u0438\u043b\u044c\u043c \u041d\u0438\u0449\u0435\u0442\u0430 \u0432 \u0411\u043e\u0440\u0438\u043d\u0430\u0436\u0435 (1934). \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n(x) \u0410\u043d\u0440\u0438 \u0428\u0442\u043e\u0440\u043a\n() \u0410\u043d\u0440\u0438 \u0420\u0443\u0441\u0441\u043e\n() \u0410\u043d\u0434\u0440\u0435 \u0414\u0435\u043b\u044c\u0432\u043e\n() \u041b\u044e\u0441\u044c\u0435\u043d \u0414\u0435\u0440\u043e\u0438\u0441\n\n>>10. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0437\u0432\u0435\u0441\u0442\u043d\u044b\u0445 \u0431\u0435\u043b\u044c\u0433\u0438\u0439\u0441\u043a\u0438\u0445 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u043e\u0432, \u043a\u043e\u0442\u043e\u0440\u044b\u0435 \u044f\u0432\u043b\u044f\u044e\u0442\u0441\u044f \u0431\u0440\u0430\u0442\u044c\u044f\u043c\u0438 \u0438 \u0441\u043d\u0438\u043c\u0430\u044e\u0442 \u043e\u0431\u044b\u0447\u043d\u043e \u043e\u0441\u0442\u0440\u043e\u0441\u043e\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0435 \u043a\u0438\u043d\u043e? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() \u041f\u0438\u0442\u0435\u0440 \u0438 \u0411\u043e\u0431\u0431\u0438 \u0424\u0430\u0440\u0440\u0435\u043b\u043b\u0438\n() \u041c\u0430\u0440\u043a \u0438 \u041c\u0430\u0439\u043a\u043b \u041f\u043e\u043b\u0438\u0448\n() \u0414\u0436\u043e\u044d\u043b \u0438 \u0418\u0442\u0430\u043d \u041a\u043e\u044d\u043d\n(x) \u0416\u0430\u043d-\u041f\u044c\u0435\u0440 \u0438 \u041b\u044e\u043a \u0414\u0430\u0440\u0434\u0435\u043d\u043d\n\n>>11. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0430, \u043a\u043e\u0442\u043e\u0440\u044b\u0439 \u00ab\u0441\u0443\u043c\u0435\u043b \u043f\u0440\u0435\u043e\u0434\u043e\u043b\u0435\u0442\u044c \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u0443\u044e \u043f\u0435\u0440\u0438\u0444\u0435\u0440\u0438\u0439\u043d\u043e\u0441\u0442\u044c\u00bb? \u0421\u0430\u043c \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0433\u043e\u0432\u043e\u0440\u0438\u043b \u043e \u0441\u0432\u043e\u0438\u0445 \u0444\u0438\u043b\u044c\u043c\u0430\u0445 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0438\u043c \u043e\u0431\u0440\u0430\u0437\u043e\u043c: \u00ab\u042f \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u0443\u044e \u0433\u0440\u043e\u0442\u0435\u0441\u043a, \u043f\u0440\u043e\u0442\u0435\u0441\u0442 \u0438 \u0447\u0435\u0440\u043d\u044b\u0439 \u044e\u043c\u043e\u0440. \u041f\u0440\u0438\u0447\u0435\u043c \u043f\u043e\u0447\u0442\u0438 \u0432\u0441\u0435\u0433\u0434\u0430 \u043f\u043e\u043c\u0438\u043c\u043e \u0441\u043e\u0431\u0441\u0442\u0432\u0435\u043d\u043d\u043e\u0439 \u0432\u043e\u043b\u0438\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n(x) \u041b\u0443\u0438\u0441 \u0411\u0443\u043d\u044e\u044d\u043b\u044c \n() \u041f\u0435\u0434\u0440\u043e \u0410\u043b\u044c\u043c\u043e\u0434\u043e\u0432\u0430\u0440\n() \u0410\u043b\u0435\u0445\u0430\u043d\u0434\u0440\u043e \u0410\u043c\u0435\u043d\u0435\u0431\u0430\u0440\n() \u0425\u0443\u043b\u0438\u043e \u041c\u0435\u0434\u0435\u043c\n\n>>12. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u0433\u043e \u00ab\u043a\u043e\u0440\u043e\u043b\u044f \u044d\u043a\u0441\u043f\u043b\u0443\u0430\u0442\u0430\u0446\u0438\u043e\u043d\u043d\u043e\u0433\u043e \u043a\u0438\u043d\u043e\u00bb? \u042d\u0442\u043e\u0442 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0438\u0437\u0432\u0435\u0441\u0442\u0435\u043d \u0442\u0435\u043c, \u0447\u0442\u043e \u0437\u0430 \u0441\u0432\u043e\u044e \u0436\u0438\u0437\u043d\u044c \u0441\u043d\u044f\u043b 180 \u0444\u0438\u043b\u044c\u043c\u043e\u0432, \u043a\u043e\u0442\u043e\u0440\u043e\u0435 \u043c\u043e\u0436\u043d\u043e \u043e\u0442\u043d\u0435\u0441\u0442\u0438 \u043a \u0442\u0430\u043a\u0438\u043c \u043a\u0430\u0442\u0435\u0433\u043e\u0440\u0438\u044f\u043c \u044d\u043a\u0441\u043f\u043b\u0443\u0430\u0442\u0430\u0446\u0438\u043e\u043d\u043d\u043e\u0433\u043e \u043a\u0438\u043d\u0435\u043c\u0430\u0442\u043e\u0433\u0440\u0430\u0444\u0430, \u043a\u0430\u043a women in prison \u0438 sexploitation. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() \u0425\u043e\u0430\u043a\u0438\u043d \u041e\u0440\u0438\u0441\u0442\u0440\u0435\u043b\u044c\n() \u0421\u0438\u043b\u044c\u0432\u0438\u044f \u041c\u0443\u043d\u0442\n(x) \u0425\u0435\u0441\u0443\u0441 \u0424\u0440\u0430\u043d\u043a\u043e\n() \u041d\u0430\u0447\u043e \u0412\u0438\u0433\u0430\u043b\u043e\u043d\u0434\u043e\n\n>>13. \u041e \u043a\u0430\u043a\u043e\u043c \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u043c \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0435 \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab_____ \u041f\u0440\u0438\u0448\u0435\u043b \u0432 \u043a\u0438\u043d\u043e \u043f\u043e\u0441\u043b\u0435 \u043f\u0430\u0434\u0435\u043d\u0438\u044f \u0440\u0435\u0436\u0438\u043c\u0430 \u0424\u0440\u0430\u043d\u043a\u043e, \u043f\u0440\u0438\u0448\u0435\u043b \u043a\u0430\u043a \u043b\u044e\u0431\u0438\u0442\u0435\u043b\u044c, \u0440\u043e\u043a-\u043c\u0443\u0437\u044b\u043a\u0430\u043d\u0442, \u0436\u0443\u0440\u043d\u0430\u043b\u0438\u0441\u0442, \u043f\u0438\u0441\u0430\u0432\u0448\u0438\u0439 \u043f\u043e\u0434 \u0436\u0435\u043d\u0441\u043a\u0438\u043c \u043f\u0441\u0435\u0432\u0434\u043e\u043d\u0438\u043c\u043e\u043c. \u0423\u0436\u0435 \u043a \u0441\u0435\u0440\u0435\u0434\u0438\u043d\u0435 1980-\u0445 \u0433\u0433. \u043f\u043e\u0441\u043b\u0435 \u0442\u0430\u043a\u0438\u0445 \u0432\u0435\u043b\u0438\u043a\u0438\u0445 \u0444\u0438\u043b\u044c\u043c\u043e\u0432 \u043a\u0430\u043a \u041c\u0430\u0442\u0430\u0434\u043e\u0440 (1986), \u0417\u0430\u043a\u043e\u043d \u0436\u0435\u043b\u0430\u043d\u0438\u0439 (1987) \u0438 \u041b\u0430\u0431\u0438\u0440\u0438\u043d\u0442 \u0441\u0442\u0440\u0430\u0441\u0442\u0435\u0439 (1982), \u0438 \u043e\u0441\u043e\u0431\u0435\u043d\u043d\u043e \u044d\u0442\u043e \u043a\u0430\u0441\u0430\u0435\u0442\u0441\u044f \u043e\u0441\u044b\u043f\u0430\u043d\u043d\u043e\u0433\u043e \u0441\u043b\u0430\u0432\u043e\u0439 \u0444\u0438\u043b\u044c\u043c\u0430 \u0416\u0435\u043d\u0449\u0438\u043d\u0430 \u043d\u0430 \u0433\u0440\u0430\u043d\u0438 \u043d\u0435\u0440\u0432\u043d\u043e\u0433\u043e \u0441\u0440\u044b\u0432\u0430 (1988), ____ \u0441\u0442\u0430\u043d\u043e\u0432\u0438\u0442\u0441\u044f \u0431\u0440\u0435\u043d\u0434\u043e\u043c\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n\n(x) \u041f\u0435\u0434\u0440\u043e \u0410\u043b\u044c\u043c\u043e\u0434\u043e\u0432\u0430\u0440\n() \u0412\u0438\u043a\u0442\u043e\u0440 \u042d\u0440\u0438\u0441\u0435\n() \u0424\u0435\u0440\u043d\u0430\u043d\u0434\u043e \u0422\u0440\u0443\u044d\u0431\u0430\n() \u042d\u043b\u0438\u0430\u0441 \u041a\u0435\u0440\u0435\u0445\u0435\u0442\u0430\n\n>>14. \u041e \u043a\u0430\u043a\u043e\u043c \u0444\u0438\u043b\u044c\u043c\u0435 \u043f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u044c\u0441\u043a\u043e\u0439 \u00ab\u043d\u043e\u0432\u043e\u0439 \u0432\u043e\u043b\u043d\u044b\u00bb \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab\u0424\u0438\u043b\u044c\u043c \u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b\u0432\u0430\u0435\u0442 \u0438\u0441\u0442\u043e\u0440\u0438\u044e \u043d\u0435\u0434\u043e\u0443\u043c\u0435\u043d\u0438\u044f, \u043a\u043e\u0442\u043e\u0440\u043e\u0435 \u0438\u0441\u043f\u044b\u0442\u044b\u0432\u0430\u0435\u0442 \u043e\u0442 \u0431\u043e\u043b\u044c\u0448\u043e\u0433\u043e \u0433\u043e\u0440\u043e\u0434\u0430 \u043d\u0435\u0434\u0430\u0432\u043d\u043e \u043f\u0435\u0440\u0435\u0435\u0445\u0430\u0432\u0448\u0438\u0439 \u0438\u0437 \u043f\u0440\u043e\u0432\u0438\u043d\u0446\u0438\u0438. \u041e\u043d \u043c\u043d\u043e\u0433\u043e\u0435 \u0443\u043c\u0435\u0435\u0442 \u0434\u0435\u043b\u0430\u0442\u044c \u0440\u0443\u043a\u0430\u043c\u0438, \u043d\u043e \u043c\u0435\u0441\u0442\u043d\u0430\u044f \u0436\u0438\u0437\u043d\u044c \u0445\u0438\u0442\u0440\u0435\u0435, \u0435\u043c\u0443 \u043f\u0440\u0438\u0445\u043e\u0434\u0438\u0442\u0441\u044f \u043d\u0435 \u0440\u0430\u0437 \u0443\u0431\u0435\u0436\u0434\u0430\u0442\u044c\u0441\u044f \u0432 \u044d\u0442\u043e\u043c\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n() \u041a\u043e\u0440\u0435\u043d\u044c \u0441\u0435\u0440\u0434\u0446\u0430 (2000)\n(x) \u0417\u0435\u043b\u0451\u043d\u044b\u0435 \u0433\u043e\u0434\u044b (1963)\n() \u041e\u0441\u0442\u0440\u043e\u0432 \u043b\u044e\u0431\u0432\u0438 (1982)\n() \u041a\u0438\u043d\u043e \u043d\u0430\u0448\u0435\u0433\u043e \u0432\u0440\u0435\u043c\u0435\u043d\u0438 (\u0441\u0435\u0440\u0438\u0430\u043b, 1988 \u2013 ...) \n\n>>15. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u043f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u044c\u0441\u043a\u043e\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0430, \u0441\u043d\u0438\u043c\u0430\u0432\u0448\u0435\u0433\u043e \u0432 \u044d\u0441\u0442\u0435\u0442\u0438\u043a\u0435 docu-fiction? \u041e\u043f\u0438\u0440\u0430\u0439\u0442\u0435\u0441\u044c \u043f\u0440\u0438 \u0432\u044b\u0431\u043e\u0440\u0435 \u043e\u0442\u0432\u0435\u0442\u0430 \u043d\u0430 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0438\u0439 \u043e\u0442\u0440\u044b\u0432\u043e\u043a: \u00ab\u0427\u0435\u0440\u043d\u043e-\u0431\u0435\u043b\u044b\u0439 \u044d\u0441\u0442\u0435\u0442\u0441\u043a\u0438\u0439 \u0440\u0435\u043f\u043e\u0440\u0442\u0430\u0436 \u0432 \u0441\u043e\u043f\u0440\u043e\u0432\u043e\u0436\u0434\u0435\u043d\u0438\u0438 \u0437\u0430\u043a\u0430\u0434\u0440\u043e\u0432\u043e\u0433\u043e \u0433\u043e\u043b\u043e\u0441\u0430 \u043f\u0440\u0435\u0432\u0440\u0430\u0449\u0430\u0435\u0442\u0441\u044f \u0432 \u0432\u0438\u0437\u0438\u0442\u043d\u0443\u044e \u043a\u0430\u0440\u0442\u043e\u0447\u043a\u0443 \u0433\u043e\u0440\u043e\u0434\u0441\u043a\u043e\u0439 \u0434\u0440\u0430\u043c\u044b \u041f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u0438\u0438. \u041e\u0448\u0435\u043b\u043e\u043c\u0438\u0442\u0435\u043b\u044c\u043d\u044b\u0439 \u0443\u0441\u043f\u0435\u0445 ___ ____1989 \u043f\u043e\u0434 \u043f\u0440\u043e\u0441\u0442\u044b\u043c, \u043d\u043e \u043c\u043d\u043e\u0433\u043e\u0437\u043d\u0430\u0447\u043d\u044b\u043c \u043d\u0430\u0437\u0432\u0430\u043d\u0438\u0435\u043c \"\u041a\u0440\u043e\u0432\u044c\" \u043b\u0438\u0448\u0435\u043d \u0438\u043d\u0441\u0442\u0430\u043d\u0446\u0438\u0438 \u043f\u043e\u0432\u0435\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u0435\u043b\u044f, \u0438 \u0441\u0432\u043e\u0435\u0439 \u043b\u0435\u043d\u0438\u0432\u043e\u0439, \u043d\u043e \u0446\u0435\u043f\u043a\u043e\u0439 \u043f\u043e\u0432\u0435\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u043d\u043e\u0441\u0442\u044c\u044e \u0431\u043e\u043b\u044c\u0448\u0435 \u0441\u043c\u0430\u0445\u0438\u0432\u0430\u0435\u0442 \u043d\u0430 \u0414\u0436\u0438\u043c\u0430 \u0414\u0436\u0430\u0440\u043c\u0443\u0448\u0430\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.<<\n\n(x) \u041f\u0435\u0434\u0440\u0443 \u041a\u043e\u0448\u0442\u0430\n() \u041c\u0430\u043d\u0443\u044d\u043b \u0434\u0438 \u041e\u043b\u0438\u0432\u0435\u0439\u0440\u0430\n() \u041c\u0430\u0440\u0438\u044f \u0434\u0435 \u041c\u0435\u0434\u0435\u0439\u0440\u0443\u0448\n() \u0410\u043b\u0431\u0435\u0440\u0442\u0443 \u0421\u0435\u0439\u0448\u0430\u0448 \u0421\u0430\u043d\u0442\u0443\u0448\n\n", "problem_name": "\u0412\u043e\u043f\u0440\u043e\u0441 \u0441 \u043c\u043d\u043e\u0436\u0435\u0441\u0442\u0432\u0435\u043d\u043d\u044b\u043c \u0432\u044b\u0431\u043e\u0440\u043e\u043c", "chapter_id": "block-v1:hse+CINCULT+spring_2020+type@chapter+block@4e30300ce4ca4fbf90111a97c79a3a18", "vertical_id": "block-v1:hse+CINCULT+spring_2020+type@vertical+block@164bfcf3bb4e4a37a804378e4dca55b5", "sequential_id": "block-v1:hse+CINCULT+spring_2020+type@sequential+block@7035a678a3a344bea41421773d49ce5e", "course_id": "course-v1:hse+CINCULT+spring_2020", "data": "<problem>\n<label>1. \u041a\u0430\u043a\u0438\u0435 \u0441\u0442\u0440\u0430\u043d\u044b \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u043d\u0430\u0441\u0442\u043e\u044f\u0449\u0435\u0433\u043e \u043a\u0443\u0440\u0441\u0430 \u0440\u0430\u0441\u0441\u043c\u0430\u0442\u0440\u0438\u0432\u0430\u044e\u0442\u0441\u044f \u0432 \u0431\u043b\u043e\u043a\u0435, \u043f\u043e\u0441\u0432\u044f\u0449\u0435\u043d\u043d\u043e\u043c\u0443 \u0417\u0430\u043f\u0430\u0434\u043d\u043e\u0439 \u0415\u0432\u0440\u043e\u043f\u0435? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0434\u0432\u0430 \u043f\u0440\u0430\u0432\u0438\u043b\u044c\u043d\u044b\u0445 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u0430 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<choiceresponse>\n  <checkboxgroup>\n    <choice correct=\"true\">\u0411\u0435\u043d\u0438\u043b\u044e\u043a\u0441\u0430</choice>\n    <choice correct=\"false\">\u0410\u0432\u0441\u0442\u0440\u0438\u0438</choice>\n    <choice correct=\"true\">\u041f\u0438\u0440\u0435\u043d\u0435\u0439\u0441\u043a\u043e\u0433\u043e \u043f\u043e\u043b\u0443\u043e\u0441\u0442\u0440\u043e\u0432\u0430</choice>\n    <choice correct=\"false\">\u0412\u0435\u043d\u0433\u0440\u0438\u0438</choice>\n  </checkboxgroup>\n</choiceresponse>\n\n<label>2. \u041a\u0430\u043a\u0438\u0435 \u0438\u0437 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u043d\u044b\u0445 \u0432\u0435\u0445 \u0432 \u043a\u0438\u043d\u043e \u0418\u0441\u043f\u0430\u043d\u0438\u0438 \u041d\u0415 \u043e\u0442\u043d\u043e\u0441\u044f\u0442\u0441\u044f \u043a \u044d\u0442\u043e\u0439 \u0441\u0442\u0440\u0430\u043d\u0435? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0434\u0432\u0430 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u0430 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<choiceresponse>\n  <checkboxgroup>\n    <choice correct=\"true\">\u041e\u0442 \u0421\u0442\u043e\u0440\u043a\u0430 \u0434\u043e \u0431\u0440\u0430\u0442\u044c\u0435\u0432 \u0414\u0430\u0440\u0434\u0435\u043d\u043d</choice>\n    <choice correct=\"false\">\u0410\u0432\u0430\u043d\u0433\u0430\u0440\u0434 (\u0411\u0443\u043d\u044e\u044d\u043b\u044c) </choice>\n    <choice correct=\"false\">\u041f\u043e\u0441\u0442\u0444\u0440\u0430\u043d\u043a\u0438\u0441\u0442\u0441\u043a\u043e\u0435 \u043a\u0438\u043d\u043e</choice>\n    <choice correct=\"true\">\u041e\u0442 \u0420\u0438\u0431\u0435\u0439\u0440\u044b \u0434\u043e \u041e\u043b\u0438\u0432\u0435\u0439\u0440\u044b</choice>\n  </checkboxgroup>\n</choiceresponse>\n\n<label>3. \u0412 \u043a\u0430\u043a\u0438\u0435 \u0433\u043e\u0434\u044b \u0432 \u041d\u0438\u0434\u0435\u0440\u043b\u0430\u043d\u0434\u0430\u0445 \u043d\u0430\u0447\u0438\u043d\u0430\u044e\u0442 \u043c\u0430\u0441\u0441\u043e\u0432\u043e \u043e\u0442\u043a\u0440\u044b\u0432\u0430\u0442\u044c\u0441\u044f \u043a\u0438\u043d\u043e\u0442\u0435\u0430\u0442\u0440\u044b? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">1890-\u0435</choice>\n    <choice correct=\"false\">1900-\u0435</choice>\n    <choice correct=\"false\">1910-\u0435</choice>\n    <choice correct=\"true\">1920-\u0435</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>4. \u041a\u0430\u043a\u0438\u043c \u0444\u0438\u043b\u044c\u043c\u043e\u043c \u0412\u0441\u0435\u0432\u043e\u043b\u043e\u0434\u0430 \u041f\u0443\u0434\u043e\u0432\u043a\u0438\u043d\u0430 \u0432 1926 \u0433\u043e\u0434\u0443 \u0431\u044b\u043b \u0432\u0434\u043e\u0445\u043d\u043e\u0432\u043b\u0435\u043d \u0419\u043e\u0440\u0438\u0441 \u0418\u0432\u0435\u043d\u0441, \u0447\u0442\u043e \u0440\u0435\u0448\u0438\u043b \u043e\u0441\u043d\u043e\u0432\u0430\u0442\u044c \u043e\u0431\u0449\u0435\u0441\u0442\u0432\u043e \u043a\u0438\u043d\u043e\u043b\u044e\u0431\u0438\u0442\u0435\u043b\u0435\u0439 \u0430\u0432\u0430\u043d\u0433\u0430\u0440\u0434\u0430? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"true\">\u041c\u0430\u0442\u044c</choice>\n    <choice correct=\"false\">\u041a\u043e\u043d\u0435\u0446 \u0421\u0430\u043d\u043a\u0442-\u041f\u0435\u0442\u0435\u0440\u0431\u0443\u0440\u0433\u0430</choice>\n    <choice correct=\"false\">\u041c\u0435\u0445\u0430\u043d\u0438\u043a\u0430 \u0433\u043e\u043b\u043e\u0432\u043d\u043e\u0433\u043e \u043c\u043e\u0437\u0433\u0430</choice>\n    <choice correct=\"false\">\u0428\u0430\u0445\u043c\u0430\u0442\u043d\u0430\u044f \u0433\u043e\u0440\u044f\u0447\u043a\u0430</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>5. \u041a\u0430\u043a \u043d\u0430\u0437\u044b\u0432\u0430\u0435\u0442\u0441\u044f \u0444\u0438\u043b\u044c\u043c \u0419\u043e\u0440\u0438\u0441\u0430 \u0418\u0432\u0435\u043d\u0441\u0430, \u0433\u0434\u0435 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0440\u0430\u0441\u043a\u0440\u044b\u043b\u0441\u044f \u043a\u0430\u043a \u043f\u0440\u043e\u0434\u043e\u043b\u0436\u0430\u0442\u0435\u043b\u044c \u0441\u0442\u0438\u043b\u0438\u0441\u0442\u0438\u043a\u0438 \u0412\u0430\u043b\u044c\u0442\u0435\u0440\u0430 \u0420\u0443\u0442\u0442\u043c\u0430\u043d\u0430 \u0438 \u0414\u0437\u0438\u0433\u0438 \u0412\u0435\u0440\u0442\u043e\u0432\u0430? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u0412\u0438\u0433\u0432\u0430\u043c (1911)</choice>\n    <choice correct=\"true\">\u0414\u043e\u0436\u0434\u044c (1929)</choice>\n    <choice correct=\"false\">\u041f\u0435\u0441\u043d\u044c \u043e \u0433\u0435\u0440\u043e\u044f\u0445 (1932)</choice>\n    <choice correct=\"false\">\u0418\u0441\u043f\u0430\u043d\u0441\u043a\u0430\u044f \u0437\u0435\u043c\u043b\u044f (1937)</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>6. \u041e \u043a\u0430\u043a\u043e\u043c \u0444\u0438\u043b\u044c\u043c\u0435 \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab\u0412 \u0441\u0442\u0430\u0440\u043e\u0439 \u0447\u0430\u0441\u0442\u0438 \u043d\u0438\u0434\u0435\u0440\u043b\u0430\u043d\u0434\u0441\u043a\u043e\u0439 \u0441\u0442\u043e\u043b\u0438\u0446\u044b \u043e\u0440\u0443\u0434\u0443\u0435\u0442 \u043c\u0430\u043d\u044c\u044f\u043a, \u0441\u0443\u0434\u044f \u043f\u043e \u043f\u043e\u0447\u0435\u0440\u043a\u0443, \u0432\u043e\u0434\u043e\u043b\u0430\u0437. \u0415\u043c\u0443 \u0443\u0434\u0430\u0435\u0442\u0441\u044f \u0443\u0445\u043e\u0434\u0438\u0442\u044c \u043b\u0438\u0448\u044c \u043e\u0434\u043d\u043e\u043c\u0443 \u0435\u043c\u0443 \u0438\u0437\u0432\u0435\u0441\u0442\u043d\u044b\u043c\u0438 \u0432\u043e\u0434\u043d\u044b\u043c\u0438 \u0430\u0440\u0442\u0435\u0440\u0438\u044f\u043c\u0438. \u041e\u0434\u043d\u0430 \u0438\u0437 \u043a\u0443\u043b\u044c\u043c\u0438\u043d\u0430\u0446\u0438\u043e\u043d\u043d\u044b\u0445 \u0441\u0446\u0435\u043d \u0444\u0438\u043b\u044c\u043c\u0430 \u2014 \u043f\u044f\u0442\u0438\u043c\u0438\u043d\u0443\u0442\u043d\u0430\u044f \u0441\u044a\u0435\u043c\u043a\u0430 \u043f\u043e\u0433\u043e\u043d\u0438 \u043d\u0430 \u0441\u043a\u0443\u0442\u0435\u0440\u0430\u0445 \u043d\u0430 100% \u0443\u043d\u0438\u043a\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043a\u0442\u0443\u0440\u0435 \u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c\u0430\u2026 \u0414\u0430\u0436\u0435 \u0441\u043f\u0443\u0441\u0442\u044f 30 \u043b\u0435\u0442, \u044d\u0442\u0443 \u043f\u043e\u0433\u043e\u043d\u044e \u0432\u043a\u043b\u044e\u0447\u0430\u044e\u0442 \u0432\u043e \u0432\u0441\u0435\u0432\u043e\u0437\u043c\u043e\u0436\u043d\u044b\u0435 \u0442\u043e\u043f\u044b \u0438 \u0443\u043f\u043e\u043c\u0438\u043d\u0430\u044e\u0442 \u043a\u0430\u043a \u043e\u0431\u0440\u0430\u0437\u0446\u043e\u0432\u044b\u0439 \u043f\u0440\u0438\u043c\u0435\u0440 \u0440\u0430\u0431\u043e\u0442\u044b \u0441 \u0435\u0441\u0442\u0435\u0441\u0442\u0432\u0435\u043d\u043d\u044b\u043c\u0438 \u0434\u0435\u043a\u043e\u0440\u0430\u0446\u0438\u044f\u043c\u0438\u00bb.  \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u041c\u0435\u043b\u044c\u043d\u0438\u0446\u0430 \u043a\u0430\u043c\u0435\u043d\u043d\u044b\u0445 \u0436\u0435\u043d\u0449\u0438\u043d (1960)</choice>\n    <choice correct=\"false\">\u041f\u0440\u043e\u0438\u0441\u0448\u0435\u0441\u0442\u0432\u0438\u0435 \u0432 \u0413\u043b\u044f\u0439\u0432\u0438\u0446\u0435 (1961)</choice>\n    <choice correct=\"true\">\u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c\u0441\u043a\u0438\u0439 \u043a\u043e\u0448\u043c\u0430\u0440/\u041f\u0440\u043e\u043a\u043b\u044f\u0442\u044b\u0439 \u0410\u043c\u0441\u0442\u0435\u0440\u0434\u0430\u043c (1987)</choice>\n    <choice correct=\"false\">\u0413\u043e\u043b\u043b\u0430\u043d\u0434\u0441\u043a\u0438\u0439 \u0441\u0432\u0435\u0442 (2003)</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>7. \u041f\u043e\u0441\u043b\u0435 \u0432\u044b\u0445\u043e\u0434\u0430 \u043a\u0430\u043a\u043e\u0433\u043e \u0444\u0438\u043b\u044c\u043c\u0430 \u0431\u044b\u043b \u0443\u0431\u0438\u0442 \u0435\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u2014  \u0422\u0435\u043e \u0432\u0430\u043d \u0413\u043e\u0433? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430. </label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u0421\u0435\u0432\u0435\u0440\u044f\u043d\u0435 (1992)</choice>\n    <choice correct=\"false\">\u0414\u0435\u0441\u044f\u0442\u044c \u043c\u043e\u043d\u043e\u043b\u043e\u0433\u043e\u0432 \u0438\u0437 \u0436\u0438\u0437\u043d\u0438 \u0441\u0435\u0440\u0438\u0439\u043d\u044b\u0445 \u0443\u0431\u0438\u0439\u0446 (1994)</choice>\n    <choice correct=\"false\">\u0424\u0440\u0430\u043d\u043a \u0438 \u0415\u0432\u0430 (1973)</choice>\n    <choice correct=\"true\">\u041f\u043e\u043a\u043e\u0440\u043d\u043e\u0441\u0442\u044c (2004)</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>8. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u00ab\u0431\u0435\u043b\u044c\u0433\u0438\u0439\u0441\u043a\u043e\u0433\u043e \u0419\u043e\u0440\u0438\u0441\u0430 \u0418\u0432\u0435\u043d\u0441\u0430\u00bb, \u0441\u043d\u0438\u043c\u0430\u0432\u0448\u0435\u0433\u043e \u0443\u0436\u0430\u0441\u0430\u044e\u0449\u0438\u0435 \u0443\u0441\u043b\u043e\u0432\u0438\u044f \u0436\u0438\u0437\u043d\u0438 \u0448\u0430\u0445\u0442\u0435\u0440\u043e\u0432? \u0412 \u0440\u0435\u0437\u0443\u043b\u044c\u0442\u0430\u0442\u0435 \u044d\u0442\u0438\u0445 \u0441\u044a\u0435\u043c\u043e\u043a \u0432\u044b\u0448\u0435\u043b \u0444\u0438\u043b\u044c\u043c \u041d\u0438\u0449\u0435\u0442\u0430 \u0432 \u0411\u043e\u0440\u0438\u043d\u0430\u0436\u0435 (1934). \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"true\">\u0410\u043d\u0440\u0438 \u0428\u0442\u043e\u0440\u043a</choice>\n    <choice correct=\"false\">\u0410\u043d\u0440\u0438 \u0420\u0443\u0441\u0441\u043e</choice>\n    <choice correct=\"false\">\u0410\u043d\u0434\u0440\u0435 \u0414\u0435\u043b\u044c\u0432\u043e</choice>\n    <choice correct=\"false\">\u041b\u044e\u0441\u044c\u0435\u043d \u0414\u0435\u0440\u043e\u0438\u0441</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>10. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0437\u0432\u0435\u0441\u0442\u043d\u044b\u0445 \u0431\u0435\u043b\u044c\u0433\u0438\u0439\u0441\u043a\u0438\u0445 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u043e\u0432, \u043a\u043e\u0442\u043e\u0440\u044b\u0435 \u044f\u0432\u043b\u044f\u044e\u0442\u0441\u044f \u0431\u0440\u0430\u0442\u044c\u044f\u043c\u0438 \u0438 \u0441\u043d\u0438\u043c\u0430\u044e\u0442 \u043e\u0431\u044b\u0447\u043d\u043e \u043e\u0441\u0442\u0440\u043e\u0441\u043e\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0435 \u043a\u0438\u043d\u043e? \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u041f\u0438\u0442\u0435\u0440 \u0438 \u0411\u043e\u0431\u0431\u0438 \u0424\u0430\u0440\u0440\u0435\u043b\u043b\u0438</choice>\n    <choice correct=\"false\">\u041c\u0430\u0440\u043a \u0438 \u041c\u0430\u0439\u043a\u043b \u041f\u043e\u043b\u0438\u0448</choice>\n    <choice correct=\"false\">\u0414\u0436\u043e\u044d\u043b \u0438 \u0418\u0442\u0430\u043d \u041a\u043e\u044d\u043d</choice>\n    <choice correct=\"true\">\u0416\u0430\u043d-\u041f\u044c\u0435\u0440 \u0438 \u041b\u044e\u043a \u0414\u0430\u0440\u0434\u0435\u043d\u043d</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>11. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0430, \u043a\u043e\u0442\u043e\u0440\u044b\u0439 \u00ab\u0441\u0443\u043c\u0435\u043b \u043f\u0440\u0435\u043e\u0434\u043e\u043b\u0435\u0442\u044c \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u0443\u044e \u043f\u0435\u0440\u0438\u0444\u0435\u0440\u0438\u0439\u043d\u043e\u0441\u0442\u044c\u00bb? \u0421\u0430\u043c \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0433\u043e\u0432\u043e\u0440\u0438\u043b \u043e \u0441\u0432\u043e\u0438\u0445 \u0444\u0438\u043b\u044c\u043c\u0430\u0445 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0438\u043c \u043e\u0431\u0440\u0430\u0437\u043e\u043c: \u00ab\u042f \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u0443\u044e \u0433\u0440\u043e\u0442\u0435\u0441\u043a, \u043f\u0440\u043e\u0442\u0435\u0441\u0442 \u0438 \u0447\u0435\u0440\u043d\u044b\u0439 \u044e\u043c\u043e\u0440. \u041f\u0440\u0438\u0447\u0435\u043c \u043f\u043e\u0447\u0442\u0438 \u0432\u0441\u0435\u0433\u0434\u0430 \u043f\u043e\u043c\u0438\u043c\u043e \u0441\u043e\u0431\u0441\u0442\u0432\u0435\u043d\u043d\u043e\u0439 \u0432\u043e\u043b\u0438\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"true\">\u041b\u0443\u0438\u0441 \u0411\u0443\u043d\u044e\u044d\u043b\u044c</choice>\n    <choice correct=\"false\">\u041f\u0435\u0434\u0440\u043e \u0410\u043b\u044c\u043c\u043e\u0434\u043e\u0432\u0430\u0440</choice>\n    <choice correct=\"false\">\u0410\u043b\u0435\u0445\u0430\u043d\u0434\u0440\u043e \u0410\u043c\u0435\u043d\u0435\u0431\u0430\u0440</choice>\n    <choice correct=\"false\">\u0425\u0443\u043b\u0438\u043e \u041c\u0435\u0434\u0435\u043c</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>12. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u0433\u043e \u00ab\u043a\u043e\u0440\u043e\u043b\u044f \u044d\u043a\u0441\u043f\u043b\u0443\u0430\u0442\u0430\u0446\u0438\u043e\u043d\u043d\u043e\u0433\u043e \u043a\u0438\u043d\u043e\u00bb? \u042d\u0442\u043e\u0442 \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440 \u0438\u0437\u0432\u0435\u0441\u0442\u0435\u043d \u0442\u0435\u043c, \u0447\u0442\u043e \u0437\u0430 \u0441\u0432\u043e\u044e \u0436\u0438\u0437\u043d\u044c \u0441\u043d\u044f\u043b 180 \u0444\u0438\u043b\u044c\u043c\u043e\u0432, \u043a\u043e\u0442\u043e\u0440\u043e\u0435 \u043c\u043e\u0436\u043d\u043e \u043e\u0442\u043d\u0435\u0441\u0442\u0438 \u043a \u0442\u0430\u043a\u0438\u043c \u043a\u0430\u0442\u0435\u0433\u043e\u0440\u0438\u044f\u043c \u044d\u043a\u0441\u043f\u043b\u0443\u0430\u0442\u0430\u0446\u0438\u043e\u043d\u043d\u043e\u0433\u043e \u043a\u0438\u043d\u0435\u043c\u0430\u0442\u043e\u0433\u0440\u0430\u0444\u0430, \u043a\u0430\u043a women in prison \u0438 sexploitation. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u0425\u043e\u0430\u043a\u0438\u043d \u041e\u0440\u0438\u0441\u0442\u0440\u0435\u043b\u044c</choice>\n    <choice correct=\"false\">\u0421\u0438\u043b\u044c\u0432\u0438\u044f \u041c\u0443\u043d\u0442</choice>\n    <choice correct=\"true\">\u0425\u0435\u0441\u0443\u0441 \u0424\u0440\u0430\u043d\u043a\u043e</choice>\n    <choice correct=\"false\">\u041d\u0430\u0447\u043e \u0412\u0438\u0433\u0430\u043b\u043e\u043d\u0434\u043e</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>13. \u041e \u043a\u0430\u043a\u043e\u043c \u0438\u0441\u043f\u0430\u043d\u0441\u043a\u043e\u043c \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0435 \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab_____ \u041f\u0440\u0438\u0448\u0435\u043b \u0432 \u043a\u0438\u043d\u043e \u043f\u043e\u0441\u043b\u0435 \u043f\u0430\u0434\u0435\u043d\u0438\u044f \u0440\u0435\u0436\u0438\u043c\u0430 \u0424\u0440\u0430\u043d\u043a\u043e, \u043f\u0440\u0438\u0448\u0435\u043b \u043a\u0430\u043a \u043b\u044e\u0431\u0438\u0442\u0435\u043b\u044c, \u0440\u043e\u043a-\u043c\u0443\u0437\u044b\u043a\u0430\u043d\u0442, \u0436\u0443\u0440\u043d\u0430\u043b\u0438\u0441\u0442, \u043f\u0438\u0441\u0430\u0432\u0448\u0438\u0439 \u043f\u043e\u0434 \u0436\u0435\u043d\u0441\u043a\u0438\u043c \u043f\u0441\u0435\u0432\u0434\u043e\u043d\u0438\u043c\u043e\u043c. \u0423\u0436\u0435 \u043a \u0441\u0435\u0440\u0435\u0434\u0438\u043d\u0435 1980-\u0445 \u0433\u0433. \u043f\u043e\u0441\u043b\u0435 \u0442\u0430\u043a\u0438\u0445 \u0432\u0435\u043b\u0438\u043a\u0438\u0445 \u0444\u0438\u043b\u044c\u043c\u043e\u0432 \u043a\u0430\u043a \u041c\u0430\u0442\u0430\u0434\u043e\u0440 (1986), \u0417\u0430\u043a\u043e\u043d \u0436\u0435\u043b\u0430\u043d\u0438\u0439 (1987) \u0438 \u041b\u0430\u0431\u0438\u0440\u0438\u043d\u0442 \u0441\u0442\u0440\u0430\u0441\u0442\u0435\u0439 (1982), \u0438 \u043e\u0441\u043e\u0431\u0435\u043d\u043d\u043e \u044d\u0442\u043e \u043a\u0430\u0441\u0430\u0435\u0442\u0441\u044f \u043e\u0441\u044b\u043f\u0430\u043d\u043d\u043e\u0433\u043e \u0441\u043b\u0430\u0432\u043e\u0439 \u0444\u0438\u043b\u044c\u043c\u0430 \u0416\u0435\u043d\u0449\u0438\u043d\u0430 \u043d\u0430 \u0433\u0440\u0430\u043d\u0438 \u043d\u0435\u0440\u0432\u043d\u043e\u0433\u043e \u0441\u0440\u044b\u0432\u0430 (1988), ____ \u0441\u0442\u0430\u043d\u043e\u0432\u0438\u0442\u0441\u044f \u0431\u0440\u0435\u043d\u0434\u043e\u043c\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"true\">\u041f\u0435\u0434\u0440\u043e \u0410\u043b\u044c\u043c\u043e\u0434\u043e\u0432\u0430\u0440</choice>\n    <choice correct=\"false\">\u0412\u0438\u043a\u0442\u043e\u0440 \u042d\u0440\u0438\u0441\u0435</choice>\n    <choice correct=\"false\">\u0424\u0435\u0440\u043d\u0430\u043d\u0434\u043e \u0422\u0440\u0443\u044d\u0431\u0430</choice>\n    <choice correct=\"false\">\u042d\u043b\u0438\u0430\u0441 \u041a\u0435\u0440\u0435\u0445\u0435\u0442\u0430</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>14. \u041e \u043a\u0430\u043a\u043e\u043c \u0444\u0438\u043b\u044c\u043c\u0435 \u043f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u044c\u0441\u043a\u043e\u0439 \u00ab\u043d\u043e\u0432\u043e\u0439 \u0432\u043e\u043b\u043d\u044b\u00bb \u0438\u0434\u0435\u0442 \u0440\u0435\u0447\u044c \u0432 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0435\u043c \u043e\u0442\u0440\u044b\u0432\u043a\u0435? \u00ab\u0424\u0438\u043b\u044c\u043c \u0440\u0430\u0441\u0441\u043a\u0430\u0437\u044b\u0432\u0430\u0435\u0442 \u0438\u0441\u0442\u043e\u0440\u0438\u044e \u043d\u0435\u0434\u043e\u0443\u043c\u0435\u043d\u0438\u044f, \u043a\u043e\u0442\u043e\u0440\u043e\u0435 \u0438\u0441\u043f\u044b\u0442\u044b\u0432\u0430\u0435\u0442 \u043e\u0442 \u0431\u043e\u043b\u044c\u0448\u043e\u0433\u043e \u0433\u043e\u0440\u043e\u0434\u0430 \u043d\u0435\u0434\u0430\u0432\u043d\u043e \u043f\u0435\u0440\u0435\u0435\u0445\u0430\u0432\u0448\u0438\u0439 \u0438\u0437 \u043f\u0440\u043e\u0432\u0438\u043d\u0446\u0438\u0438. \u041e\u043d \u043c\u043d\u043e\u0433\u043e\u0435 \u0443\u043c\u0435\u0435\u0442 \u0434\u0435\u043b\u0430\u0442\u044c \u0440\u0443\u043a\u0430\u043c\u0438, \u043d\u043e \u043c\u0435\u0441\u0442\u043d\u0430\u044f \u0436\u0438\u0437\u043d\u044c \u0445\u0438\u0442\u0440\u0435\u0435, \u0435\u043c\u0443 \u043f\u0440\u0438\u0445\u043e\u0434\u0438\u0442\u0441\u044f \u043d\u0435 \u0440\u0430\u0437 \u0443\u0431\u0435\u0436\u0434\u0430\u0442\u044c\u0441\u044f \u0432 \u044d\u0442\u043e\u043c\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"false\">\u041a\u043e\u0440\u0435\u043d\u044c \u0441\u0435\u0440\u0434\u0446\u0430 (2000)</choice>\n    <choice correct=\"true\">\u0417\u0435\u043b\u0451\u043d\u044b\u0435 \u0433\u043e\u0434\u044b (1963)</choice>\n    <choice correct=\"false\">\u041e\u0441\u0442\u0440\u043e\u0432 \u043b\u044e\u0431\u0432\u0438 (1982)</choice>\n    <choice correct=\"false\">\u041a\u0438\u043d\u043e \u043d\u0430\u0448\u0435\u0433\u043e \u0432\u0440\u0435\u043c\u0435\u043d\u0438 (\u0441\u0435\u0440\u0438\u0430\u043b, 1988 \u2013 ...)</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n<label>15. \u041a\u0430\u043a \u0437\u043e\u0432\u0443\u0442 \u043f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u044c\u0441\u043a\u043e\u0433\u043e \u0440\u0435\u0436\u0438\u0441\u0441\u0435\u0440\u0430, \u0441\u043d\u0438\u043c\u0430\u0432\u0448\u0435\u0433\u043e \u0432 \u044d\u0441\u0442\u0435\u0442\u0438\u043a\u0435 docu-fiction? \u041e\u043f\u0438\u0440\u0430\u0439\u0442\u0435\u0441\u044c \u043f\u0440\u0438 \u0432\u044b\u0431\u043e\u0440\u0435 \u043e\u0442\u0432\u0435\u0442\u0430 \u043d\u0430 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0438\u0439 \u043e\u0442\u0440\u044b\u0432\u043e\u043a: \u00ab\u0427\u0435\u0440\u043d\u043e-\u0431\u0435\u043b\u044b\u0439 \u044d\u0441\u0442\u0435\u0442\u0441\u043a\u0438\u0439 \u0440\u0435\u043f\u043e\u0440\u0442\u0430\u0436 \u0432 \u0441\u043e\u043f\u0440\u043e\u0432\u043e\u0436\u0434\u0435\u043d\u0438\u0438 \u0437\u0430\u043a\u0430\u0434\u0440\u043e\u0432\u043e\u0433\u043e \u0433\u043e\u043b\u043e\u0441\u0430 \u043f\u0440\u0435\u0432\u0440\u0430\u0449\u0430\u0435\u0442\u0441\u044f \u0432 \u0432\u0438\u0437\u0438\u0442\u043d\u0443\u044e \u043a\u0430\u0440\u0442\u043e\u0447\u043a\u0443 \u0433\u043e\u0440\u043e\u0434\u0441\u043a\u043e\u0439 \u0434\u0440\u0430\u043c\u044b \u041f\u043e\u0440\u0442\u0443\u0433\u0430\u043b\u0438\u0438. \u041e\u0448\u0435\u043b\u043e\u043c\u0438\u0442\u0435\u043b\u044c\u043d\u044b\u0439 \u0443\u0441\u043f\u0435\u0445 ___ ____1989 \u043f\u043e\u0434 \u043f\u0440\u043e\u0441\u0442\u044b\u043c, \u043d\u043e \u043c\u043d\u043e\u0433\u043e\u0437\u043d\u0430\u0447\u043d\u044b\u043c \u043d\u0430\u0437\u0432\u0430\u043d\u0438\u0435\u043c \"\u041a\u0440\u043e\u0432\u044c\" \u043b\u0438\u0448\u0435\u043d \u0438\u043d\u0441\u0442\u0430\u043d\u0446\u0438\u0438 \u043f\u043e\u0432\u0435\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u0435\u043b\u044f, \u0438 \u0441\u0432\u043e\u0435\u0439 \u043b\u0435\u043d\u0438\u0432\u043e\u0439, \u043d\u043e \u0446\u0435\u043f\u043a\u043e\u0439 \u043f\u043e\u0432\u0435\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u043d\u043e\u0441\u0442\u044c\u044e \u0431\u043e\u043b\u044c\u0448\u0435 \u0441\u043c\u0430\u0445\u0438\u0432\u0430\u0435\u0442 \u043d\u0430 \u0414\u0436\u0438\u043c\u0430 \u0414\u0436\u0430\u0440\u043c\u0443\u0448\u0430\u00bb. \u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u043e\u0434\u0438\u043d \u0432\u0430\u0440\u0438\u0430\u043d\u0442 \u043e\u0442\u0432\u0435\u0442\u0430.</label>\n<multiplechoiceresponse>\n  <choicegroup type=\"MultipleChoice\">\n    <choice correct=\"true\">\u041f\u0435\u0434\u0440\u0443 \u041a\u043e\u0448\u0442\u0430</choice>\n    <choice correct=\"false\">\u041c\u0430\u043d\u0443\u044d\u043b \u0434\u0438 \u041e\u043b\u0438\u0432\u0435\u0439\u0440\u0430</choice>\n    <choice correct=\"false\">\u041c\u0430\u0440\u0438\u044f \u0434\u0435 \u041c\u0435\u0434\u0435\u0439\u0440\u0443\u0448</choice>\n    <choice correct=\"false\">\u0410\u043b\u0431\u0435\u0440\u0442\u0443 \u0421\u0435\u0439\u0448\u0430\u0448 \u0421\u0430\u043d\u0442\u0443\u0448</choice>\n  </choicegroup>\n</multiplechoiceresponse>\n\n\n</problem>", "problem_id": "block-v1:hse+CINCULT+spring_2020+type@problem+block@0492a032cac64b6b8e6fb7397fa54ec8"}, "event_type": "problem_publish"}'
d = json.loads(s)

f = open('textline2.json', 'w', encoding='cp1251')
json.dump(d, f, indent=4, ensure_ascii=False)
f.close()

In [80]:
s = r'{"username": "olabykina111", "event_type": "problem_check", "ip": "94.253.69.247", "agent": "Mozilla/5.0 (Linux; Android 9; PAR-LX1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.117 Mobile Safari/537.36", "host": "courses.openedu.ru", "referer": "https://courses.openedu.ru/courses/course-v1:hse+QUALSOC+spring_2020/courseware/4b058663bf50403bbb8f058f6eb4c7f9/8ee7a9010ca84cc9b57fb4b3f1cc3493/?child=first", "accept_language": "ru;q=1.0, ru;q=0.9", "event": {"submission": {"6862136954f64ed1aa24e0264d2ef591_13_1": {"choices_map": {"choice_0": "\u0421\u043e\u0435\u0434\u0438\u043d\u044f\u0442\u044c \u043b\u0438\u0447\u043d\u043e\u0435 \u0438 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u043e\u0435 \u00ab\u042f\u00bb \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "choice_1": "\u041c\u0430\u043d\u0438\u043f\u0443\u043b\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043c\u043d\u0435\u043d\u0438\u0435\u043c \u043b\u044e\u0434\u0435\u0439 \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "choice_2": "\u0418\u0441\u043f\u0440\u0430\u0432\u043b\u044f\u0442\u044c \u043e\u0431\u0449\u0435\u0441\u0442\u0432\u043e \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "choice_3": "\u0411\u044b\u0442\u044c \u0430\u043d\u0433\u0430\u0436\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u044b\u043c \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"}, "answers_available": ["\u0421\u043e\u0435\u0434\u0438\u043d\u044f\u0442\u044c \u043b\u0438\u0447\u043d\u043e\u0435 \u0438 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u043e\u0435 \u00ab\u042f\u00bb \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "\u041c\u0430\u043d\u0438\u043f\u0443\u043b\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043c\u043d\u0435\u043d\u0438\u0435\u043c \u043b\u044e\u0434\u0435\u0439 \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "\u0418\u0441\u043f\u0440\u0430\u0432\u043b\u044f\u0442\u044c \u043e\u0431\u0449\u0435\u0441\u0442\u0432\u043e \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "\u0411\u044b\u0442\u044c \u0430\u043d\u0433\u0430\u0436\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u044b\u043c \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"], "question": "12. \u0421\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u044e\u0442 \u0440\u0430\u0437\u043d\u044b\u0435 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u0438\u044f \u043e\u0431 \u044d\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0445 \u043f\u0440\u0438\u043d\u0446\u0438\u043f\u0430\u0445. \u0418\u043d\u043e\u0433\u0434\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u0438 \u043d\u0435 \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u044e\u0442\u0441\u044f \u0434\u0440\u0443\u0433 \u0441 \u0434\u0440\u0443\u0433\u043e\u043c \u043f\u043e \u043c\u043d\u043e\u0433\u0438\u043c \u0432\u043e\u043f\u0440\u043e\u0441\u0430\u043c. \u041e\u0434\u043d\u0430\u043a\u043e, \u0435\u0441\u0442\u044c \u043f\u0440\u0438\u043d\u0446\u0438\u043f, \u043a\u043e\u0442\u043e\u0440\u044b\u0439 \u043f\u043e\u0434\u0434\u0435\u0440\u0436\u0438\u0432\u0430\u044e\u0442 \u0432\u0441\u0435 \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u044b\u0435 \u0441\u043e\u0446\u0438\u043e\u043b\u043e\u0433\u0438. \u0427\u0442\u043e \u043d\u0435\u043b\u044c\u0437\u044f \u0434\u0435\u043b\u0430\u0442\u044c \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u0441\u043e\u0446\u0438\u043e\u043b\u043e\u0433\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_1", "choice_correct": ["choice_1"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041c\u0430\u043d\u0438\u043f\u0443\u043b\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043c\u043d\u0435\u043d\u0438\u0435\u043c \u043b\u044e\u0434\u0435\u0439 \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"], "answer": "\u041c\u0430\u043d\u0438\u043f\u0443\u043b\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043c\u043d\u0435\u043d\u0438\u0435\u043c \u043b\u044e\u0434\u0435\u0439 \u0432 \u0445\u043e\u0434\u0435 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_16_1": {"choices_map": {"choice_4": "\u042d\u0442\u0430 \u043f\u0440\u0430\u043a\u0442\u0438\u043a\u0430 \u044e\u0440\u0438\u0434\u0438\u0447\u0435\u0441\u043a\u0438 \u043d\u0435 \u043f\u043e\u0434\u043a\u0440\u0435\u043f\u043b\u0435\u043d\u0430 \u0438 \u043d\u0435 \u0438\u043c\u0435\u0435\u0442 \u0440\u0435\u0430\u043b\u044c\u043d\u043e\u0439 \u0441\u0438\u043b\u044b", "choice_0": "\u041e\u043d\u0430 \u043d\u0435 \u0432\u0441\u0435\u0433\u0434\u0430 \u0443\u043c\u0435\u0441\u0442\u043d\u0430", "choice_1": "\u0415\u0435 \u0441\u043b\u043e\u0436\u043d\u043e \u043e\u0440\u0433\u0430\u043d\u0438\u0437\u043e\u0432\u0430\u0442\u044c", "choice_2": "\u041e\u043d\u0430 \u043c\u043e\u0436\u0435\u0442 \u043f\u043e\u0432\u044b\u0441\u0438\u0442\u044c \u043e\u0449\u0443\u0449\u0435\u043d\u0438\u0435 \u0440\u0438\u0441\u043a\u0430 \u0443 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0430 \u0438 \u043e\u0442\u043f\u0443\u0433\u043d\u0443\u0442\u044c \u0435\u0433\u043e", "choice_3": "\u042d\u0442\u043e \u043f\u0440\u043e\u0441\u0442\u043e \u0444\u043e\u0440\u043c\u0430\u043b\u044c\u043d\u044b\u0439 \u0434\u043e\u043a\u0443\u043c\u0435\u043d\u0442, \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043d\u0435 \u0432\u043b\u0438\u044f\u0435\u0442 \u043d\u0430 \u043f\u0440\u0430\u043a\u0442\u0438\u043a\u0443 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0439"}, "answers_available": ["\u042d\u0442\u0430 \u043f\u0440\u0430\u043a\u0442\u0438\u043a\u0430 \u044e\u0440\u0438\u0434\u0438\u0447\u0435\u0441\u043a\u0438 \u043d\u0435 \u043f\u043e\u0434\u043a\u0440\u0435\u043f\u043b\u0435\u043d\u0430 \u0438 \u043d\u0435 \u0438\u043c\u0435\u0435\u0442 \u0440\u0435\u0430\u043b\u044c\u043d\u043e\u0439 \u0441\u0438\u043b\u044b", "\u041e\u043d\u0430 \u043d\u0435 \u0432\u0441\u0435\u0433\u0434\u0430 \u0443\u043c\u0435\u0441\u0442\u043d\u0430", "\u0415\u0435 \u0441\u043b\u043e\u0436\u043d\u043e \u043e\u0440\u0433\u0430\u043d\u0438\u0437\u043e\u0432\u0430\u0442\u044c", "\u041e\u043d\u0430 \u043c\u043e\u0436\u0435\u0442 \u043f\u043e\u0432\u044b\u0441\u0438\u0442\u044c \u043e\u0449\u0443\u0449\u0435\u043d\u0438\u0435 \u0440\u0438\u0441\u043a\u0430 \u0443 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0430 \u0438 \u043e\u0442\u043f\u0443\u0433\u043d\u0443\u0442\u044c \u0435\u0433\u043e", "\u042d\u0442\u043e \u043f\u0440\u043e\u0441\u0442\u043e \u0444\u043e\u0440\u043c\u0430\u043b\u044c\u043d\u044b\u0439 \u0434\u043e\u043a\u0443\u043c\u0435\u043d\u0442, \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043d\u0435 \u0432\u043b\u0438\u044f\u0435\u0442 \u043d\u0430 \u043f\u0440\u0430\u043a\u0442\u0438\u043a\u0443 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0439"], "question": "15. \u0417\u0430 \u0447\u0442\u043e \u043a\u0440\u0438\u0442\u0438\u043a\u0443\u0435\u0442\u0441\u044f \u043f\u0440\u0430\u043a\u0442\u0438\u043a\u0430 \u043f\u043e\u043b\u0443\u0447\u0435\u043d\u0438\u044f \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0433\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041e\u043d\u0430 \u043c\u043e\u0436\u0435\u0442 \u043f\u043e\u0432\u044b\u0441\u0438\u0442\u044c \u043e\u0449\u0443\u0449\u0435\u043d\u0438\u0435 \u0440\u0438\u0441\u043a\u0430 \u0443 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0430 \u0438 \u043e\u0442\u043f\u0443\u0433\u043d\u0443\u0442\u044c \u0435\u0433\u043e"], "answer": "\u041e\u043d\u0430 \u043c\u043e\u0436\u0435\u0442 \u043f\u043e\u0432\u044b\u0441\u0438\u0442\u044c \u043e\u0449\u0443\u0449\u0435\u043d\u0438\u0435 \u0440\u0438\u0441\u043a\u0430 \u0443 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0430 \u0438 \u043e\u0442\u043f\u0443\u0433\u043d\u0443\u0442\u044c \u0435\u0433\u043e", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_5_1": {"choices_map": {"choice_0": "\u044d\u0442\u0430\u043f\u0443 \u0441\u0431\u043e\u0440\u0430 \u0434\u0430\u043d\u043d\u044b\u0445", "choice_1": "\u044d\u0442\u0430\u043f\u0443 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u0438 \u0440\u0435\u0437\u0443\u043b\u044c\u0442\u0430\u0442\u043e\u0432", "choice_2": "\u0432\u0441\u0435\u043c \u044d\u0442\u0430\u043f\u0430\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"}, "answers_available": ["\u044d\u0442\u0430\u043f\u0443 \u0441\u0431\u043e\u0440\u0430 \u0434\u0430\u043d\u043d\u044b\u0445", "\u044d\u0442\u0430\u043f\u0443 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u0438 \u0440\u0435\u0437\u0443\u043b\u044c\u0442\u0430\u0442\u043e\u0432", "\u0432\u0441\u0435\u043c \u044d\u0442\u0430\u043f\u0430\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"], "question": "4. \u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430 \u0441\u043e\u0446\u0438\u043e\u043b\u043e\u0433\u0430 \u0441\u043e\u043f\u0443\u0442\u0441\u0442\u0432\u0443\u0435\u0442", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0432\u0441\u0435\u043c \u044d\u0442\u0430\u043f\u0430\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"], "answer": "\u0432\u0441\u0435\u043c \u044d\u0442\u0430\u043f\u0430\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_4_1": {"choices_map": {"choice_0": "\u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c", "choice_1": "\u0430\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c", "choice_2": "\u043c\u043e\u0440\u0430\u043b\u044c", "choice_3": "\u0446\u0435\u043d\u043d\u043e\u0441\u0442\u044c"}, "answers_available": ["\u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c", "\u0430\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c", "\u043c\u043e\u0440\u0430\u043b\u044c", "\u0446\u0435\u043d\u043d\u043e\u0441\u0442\u044c"], "question": "3. \u0413\u0430\u0440\u0430\u043d\u0442\u0438\u044f \u043d\u0435\u0440\u0430\u0437\u0433\u043b\u0430\u0448\u0435\u043d\u0438\u044f \u0438 \u043e\u0442\u0441\u0443\u0442\u0441\u0442\u0432\u0438\u044f \u0443\u0442\u0435\u0447\u043a\u0438 \u043f\u0435\u0440\u0441\u043e\u043d\u0430\u043b\u044c\u043d\u043e\u0439 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0438, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u043e\u0431\u0435\u0441\u043f\u0435\u0447\u0438\u0432\u0430\u0435\u0442\u0441\u044f \u0441\u043f\u0435\u0446\u0438\u0430\u043b\u044c\u043d\u044b\u043c\u0438 \u043f\u0440\u0430\u0432\u0438\u043b\u0430\u043c\u0438 \u0441\u0431\u043e\u0440\u0430 \u0438 \u0445\u0440\u0430\u043d\u0435\u043d\u0438\u044f \u0434\u0430\u043d\u043d\u044b\u0445 \u2013 \u044d\u0442\u043e", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_0"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c"], "answer": "\u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_11_1": {"choices_map": {}, "answers_available": ["\u0417\u0430\u043f\u0438\u0441\u044b\u0432\u0430\u0442\u044c \u0441\u043a\u0440\u044b\u0442\u043e, \u0438\u043d\u0430\u0447\u0435 \u043f\u043e\u0442\u043e\u043c \u0431\u0443\u0434\u0435\u0442 \u043d\u0435\u043b\u044c\u0437\u044f \u0430\u043d\u0430\u043b\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0434\u0430\u043d\u043d\u043e\u0435 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "\u0412\u044b\u043a\u043b\u044e\u0447\u0438\u0442\u044c \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d, \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043e\u0442 \u0440\u0443\u043a\u0438, \u043f\u043e\u043d\u0438\u043c\u0430\u044f, \u0447\u0442\u043e \u044d\u0442\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0431\u0443\u0434\u0435\u0442 \u043d\u0435\u043f\u043e\u043b\u043d\u043e\u0439"], "question": "10. \u041f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u044c\u0442\u0435 \u0441\u0435\u0431\u0435 \u0441\u0438\u0442\u0443\u0430\u0446\u0438\u044e: \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442 \u043d\u0435 \u0434\u0430\u043b \u0441\u0432\u043e\u0435\u0433\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f \u0437\u0430\u043f\u0438\u0441\u044b\u0432\u0430\u0442\u044c \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d. \u0427\u0442\u043e \u0434\u0435\u043b\u0430\u0442\u044c \u0432 \u0434\u0430\u043d\u043d\u043e\u0439 \u0441\u0438\u0442\u0443\u0430\u0446\u0438\u0438?", "input_type": "optioninput", "variant": "", "group_label": "", "choice": [], "choice_correct": [], "response_type": "optionresponse", "answer_correct": ["\u0412\u044b\u043a\u043b\u044e\u0447\u0438\u0442\u044c \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d, \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043e\u0442 \u0440\u0443\u043a\u0438, \u043f\u043e\u043d\u0438\u043c\u0430\u044f, \u0447\u0442\u043e \u044d\u0442\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0431\u0443\u0434\u0435\u0442 \u043d\u0435\u043f\u043e\u043b\u043d\u043e\u0439"], "answer": "\u0412\u044b\u043a\u043b\u044e\u0447\u0438\u0442\u044c \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d, \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043e\u0442 \u0440\u0443\u043a\u0438, \u043f\u043e\u043d\u0438\u043c\u0430\u044f, \u0447\u0442\u043e \u044d\u0442\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0431\u0443\u0434\u0435\u0442 \u043d\u0435\u043f\u043e\u043b\u043d\u043e\u0439", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_14_1": {"choices_map": {"choice_0": "\u041f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e: \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0443 \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e \u043e\u0437\u043d\u0430\u043a\u043e\u043c\u0438\u0442\u044c\u0441\u044f \u0441 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0435\u0439, \u0443\u0441\u043b\u043e\u0432\u0438\u044f\u043c\u0438 \u0435\u0433\u043e \u0443\u0447\u0430\u0441\u0442\u0438\u044f, \u043f\u0440\u0430\u0432\u0430\u043c\u0438 \u0438 \u0437\u0430\u043f\u043e\u043b\u043d\u0438\u0442\u044c \u0444\u043e\u0440\u043c\u0443, \u0432 \u043a\u043e\u0442\u043e\u0440\u043e\u0439 \u043e\u043d \u0441\u0432\u043e\u0435\u0439 \u043f\u043e\u0434\u043f\u0438\u0441\u044c\u044e \u043f\u043e\u0434\u0442\u0432\u0435\u0440\u0434\u0438\u0442, \u0447\u0442\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d \u0443\u0447\u0430\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u044c", "choice_1": "\u0423\u0441\u0442\u043d\u043e: \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0432 \u0443\u0441\u0442\u043d\u043e\u0439 \u0444\u043e\u0440\u043c\u0435 \u0432\u044b\u0440\u0430\u0436\u0430\u0435\u0442 \u0441\u0432\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435, \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d", "choice_2": "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043c\u043e\u0436\u043d\u043e \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043a\u0430\u043a \u043f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e (\u043f\u043e\u0434\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u0444\u043e\u0440\u043c\u044b), \u0442\u0430\u043a \u0438 \u0443\u0441\u0442\u043d\u043e (\u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435)"}, "answers_available": ["\u041f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e: \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0443 \u043d\u0435\u043e\u0431\u0445\u043e\u0434\u0438\u043c\u043e \u043e\u0437\u043d\u0430\u043a\u043e\u043c\u0438\u0442\u044c\u0441\u044f \u0441 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u0435\u0439, \u0443\u0441\u043b\u043e\u0432\u0438\u044f\u043c\u0438 \u0435\u0433\u043e \u0443\u0447\u0430\u0441\u0442\u0438\u044f, \u043f\u0440\u0430\u0432\u0430\u043c\u0438 \u0438 \u0437\u0430\u043f\u043e\u043b\u043d\u0438\u0442\u044c \u0444\u043e\u0440\u043c\u0443, \u0432 \u043a\u043e\u0442\u043e\u0440\u043e\u0439 \u043e\u043d \u0441\u0432\u043e\u0435\u0439 \u043f\u043e\u0434\u043f\u0438\u0441\u044c\u044e \u043f\u043e\u0434\u0442\u0432\u0435\u0440\u0434\u0438\u0442, \u0447\u0442\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d \u0443\u0447\u0430\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u044c", "\u0423\u0441\u0442\u043d\u043e: \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0432 \u0443\u0441\u0442\u043d\u043e\u0439 \u0444\u043e\u0440\u043c\u0435 \u0432\u044b\u0440\u0430\u0436\u0430\u0435\u0442 \u0441\u0432\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435, \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d", "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043c\u043e\u0436\u043d\u043e \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043a\u0430\u043a \u043f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e (\u043f\u043e\u0434\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u0444\u043e\u0440\u043c\u044b), \u0442\u0430\u043a \u0438 \u0443\u0441\u0442\u043d\u043e (\u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435)"], "question": "13. \u041a\u0430\u043a\u0438\u043c \u043e\u0431\u0440\u0430\u0437\u043e\u043c \u043c\u043e\u0436\u043d\u043e \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043c\u043e\u0436\u043d\u043e \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043a\u0430\u043a \u043f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e (\u043f\u043e\u0434\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u0444\u043e\u0440\u043c\u044b), \u0442\u0430\u043a \u0438 \u0443\u0441\u0442\u043d\u043e (\u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435)"], "answer": "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u043c\u043e\u0436\u043d\u043e \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043a\u0430\u043a \u043f\u0438\u0441\u044c\u043c\u0435\u043d\u043d\u043e (\u043f\u043e\u0434\u043f\u0438\u0441\u0430\u043d\u0438\u0435 \u0444\u043e\u0440\u043c\u044b), \u0442\u0430\u043a \u0438 \u0443\u0441\u0442\u043d\u043e (\u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435 \u0444\u0438\u043a\u0441\u0438\u0440\u0443\u0435\u0442\u0441\u044f \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435)", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_7_1": {"choices_map": {"choice_4": "\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u0443\u043f\u043e\u0442\u0440\u0435\u0431\u043b\u044f\u0442\u044c \u0441\u043f\u0438\u0440\u0442\u043d\u044b\u0435 \u043d\u0430\u043f\u0438\u0442\u043a\u0438 \u0432\u043e \u0432\u0440\u0435\u043c\u044f \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "choice_5": "\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0431\u0435\u0441\u0435\u0434\u044b \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d ", "choice_0": "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0437\u0430\u043a\u043e\u043d\u0447\u0438\u0442\u044c \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0432 \u043b\u044e\u0431\u043e\u0439 \u043c\u043e\u043c\u0435\u043d\u0442", "choice_1": "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0438\u0441\u043f\u0440\u0430\u0432\u043b\u044f\u0442\u044c \u0432\u043e\u043f\u0440\u043e\u0441\u044b \u0433\u0430\u0439\u0434\u0430", "choice_2": "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0441\u0435\u0440\u0434\u0438\u0442\u044c\u0441\u044f \u043d\u0430 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e\u0435\u0440\u0430", "choice_3": "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u043d\u0435 \u043e\u0442\u0432\u0435\u0447\u0430\u0442\u044c \u043d\u0430 \u043b\u044e\u0431\u044b\u0435 \u0438\u0437 \u0437\u0430\u0434\u0430\u0432\u0430\u0435\u043c\u044b\u0445 \u0412\u0430\u043c \u0432\u043e\u043f\u0440\u043e\u0441\u043e\u0432"}, "answers_available": ["\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u0443\u043f\u043e\u0442\u0440\u0435\u0431\u043b\u044f\u0442\u044c \u0441\u043f\u0438\u0440\u0442\u043d\u044b\u0435 \u043d\u0430\u043f\u0438\u0442\u043a\u0438 \u0432\u043e \u0432\u0440\u0435\u043c\u044f \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0431\u0435\u0441\u0435\u0434\u044b \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d ", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0437\u0430\u043a\u043e\u043d\u0447\u0438\u0442\u044c \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0432 \u043b\u044e\u0431\u043e\u0439 \u043c\u043e\u043c\u0435\u043d\u0442", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0438\u0441\u043f\u0440\u0430\u0432\u043b\u044f\u0442\u044c \u0432\u043e\u043f\u0440\u043e\u0441\u044b \u0433\u0430\u0439\u0434\u0430", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0441\u0435\u0440\u0434\u0438\u0442\u044c\u0441\u044f \u043d\u0430 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e\u0435\u0440\u0430", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u043d\u0435 \u043e\u0442\u0432\u0435\u0447\u0430\u0442\u044c \u043d\u0430 \u043b\u044e\u0431\u044b\u0435 \u0438\u0437 \u0437\u0430\u0434\u0430\u0432\u0430\u0435\u043c\u044b\u0445 \u0412\u0430\u043c \u0432\u043e\u043f\u0440\u043e\u0441\u043e\u0432"], "question": "6. \u041e \u043a\u0430\u043a\u0438\u0445 \u043f\u0440\u0430\u0432\u0430\u0445 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f \u0441\u043e\u043e\u0431\u0449\u0430\u0435\u0442\u0441\u044f \u043f\u043e\u0442\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u043c\u0443 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0443 \u043f\u0435\u0440\u0435\u0434 \u043d\u0430\u0447\u0430\u043b\u043e\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f?\n\u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0412\u0421\u0415 \u0432\u0435\u0440\u043d\u044b\u0435 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u044b \u043e\u0442\u0432\u0435\u0442\u0430.", "input_type": "checkboxgroup", "variant": "", "group_label": "", "choice": ["choice_0", "choice_3", "choice_5"], "choice_correct": ["choice_5", "choice_0", "choice_3"], "response_type": "choiceresponse", "answer_correct": ["\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0431\u0435\u0441\u0435\u0434\u044b \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d ", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0437\u0430\u043a\u043e\u043d\u0447\u0438\u0442\u044c \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0432 \u043b\u044e\u0431\u043e\u0439 \u043c\u043e\u043c\u0435\u043d\u0442", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u043d\u0435 \u043e\u0442\u0432\u0435\u0447\u0430\u0442\u044c \u043d\u0430 \u043b\u044e\u0431\u044b\u0435 \u0438\u0437 \u0437\u0430\u0434\u0430\u0432\u0430\u0435\u043c\u044b\u0445 \u0412\u0430\u043c \u0432\u043e\u043f\u0440\u043e\u0441\u043e\u0432"], "answer": ["\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u0437\u0430\u043a\u043e\u043d\u0447\u0438\u0442\u044c \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0432 \u043b\u044e\u0431\u043e\u0439 \u043c\u043e\u043c\u0435\u043d\u0442", "\u0412\u044b \u0438\u043c\u0435\u0435\u0442\u0435 \u043f\u0440\u0430\u0432\u043e \u043d\u0435 \u043e\u0442\u0432\u0435\u0447\u0430\u0442\u044c \u043d\u0430 \u043b\u044e\u0431\u044b\u0435 \u0438\u0437 \u0437\u0430\u0434\u0430\u0432\u0430\u0435\u043c\u044b\u0445 \u0412\u0430\u043c \u0432\u043e\u043f\u0440\u043e\u0441\u043e\u0432", "\u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0431\u0435\u0441\u0435\u0434\u044b \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d "], "correct": true}, "6862136954f64ed1aa24e0264d2ef591_9_1": {"choices_map": {"choice_0": "\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0443 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f \u0432\u0441\u0435 \u0440\u0430\u0432\u043d\u043e, \u0443\u043f\u043e\u043c\u0438\u043d\u0430\u044e\u0442 \u0435\u0433\u043e \u043d\u0430\u0441\u0442\u043e\u044f\u0449\u0435\u0435 \u0438\u043c\u044f \u0438\u043b\u0438 \u043d\u0435\u0442", "choice_1": "\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0441\u043e\u043e\u0431\u0449\u0438\u043b \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u0443\u043a\u0430\u0437\u044b\u0432\u0430\u044e\u0449\u0443\u044e \u043d\u0430 \u043d\u0430\u0440\u0443\u0448\u0435\u043d\u0438\u0435 \u0437\u0430\u043a\u043e\u043d\u043e\u0434\u0430\u0442\u0435\u043b\u044c\u0441\u0442\u0432\u0430", "choice_2": "\u0415\u0441\u043b\u0438 \u043f\u0440\u043e\u0435\u043a\u0442 \u0431\u044b\u043b \u0437\u0430\u0432\u0435\u0440\u0448\u0435\u043d \u0431\u043e\u043b\u0435\u0435 5 \u043b\u0435\u0442 \u043d\u0430\u0437\u0430\u0434", "choice_3": "\u041d\u0438 \u0432 \u043a\u0430\u043a\u0438\u0445 \u0441\u043b\u0443\u0447\u0430\u044f\u0445 \u043d\u0435\u043b\u044c\u0437\u044f \u043d\u0430\u0440\u0443\u0448\u0430\u0442\u044c \u0433\u0430\u0440\u0430\u043d\u0442\u0438\u0438 \u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u0438"}, "answers_available": ["\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0443 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f \u0432\u0441\u0435 \u0440\u0430\u0432\u043d\u043e, \u0443\u043f\u043e\u043c\u0438\u043d\u0430\u044e\u0442 \u0435\u0433\u043e \u043d\u0430\u0441\u0442\u043e\u044f\u0449\u0435\u0435 \u0438\u043c\u044f \u0438\u043b\u0438 \u043d\u0435\u0442", "\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0441\u043e\u043e\u0431\u0449\u0438\u043b \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u0443\u043a\u0430\u0437\u044b\u0432\u0430\u044e\u0449\u0443\u044e \u043d\u0430 \u043d\u0430\u0440\u0443\u0448\u0435\u043d\u0438\u0435 \u0437\u0430\u043a\u043e\u043d\u043e\u0434\u0430\u0442\u0435\u043b\u044c\u0441\u0442\u0432\u0430", "\u0415\u0441\u043b\u0438 \u043f\u0440\u043e\u0435\u043a\u0442 \u0431\u044b\u043b \u0437\u0430\u0432\u0435\u0440\u0448\u0435\u043d \u0431\u043e\u043b\u0435\u0435 5 \u043b\u0435\u0442 \u043d\u0430\u0437\u0430\u0434", "\u041d\u0438 \u0432 \u043a\u0430\u043a\u0438\u0445 \u0441\u043b\u0443\u0447\u0430\u044f\u0445 \u043d\u0435\u043b\u044c\u0437\u044f \u043d\u0430\u0440\u0443\u0448\u0430\u0442\u044c \u0433\u0430\u0440\u0430\u043d\u0442\u0438\u0438 \u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u0438"], "question": "8. \u0412 \u043a\u0430\u043a\u043e\u043c \u0441\u043b\u0443\u0447\u0430\u0435 \u0412\u044b \u043c\u043e\u0436\u0435\u0442\u0435 \u043d\u0430\u0440\u0443\u0448\u0438\u0442\u044c \u0433\u0430\u0440\u0430\u043d\u0442\u0438\u0438 \u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u0438\u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f?.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_1", "choice_correct": ["choice_1"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0441\u043e\u043e\u0431\u0449\u0438\u043b \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u0443\u043a\u0430\u0437\u044b\u0432\u0430\u044e\u0449\u0443\u044e \u043d\u0430 \u043d\u0430\u0440\u0443\u0448\u0435\u043d\u0438\u0435 \u0437\u0430\u043a\u043e\u043d\u043e\u0434\u0430\u0442\u0435\u043b\u044c\u0441\u0442\u0432\u0430"], "answer": "\u0415\u0441\u043b\u0438 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u0441\u043e\u043e\u0431\u0449\u0438\u043b \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u0443\u043a\u0430\u0437\u044b\u0432\u0430\u044e\u0449\u0443\u044e \u043d\u0430 \u043d\u0430\u0440\u0443\u0448\u0435\u043d\u0438\u0435 \u0437\u0430\u043a\u043e\u043d\u043e\u0434\u0430\u0442\u0435\u043b\u044c\u0441\u0442\u0432\u0430", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_10_1": {"choices_map": {"choice_0": "\u041a\u0430\u0442\u0435\u0433\u043e\u0440\u0438\u0447\u0435\u0441\u043a\u0438 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f, \u043e\u043d \u0437\u0434\u0435\u0441\u044c \u043d\u0435 \u0437\u0430 \u044d\u0442\u0438\u043c", "choice_1": "\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u043f\u043e\u043c\u043e\u0449\u044c \u0432 \u043c\u0435\u0440\u0443 \u0441\u0432\u043e\u0438\u0445 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0435\u0439 \u043a\u0430\u043a \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f", "choice_2": "\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u0432\u0441\u044e \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u0443\u044e \u043f\u043e\u043c\u043e\u0449\u044c, \u0447\u0435\u043b\u043e\u0432\u0435\u043a \u0436\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u043b\u0441\u044f \u0443\u0447\u0430\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u044c \u0432 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0438"}, "answers_available": ["\u041a\u0430\u0442\u0435\u0433\u043e\u0440\u0438\u0447\u0435\u0441\u043a\u0438 \u043e\u0442\u043a\u0430\u0437\u0430\u0442\u044c\u0441\u044f, \u043e\u043d \u0437\u0434\u0435\u0441\u044c \u043d\u0435 \u0437\u0430 \u044d\u0442\u0438\u043c", "\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u043f\u043e\u043c\u043e\u0449\u044c \u0432 \u043c\u0435\u0440\u0443 \u0441\u0432\u043e\u0438\u0445 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0435\u0439 \u043a\u0430\u043a \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f", "\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u0432\u0441\u044e \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u0443\u044e \u043f\u043e\u043c\u043e\u0449\u044c, \u0447\u0435\u043b\u043e\u0432\u0435\u043a \u0436\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u043b\u0441\u044f \u0443\u0447\u0430\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u044c \u0432 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0438"], "question": "9. \u0415\u0441\u043b\u0438 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f \u043f\u0440\u043e\u0441\u044f\u0442 \u043e \u043f\u043e\u043c\u043e\u0449\u0438, \u043a\u0430\u043a \u0435\u043c\u0443 \u0440\u0435\u043a\u043e\u043c\u0435\u043d\u0434\u0443\u0435\u0442\u0441\u044f \u0441\u0435\u0431\u044f \u043f\u043e\u0432\u0435\u0441\u0442\u0438?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_1", "choice_correct": ["choice_1"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u043f\u043e\u043c\u043e\u0449\u044c \u0432 \u043c\u0435\u0440\u0443 \u0441\u0432\u043e\u0438\u0445 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0435\u0439 \u043a\u0430\u043a \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f"], "answer": "\u041e\u043a\u0430\u0437\u0430\u0442\u044c \u043f\u043e\u043c\u043e\u0449\u044c \u0432 \u043c\u0435\u0440\u0443 \u0441\u0432\u043e\u0438\u0445 \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u0435\u0439 \u043a\u0430\u043a \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_3_1": {"choices_map": {}, "answers_available": [], "question": "2. \u041f\u043e\u043b\u0443\u0447\u0435\u043d\u0438\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f \u043d\u0430 \u0443\u0447\u0430\u0441\u0442\u0438\u0435 \u0432 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0438 \u2013 \u044d\u0442\u043e (\u0434\u0432\u0430 \u0441\u043b\u043e\u0432\u0430)", "input_type": "textline", "variant": "", "group_label": "", "choice": [], "choice_correct": [], "response_type": "stringresponse", "answer_correct": ["\u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435"], "answer": "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_6_1": {"choices_map": {"choice_4": "\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "choice_0": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u0441\u043a\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "choice_1": "\u0418\u043d\u0442\u0435\u043b\u043b\u0435\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "choice_2": "\u041b\u0438\u0447\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "choice_3": "\u041a\u043e\u0440\u043f\u043e\u0440\u0430\u0442\u0438\u0432\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430"}, "answers_available": ["\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c\u0441\u043a\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "\u0418\u043d\u0442\u0435\u043b\u043b\u0435\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "\u041b\u0438\u0447\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "\u041a\u043e\u0440\u043f\u043e\u0440\u0430\u0442\u0438\u0432\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430"], "question": "5. \u041a\u0430\u043a\u043e\u0439 \u0432\u0438\u0434 \u044d\u0442\u0438\u043a\u0438 \u043c\u043e\u0436\u043d\u043e \u043e\u043f\u0438\u0441\u0430\u0442\u044c \u0441\u043b\u043e\u0432\u0430\u043c\u0438: \u0438\u043d\u0434\u0438\u0432\u0438\u0434\u0443\u0430\u043b\u044c\u043d\u044b\u0435 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u0438\u044f \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f \u043e \u0442\u043e\u043c, \u0447\u0442\u043e \u043f\u0440\u0430\u0432\u0438\u043b\u044c\u043d\u043e/\u043d\u0435\u043f\u0440\u0430\u0432\u0438\u043b\u044c\u043d\u043e,\n\u0441\u043f\u0440\u0430\u0432\u0435\u0434\u043b\u0438\u0432\u043e/\u043d\u0435\u0441\u043f\u0440\u0430\u0432\u0435\u0434\u043b\u0438\u0432\u043e?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041b\u0438\u0447\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430"], "answer": "\u041b\u0438\u0447\u043d\u0430\u044f \u044d\u0442\u0438\u043a\u0430", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_15_1": {"choices_map": {"choice_4": "\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043f\u0440\u0435\u043a\u0440\u0430\u0449\u0435\u043d\u0438\u0435 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "choice_0": "\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043f\u0440\u043e\u0435\u043a\u0442\u0435", "choice_1": "\u0414\u043e\u0431\u0440\u043e\u0432\u043e\u043b\u044c\u043d\u043e\u0441\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u0438\u044f", "choice_2": "\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0430\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c \u0438 \u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c", "choice_3": "\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043e\u0442\u043a\u0430\u0437 \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e"}, "answers_available": ["\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043f\u0440\u0435\u043a\u0440\u0430\u0449\u0435\u043d\u0438\u0435 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043f\u0440\u043e\u0435\u043a\u0442\u0435", "\u0414\u043e\u0431\u0440\u043e\u0432\u043e\u043b\u044c\u043d\u043e\u0441\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u0438\u044f", "\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0430\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c \u0438 \u043a\u043e\u043d\u0444\u0438\u0434\u0435\u043d\u0446\u0438\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c", "\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043e\u0442\u043a\u0430\u0437 \u043e\u0442 \u0437\u0430\u043f\u0438\u0441\u0438 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e"], "question": "14. \u041f\u0440\u043e\u0430\u043d\u0430\u043b\u0438\u0437\u0438\u0440\u0443\u0439\u0442\u0435 \u043f\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043b\u0435\u043d\u043d\u0443\u044e \u043d\u0438\u0436\u0435 \u0444\u043e\u0440\u043c\u0443 \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0433\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f.\n<p/>\n<p><b>\u0424\u043e\u0440\u043c\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0433\u043e \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f</b></p>\n<p>\u0415\u0441\u043b\u0438 \u0412\u044b \u0441\u043e\u0433\u043b\u0430\u0441\u043d\u044b \u0443\u0447\u0430\u0441\u0442\u0432\u043e\u0432\u0430\u0442\u044c \u0432 \u043d\u0430\u0448\u0435\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0438, \u043f\u043e\u0436\u0430\u043b\u0443\u0439\u0441\u0442\u0430, \u0437\u0430\u043f\u043e\u043b\u043d\u0438\u0442\u0435 \u0438 \u043f\u043e\u0434\u043f\u0438\u0448\u0438\u0442\u0435 \u0441\u043b\u0435\u0434\u0443\u044e\u0449\u0443\u044e \u0444\u043e\u0440\u043c\u0443 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u044f. \u041f\u043e\u0436\u0430\u043b\u0443\u0439\u0441\u0442\u0430, \u043d\u0430\u043f\u0438\u0448\u0438\u0442\u0435 \u0441\u0432\u043e\u0438 \u0438\u043d\u0438\u0446\u0438\u0430\u043b\u044b \u0432 \u044f\u0447\u0435\u0439\u043a\u0435 \u043d\u0430\u043f\u0440\u043e\u0442\u0438\u0432 \u0442\u0435\u0445 \u0441\u0443\u0436\u0434\u0435\u043d\u0438\u0439, \u0441 \u043a\u043e\u0442\u043e\u0440\u044b\u043c\u0438 \u0412\u044b \u0441\u043e\u0433\u043b\u0430\u0441\u043d\u044b, \u0438 \u043f\u043e\u0441\u0442\u0430\u0432\u044c\u0442\u0435 \u043f\u0440\u043e\u0447\u0435\u0440\u043a \u043d\u0430\u043f\u0440\u043e\u0442\u0438\u0432 \u0442\u0435\u0445, \u0441 \u043a\u043e\u0442\u043e\u0440\u044b\u043c\u0438 \u043d\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u043d\u044b;\n\u0412\u0430\u043c \u043d\u0435\u043e\u0431\u044f\u0437\u0430\u0442\u0435\u043b\u044c\u043d\u043e \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u0442\u044c\u0441\u044f \u0441\u043e \u0432\u0441\u0435\u043c\u0438 \u0443\u0442\u0432\u0435\u0440\u0436\u0434\u0435\u043d\u0438\u044f\u043c\u0438.</p>\n\n<p>\u0418\u043d\u0438\u0446\u0438\u0430\u043b\u044b</p>\n\n<p>\u042f \u043f\u043e\u043d\u0438\u043c\u0430\u044e, \u0447\u0442\u043e \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0431\u0443\u0434\u0443\u0442 \u0437\u0430\u043f\u0438\u0441\u0430\u043d\u044b \u043d\u0430 \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d (\u0430\u0443\u0434\u0438\u043e-\u0437\u0430\u043f\u0438\u0441\u044c)</p>\n<p>\u042f \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d/\u0441\u043e\u0433\u043b\u0430\u0441\u043d\u0430 \u043d\u0430 \u0442\u043e, \u0447\u0442\u043e\u0431\u044b \u0446\u0438\u0442\u0430\u0442\u044b \u0438\u0437 \u043c\u043e\u0435\u0433\u043e \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e \u0438\u0441\u043f\u043e\u043b\u044c\u0437\u043e\u0432\u0430\u043b\u0438\u0441\u044c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f\u043c\u0438 \u0432 \u0430\u043d\u043e\u043d\u0438\u043c\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u043c \u0432\u0438\u0434\u0435</p>\n<p>\u042f \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d/\u0441\u043e\u0433\u043b\u0430\u0441\u043d\u0430 \u043d\u0430 \u0442\u043e, \u0447\u0442\u043e\u0431\u044b \u043c\u043e\u0438 \u0434\u0430\u043d\u043d\u044b\u0435 \u0432 \u0430\u043d\u043e\u043d\u0438\u043c\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u043c \u0432\u0438\u0434\u0435 \u043f\u0435\u0440\u0435\u0434\u0430\u0432\u0430\u043b\u0438\u0441\u044c \u0434\u0440\u0443\u0433\u0438\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f\u043c \u0432 \u043f\u0440\u043e\u0435\u043a\u0442\u0435</p>\n<p>\u042f \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d/\u0441\u043e\u0433\u043b\u0430\u0441\u043d\u0430, \u0447\u0442\u043e\u0431\u044b \u0432 \u043f\u0440\u043e\u0435\u043a\u0442\u0435 \u0431\u044b\u043b\u0438 \u0441\u043e\u0431\u0440\u0430\u043d\u044b \u0444\u043e\u0442\u043e/\u0432\u0438\u0434\u0435\u043e \u0441\u043e \u043c\u043d\u043e\u0439</p>\n<p>\u042f \u0441\u043e\u0433\u043b\u0430\u0441\u0435\u043d \u043f\u0440\u0438\u043d\u044f\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u0438\u044f \u0432 \u0434\u0430\u043d\u043d\u043e\u043c \u043f\u0440\u043e\u0435\u043a\u0442\u0435</p>\n\n<p>\u0418\u043c\u044f \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430</p>\n<p>\u0414\u0430\u0442\u0430</p>\n<p>\u041f\u043e\u0434\u043f\u0438\u0441\u044c</p>\n\n<p>\u0418\u043c\u044f \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f, \u043a\u043e\u0442\u043e\u0440\u044b\u0439 \u043f\u043e\u043b\u0443\u0447\u0430\u0435\u0442 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435</p>\n<p>\u0414\u0430\u0442\u0430</p>\n<p>\u041f\u043e\u0434\u043f\u0438\u0441\u044c</p>\n\n<p>\u041a\u0430\u043a\u0438\u0435 \u044d\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u043f\u0440\u0438\u043d\u0446\u0438\u043f\u044b \u041d\u0415 \u043e\u0442\u0440\u0430\u0436\u0435\u043d\u044b \u0432 \u0434\u0430\u043d\u043d\u043e\u0439 \u0444\u043e\u0440\u043c\u0435?</p>\n<p>\u0412\u044b\u0431\u0435\u0440\u0438\u0442\u0435 \u0412\u0421\u0415 \u0432\u0435\u0440\u043d\u044b\u0435 \u0432\u0430\u0440\u0438\u0430\u043d\u0442\u044b \u043e\u0442\u0432\u0435\u0442\u0430</p>", "input_type": "checkboxgroup", "variant": "", "group_label": "", "choice": ["choice_0", "choice_4"], "choice_correct": ["choice_4", "choice_0"], "response_type": "choiceresponse", "answer_correct": ["\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043f\u0440\u0435\u043a\u0440\u0430\u0449\u0435\u043d\u0438\u0435 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e", "\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043f\u0440\u043e\u0435\u043a\u0442\u0435"], "answer": ["\u041f\u0440\u0430\u0432\u043e \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u043d\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e \u043e \u043f\u0440\u043e\u0435\u043a\u0442\u0435", "\u041f\u0440\u0430\u0432\u043e \u043d\u0430 \u043f\u0440\u0435\u043a\u0440\u0430\u0449\u0435\u043d\u0438\u0435 \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e"], "correct": true}, "6862136954f64ed1aa24e0264d2ef591_8_1": {"choices_map": {"choice_0": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0434\u0435\u043b\u0430\u0435\u0442 \u0432\u044b\u0432\u043e\u0434\u044b \u043d\u0430 \u043e\u0434\u043d\u043e\u043c \u043f\u0440\u0438\u043c\u0435\u0440\u0435", "choice_1": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u043d\u0435 \u0432\u044b\u0441\u0442\u0440\u043e\u0438\u043b \u0434\u043e\u0432\u0435\u0440\u0438\u0442\u0435\u043b\u044c\u043d\u044b\u0445 \u043e\u0442\u043d\u043e\u0448\u0435\u043d\u0438\u0439 \u0441 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u043e\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "choice_2": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0432\u043a\u043b\u044e\u0447\u0438\u043b \u0432 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u043f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0438\u0434\u0435\u043d\u0442\u0438\u0444\u0438\u0446\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "choice_3": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u043d\u0435 \u043a\u043e\u043d\u043a\u0440\u0435\u0442\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u043b \u043f\u0440\u043e\u0435\u043a\u0442, \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u043a\u043e\u0442\u043e\u0440\u043e\u0433\u043e \u043f\u0440\u043e\u0445\u043e\u0434\u0438\u043b \u0441\u0431\u043e\u0440 \u0438 \u0430\u043d\u0430\u043b\u0438\u0437 \u0434\u0430\u043d\u043d\u044b\u0445"}, "answers_available": ["\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0434\u0435\u043b\u0430\u0435\u0442 \u0432\u044b\u0432\u043e\u0434\u044b \u043d\u0430 \u043e\u0434\u043d\u043e\u043c \u043f\u0440\u0438\u043c\u0435\u0440\u0435", "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u043d\u0435 \u0432\u044b\u0441\u0442\u0440\u043e\u0438\u043b \u0434\u043e\u0432\u0435\u0440\u0438\u0442\u0435\u043b\u044c\u043d\u044b\u0445 \u043e\u0442\u043d\u043e\u0448\u0435\u043d\u0438\u0439 \u0441 \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u043e\u043c \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0432\u043a\u043b\u044e\u0447\u0438\u043b \u0432 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u043f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0438\u0434\u0435\u043d\u0442\u0438\u0444\u0438\u0446\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u043d\u0435 \u043a\u043e\u043d\u043a\u0440\u0435\u0442\u0438\u0437\u0438\u0440\u043e\u0432\u0430\u043b \u043f\u0440\u043e\u0435\u043a\u0442, \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u043a\u043e\u0442\u043e\u0440\u043e\u0433\u043e \u043f\u0440\u043e\u0445\u043e\u0434\u0438\u043b \u0441\u0431\u043e\u0440 \u0438 \u0430\u043d\u0430\u043b\u0438\u0437 \u0434\u0430\u043d\u043d\u044b\u0445"], "question": "7. \u041f\u0440\u043e\u0430\u043d\u0430\u043b\u0438\u0437\u0438\u0440\u0443\u0439\u0442\u0435 \u043f\u0440\u0435\u0434\u043b\u043e\u0436\u0435\u043d\u043d\u044b\u0439 \u043e\u0442\u0440\u044b\u0432\u043e\u043a \u0438\u0437 \u0432\u044b\u0441\u0442\u0443\u043f\u043b\u0435\u043d\u0438\u044f \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044f \u043d\u0430 \u043a\u043e\u043d\u0444\u0435\u0440\u0435\u043d\u0446\u0438\u0438 (\u043f\u0440\u0438\u043c\u0435\u0440 \u0432\u044b\u043c\u044b\u0448\u043b\u0435\u043d\u043d\u044b\u0439). \u041a\u0430\u043a\u0430\u044f \u044d\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u043e\u0448\u0438\u0431\u043a\u0430 \u0434\u043e\u043f\u0443\u0449\u0435\u043d\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u0435\u043c \u0432 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u0438 \u0440\u0435\u0437\u0443\u043b\u044c\u0442\u0430\u0442\u043e\u0432? \u00ab\u0412 \u0434\u0430\u043d\u043d\u043e\u0439 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u0438 \u044f \u043e\u0431\u0440\u0430\u0449\u0443\u0441\u044c \u043a \u043e\u0434\u043d\u043e\u043c\u0443 \u0441\u043b\u0443\u0447\u0430\u044e \u0438 \u043d\u0430 \u0435\u0433\u043e \u043f\u0440\u0438\u043c\u0435\u0440\u0435 \u0440\u0430\u0441\u0441\u043c\u043e\u0442\u0440\u044e,\n\u043a\u0430\u043a \u043f\u0440\u043e\u0438\u0441\u0445\u043e\u0434\u0438\u0442 \u043f\u0435\u0440\u0435\u0445\u043e\u0434 \u043e\u0442 \u043f\u0430\u0441\u0441\u0438\u0432\u043d\u043e\u0433\u043e \u0433\u0440\u0430\u0436\u0434\u0430\u043d\u0441\u0442\u0432\u0430 \u043a \u0430\u043a\u0442\u0438\u0432\u043d\u043e\u043c\u0443. \u0412 \u043d\u0430\u0448\u0435\u043c \u043f\u0440\u043e\u0435\u043a\u0442\u0435, \u043e \u043a\u043e\u0442\u043e\u0440\u043e\u043c \u044f \u0443\u0436\u0435 \u0443\u043f\u043e\u043c\u0438\u043d\u0430\u043b\u0430 \u0432\u044b\u0448\u0435, \u0431\u044b\u043b \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a \u2013 \u0412\u043b\u0430\u0434\u0438\u043c\u0438\u0440, \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0441\u0442 24 \u043b\u0435\u0442, \u0440\u0430\u0431\u043e\u0442\u0430\u044e\u0449\u0438\u0439 \u0432 \u042f\u043d\u0434\u0435\u043a\u0441\u0435 \u0438 \u043f\u0440\u043e\u0436\u0438\u0432\u0430\u044e\u0449\u0438\u0439 \u043d\u0430 \u0434\u0430\u043d\u043d\u044b\u0439 \u043c\u043e\u043c\u0435\u043d\u0442 \u0432 \u041f\u0435\u0442\u0435\u0440\u0431\u0443\u0440\u0433\u0435 \u2013 \u0441\u0435\u0433\u043e\u0434\u043d\u044f \u043e\u043d \u0438\u0434\u0435\u043d\u0442\u0438\u0444\u0438\u0446\u0438\u0440\u0443\u0435\u0442 \u0441\u0435\u0431\u044f \u043a\u0430\u043a \u0430\u043a\u0442\u0438\u0432\u043d\u043e\u0433\u043e \u0433\u0440\u0430\u0436\u0434\u0430\u043d\u0438\u043d\u0430, \u043e\u0434\u043d\u0430\u043a\u043e \u0442\u0430\u043a\u0438\u043c \u043e\u043d \u0431\u044b\u043b \u043d\u0435 \u0432\u0441\u0435\u0433\u0434\u0430\u2026\u00bb.", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0432\u043a\u043b\u044e\u0447\u0438\u043b \u0432 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u043f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0438\u0434\u0435\u043d\u0442\u0438\u0444\u0438\u0446\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f"], "answer": "\u0418\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u0442\u0435\u043b\u044c \u0432\u043a\u043b\u044e\u0447\u0438\u043b \u0432 \u043f\u0440\u0435\u0437\u0435\u043d\u0442\u0430\u0446\u0438\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044e, \u043a\u043e\u0442\u043e\u0440\u0430\u044f \u043f\u043e\u0437\u0432\u043e\u043b\u044f\u0435\u0442 \u0438\u0434\u0435\u043d\u0442\u0438\u0444\u0438\u0446\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u0443\u0447\u0430\u0441\u0442\u043d\u0438\u043a\u0430 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u044f", "correct": true}, "6862136954f64ed1aa24e0264d2ef591_2_1": {"choices_map": {"choice_0": "\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u0438\u0437\u043c", "choice_1": "\u0410\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c", "choice_2": "\u0414\u043e\u0432\u0435\u0440\u0438\u0435", "choice_3": "\u041e\u0431\u044a\u0435\u043a\u0442\u0438\u0432\u043d\u043e\u0441\u0442\u044c"}, "answers_available": ["\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u0438\u0437\u043c", "\u0410\u043d\u043e\u043d\u0438\u043c\u043d\u043e\u0441\u0442\u044c", "\u0414\u043e\u0432\u0435\u0440\u0438\u0435", "\u041e\u0431\u044a\u0435\u043a\u0442\u0438\u0432\u043d\u043e\u0441\u0442\u044c"], "question": "1.\u0427\u0442\u043e \u044f\u0432\u043b\u044f\u0435\u0442\u0441\u044f \u0431\u0430\u0437\u043e\u0432\u043e\u0439 \u0446\u0435\u043d\u043d\u043e\u0441\u0442\u044c\u044e \u043f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u044c\u043d\u043e\u0439 \u044d\u0442\u0438\u043a\u0438 \u0441\u043e\u0446\u0438\u043e\u043b\u043e\u0433\u0430?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_0", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u0414\u043e\u0432\u0435\u0440\u0438\u0435"], "answer": "\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u0438\u043e\u043d\u0430\u043b\u0438\u0437\u043c", "correct": false}, "6862136954f64ed1aa24e0264d2ef591_12_1": {"choices_map": {"choice_0": "\u041e\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u044e \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e, \u043d\u0435\u043a\u043e\u0442\u043e\u0440\u044b\u0435 \u0438\u0434\u0435\u0438 \u043d\u0435\u043b\u044c\u0437\u044f \u0442\u0435\u0440\u043f\u0435\u0442\u044c \u0438 \u0432\u043a\u043b\u044e\u0447\u0430\u0442\u044c \u0432 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0435", "choice_1": "\u041e\u0431\u044a\u044f\u0441\u043d\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u043d\u0435\u043f\u0440\u0430\u0432", "choice_2": "\u041f\u0440\u043e\u0434\u043e\u043b\u0436\u0443 \u0441\u043b\u0443\u0448\u0430\u0442\u044c, \u0434\u0430\u0432\u0430\u044f \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u044c \u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u0441\u0447\u0438\u0442\u0430\u0435\u0442 \u043d\u0443\u0436\u043d\u044b\u043c, \u043e\u0434\u043d\u0430\u043a\u043e \u043d\u0435 \u0431\u0443\u0434\u0443 \u043f\u043e\u0434\u044b\u0433\u0440\u044b\u0432\u0430\u0442\u044c \u0438 \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u0442\u044c\u0441\u044f"}, "answers_available": ["\u041e\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u044e \u0438\u043d\u0442\u0435\u0440\u0432\u044c\u044e, \u043d\u0435\u043a\u043e\u0442\u043e\u0440\u044b\u0435 \u0438\u0434\u0435\u0438 \u043d\u0435\u043b\u044c\u0437\u044f \u0442\u0435\u0440\u043f\u0435\u0442\u044c \u0438 \u0432\u043a\u043b\u044e\u0447\u0430\u0442\u044c \u0432 \u0438\u0441\u0441\u043b\u0435\u0434\u043e\u0432\u0430\u043d\u0438\u0435", "\u041e\u0431\u044a\u044f\u0441\u043d\u044e \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u043d\u0435\u043f\u0440\u0430\u0432", "\u041f\u0440\u043e\u0434\u043e\u043b\u0436\u0443 \u0441\u043b\u0443\u0448\u0430\u0442\u044c, \u0434\u0430\u0432\u0430\u044f \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u044c \u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u0441\u0447\u0438\u0442\u0430\u0435\u0442 \u043d\u0443\u0436\u043d\u044b\u043c, \u043e\u0434\u043d\u0430\u043a\u043e \u043d\u0435 \u0431\u0443\u0434\u0443 \u043f\u043e\u0434\u044b\u0433\u0440\u044b\u0432\u0430\u0442\u044c \u0438 \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u0442\u044c\u0441\u044f"], "question": "11. \u0415\u0441\u043b\u0438 \u0412\u044b, \u043a\u0430\u043a \u0447\u0435\u043b\u043e\u0432\u0435\u043a, \u043a\u0430\u0442\u0435\u0433\u043e\u0440\u0438\u0447\u0435\u0441\u043a\u0438 \u043d\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u043d\u044b \u0441 \u0442\u0435\u043c, \u0447\u0442\u043e \u0433\u043e\u0432\u043e\u0440\u0438\u0442 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442 (\u043d\u0430\u043f\u0440\u0438\u043c\u0435\u0440, \u0432\u044b\u0441\u043a\u0430\u0437\u044b\u0432\u0430\u0435\u0442 \u043d\u0430\u0446\u0438\u043e\u043d\u0430\u043b\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0435 \u0438\u0434\u0435\u0438), \u0447\u0442\u043e \u0412\u044b \u0431\u0443\u0434\u0435\u0442\u0435 \u0434\u0435\u043b\u0430\u0442\u044c?", "input_type": "choicegroup", "variant": "", "group_label": "", "choice": "choice_2", "choice_correct": ["choice_2"], "response_type": "multiplechoiceresponse", "answer_correct": ["\u041f\u0440\u043e\u0434\u043e\u043b\u0436\u0443 \u0441\u043b\u0443\u0448\u0430\u0442\u044c, \u0434\u0430\u0432\u0430\u044f \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u044c \u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u0441\u0447\u0438\u0442\u0430\u0435\u0442 \u043d\u0443\u0436\u043d\u044b\u043c, \u043e\u0434\u043d\u0430\u043a\u043e \u043d\u0435 \u0431\u0443\u0434\u0443 \u043f\u043e\u0434\u044b\u0433\u0440\u044b\u0432\u0430\u0442\u044c \u0438 \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u0442\u044c\u0441\u044f"], "answer": "\u041f\u0440\u043e\u0434\u043e\u043b\u0436\u0443 \u0441\u043b\u0443\u0448\u0430\u0442\u044c, \u0434\u0430\u0432\u0430\u044f \u0432\u043e\u0437\u043c\u043e\u0436\u043d\u043e\u0441\u0442\u044c \u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u043d\u0442\u0443, \u0447\u0442\u043e \u043e\u043d \u0441\u0447\u0438\u0442\u0430\u0435\u0442 \u043d\u0443\u0436\u043d\u044b\u043c, \u043e\u0434\u043d\u0430\u043a\u043e \u043d\u0435 \u0431\u0443\u0434\u0443 \u043f\u043e\u0434\u044b\u0433\u0440\u044b\u0432\u0430\u0442\u044c \u0438 \u0441\u043e\u0433\u043b\u0430\u0448\u0430\u0442\u044c\u0441\u044f", "correct": true}}, "success": "incorrect", "grade": 14, "correct_map": {"6862136954f64ed1aa24e0264d2ef591_13_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_16_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_5_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_12_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_4_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_14_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_7_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_9_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_10_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_3_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_6_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_15_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_8_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_2_1": {"hint": "", "hintmode": null, "correctness": "incorrect", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}, "6862136954f64ed1aa24e0264d2ef591_11_1": {"hint": "", "hintmode": null, "correctness": "correct", "npoints": null, "answervariable": null, "msg": "", "queuestate": null}}, "state": {"correct_map": {}, "input_state": {"6862136954f64ed1aa24e0264d2ef591_13_1": {}, "6862136954f64ed1aa24e0264d2ef591_16_1": {}, "6862136954f64ed1aa24e0264d2ef591_5_1": {}, "6862136954f64ed1aa24e0264d2ef591_4_1": {}, "6862136954f64ed1aa24e0264d2ef591_11_1": {}, "6862136954f64ed1aa24e0264d2ef591_14_1": {}, "6862136954f64ed1aa24e0264d2ef591_7_1": {}, "6862136954f64ed1aa24e0264d2ef591_9_1": {}, "6862136954f64ed1aa24e0264d2ef591_10_1": {}, "6862136954f64ed1aa24e0264d2ef591_3_1": {}, "6862136954f64ed1aa24e0264d2ef591_6_1": {}, "6862136954f64ed1aa24e0264d2ef591_15_1": {}, "6862136954f64ed1aa24e0264d2ef591_8_1": {}, "6862136954f64ed1aa24e0264d2ef591_2_1": {}, "6862136954f64ed1aa24e0264d2ef591_12_1": {}}, "has_saved_answers": true, "seed": 1, "done": null, "student_answers": {"6862136954f64ed1aa24e0264d2ef591_3_1": "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435", "6862136954f64ed1aa24e0264d2ef591_2_1": "choice_0", "6862136954f64ed1aa24e0264d2ef591_11_1": "option_6862136954f64ed1aa24e0264d2ef591_11_1_dummy_default", "6862136954f64ed1aa24e0264d2ef591_4_1": "choice_0", "6862136954f64ed1aa24e0264d2ef591_5_1": "choice_2"}}, "answers": {"6862136954f64ed1aa24e0264d2ef591_13_1": "choice_1", "6862136954f64ed1aa24e0264d2ef591_16_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_5_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_8_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_4_1": "choice_0", "6862136954f64ed1aa24e0264d2ef591_14_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_7_1": ["choice_0", "choice_3", "choice_5"], "6862136954f64ed1aa24e0264d2ef591_9_1": "choice_1", "6862136954f64ed1aa24e0264d2ef591_10_1": "choice_1", "6862136954f64ed1aa24e0264d2ef591_3_1": "\u0418\u043d\u0444\u043e\u0440\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u0441\u043e\u0433\u043b\u0430\u0441\u0438\u0435", "6862136954f64ed1aa24e0264d2ef591_6_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_15_1": ["choice_0", "choice_4"], "6862136954f64ed1aa24e0264d2ef591_12_1": "choice_2", "6862136954f64ed1aa24e0264d2ef591_2_1": "choice_0", "6862136954f64ed1aa24e0264d2ef591_11_1": "\u0412\u044b\u043a\u043b\u044e\u0447\u0438\u0442\u044c \u0434\u0438\u043a\u0442\u043e\u0444\u043e\u043d, \u0444\u0438\u043a\u0441\u0438\u0440\u043e\u0432\u0430\u0442\u044c \u043e\u0442 \u0440\u0443\u043a\u0438, \u043f\u043e\u043d\u0438\u043c\u0430\u044f, \u0447\u0442\u043e \u044d\u0442\u0430 \u0438\u043d\u0444\u043e\u0440\u043c\u0430\u0446\u0438\u044f \u0431\u0443\u0434\u0435\u0442 \u043d\u0435\u043f\u043e\u043b\u043d\u043e\u0439"}, "attempts": 1, "max_grade": 15, "problem_id": "block-v1:hse+QUALSOC+spring_2020+type@problem+block@6862136954f64ed1aa24e0264d2ef591"}, "event_source": "server", "context": {"course_user_tags": {}, "user_id": 1702781, "org_id": "hse", "asides": {}, "module": {"usage_key": "block-v1:hse+QUALSOC+spring_2020+type@problem+block@6862136954f64ed1aa24e0264d2ef591", "display_name": "\u0412\u043e\u043f\u0440\u043e\u0441 \u0441 \u043c\u043d\u043e\u0436\u0435\u0441\u0442\u0432\u0435\u043d\u043d\u044b\u043c \u0432\u044b\u0431\u043e\u0440\u043e\u043c"}, "course_id": "course-v1:hse+QUALSOC+spring_2020", "path": "/courses/course-v1:hse+QUALSOC+spring_2020/xblock/block-v1:hse+QUALSOC+spring_2020+type@problem+block@6862136954f64ed1aa24e0264d2ef591/handler/xmodule_handler/problem_check"}, "time": "2020-04-26T21:28:19.437197+00:00", "page": "x_module"}'
d = json.loads(s)
t = d['time']


dt = "2020-04-26T21:28:19.437197+00:00"
datetime_object = datetime.strptime(dt[:-6],'%Y-%m-%dT%H:%M:%S.%f')
datetime_object > datetime.strptime("2020-04-01", '%Y-%m-%d')

True

In [82]:
dt = "2020-04-26T21:28:19.437197+00:00"
dt[:19]

'2020-04-26T21:28:19'

In [46]:
f.close()

In [15]:
# парсинг csv одного курса из архива в DataFrame
def parse_structure():
    df['course_id'] = df.apply(lambda x: x[1][:x[1].find('+type@chapter')].replace('block-v1:', ''), axis=1)
    df['chapter_id'] = df.apply(lambda x: x[1].replace(f'block-v1:{x["course_id"]}+type@chapter+block@', ''), axis=1) 
    df['sequential_id'] = df.apply(lambda x: x[2].replace(f'block-v1:{x["course_id"]}+type@sequential+block@', ''), axis=1)
    df['vertical_id'] = df.apply(lambda x: x[3].replace(f'block-v1:{x["course_id"]}+type@vertical+block@', ''), axis=1)
    
    df['item_type'] = df.apply(lambda x: x[4].replace(f'block-v1:{x["course_id"]}+type@', ''), axis=1)
    df['item_type'] = df.apply(lambda x: x['item_type'][:x['item_type'].find('+')], axis=1)
                                                      
    df['item_id'] = df.apply(lambda x: x[4].replace(f'block-v1:{x["course_id"]}+type@{x["item_type"]}+block@', ''), axis=1)
                                                    
    df[['chapter_name', 'sequential_name', 'vertical_name', 'item_name']] = df[[5, 6, 7, 8]]
                                                          

### Функции-обертки для обращений к БД

In [6]:
def find_course(course_extid):
    cursor.callproc("openedu_structure.find_course", [course_extid,])
    return cursor.fetchone()[0]

In [7]:
def add_chapter(course_id, chapter_extid, chapter_name):
    cursor.callproc("openedu_structure.add_chapter", [course_id, chapter_extid, chapter_name,])
    return cursor.fetchone()[0]

In [8]:
def add_sequential(chapter_id, sequential_extid, sequential_name):
    cursor.callproc("openedu_structure.add_sequential", [chapter_id, sequential_extid, sequential_name,])
    return cursor.fetchone()[0]

In [9]:
def add_vertical(sequential_id, vertical_extid, vertical_name):
    cursor.callproc("openedu_structure.add_vertical", [sequential_id, vertical_extid, vertical_name,])
    return cursor.fetchone()[0]

In [10]:
def add_item(vertical_id, item_extid, item_type, item_name):
    cursor.callproc("openedu_structure.add_item", [vertical_id, item_extid, item_type, item_name,])
    return cursor.fetchone()[0]

### Загрузка курса в БД

In [11]:
def load_course_to_db():
    conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='pg215', host='VM-AS494')
    cursor = conn.cursor()

    course_extid = 0
    chapter_extid = 0
    sequential_extid = 0
    vertical_extid = 0
    item_extid = 0

    for row in df.iterrows():
        if course_extid != row[1]['course_id']:
            course_extid = row[1]['course_id']
            course_id = find_course(course_extid)
            if not course_id:
                continue

        if chapter_extid != row[1]['chapter_id']:
            chapter_extid = row[1]['chapter_id']
            chapter_id = add_chapter(course_id, chapter_extid, row[1]['chapter_name'])

        if sequential_extid != row[1]['sequential_id']:
            sequential_extid = row[1]['sequential_id']
            sequential_id = add_sequential(chapter_id, sequential_extid, row[1]['sequential_name'])

        if vertical_extid != row[1]['vertical_id']:
            vertical_extid = row[1]['vertical_id']
            vertical_id = add_vertical(sequential_id, vertical_extid, row[1]['vertical_name'])

        if item_extid != row[1]['item_id']:
            item_extid = row[1]['item_id']
            #print(vertical_id, item_extid, row[1]['item_type'], row[1]['item_name']) 
            add_item(vertical_id, item_extid, row[1]['item_type'], row[1]['item_name'])    

    
    conn.commit()
    cursor.close()
    return course_id, course_extid

In [12]:
# подключение к БД
conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='pg215', host='VM-AS494')
cursor = conn.cursor()

In [13]:
with open('qq.txt', 'r', encoding='utf-8') as g:
    for line in g:
        print(f'Обрабатывается {line}')
        fname = load_from_url(line)
        
        with gzip.open(fname) as f:
            df = pd.read_csv(f, sep=';', header=None)
        
        parse_structure()
        print(f'Размер датафрейма к загрузке {df.shape}')
        
        print(f'Загружен в БД контент для курса  {load_course_to_db()}')


https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_ECONOM_spring_2019.csv.gz

(120, 19)
(133, 'hse+ECONOM+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_EXACTS_spring_2019.csv.gz

(209, 19)
(159, 'hse+EXACTS+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FIL_spring_2019.csv.gz

(274, 19)
(173, 'hse+FIL+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FORLANG_spring_2019.csv.gz

(226, 19)
(227, 'hse+FORLANG+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FUNDLAW_spring_2019.csv.gz

(97, 19)
(234, 'hse+FUNDLAW+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_LABLAW_spring_2019.csv.gz

(219, 19)
(305, 'hse+LABLAW+spring_2019')
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_MACROEC_spring_2019.csv.gz

(139, 19)
(335, 'hse+MACROEC+spring_2019')
https://207918.selcdn.ru/jCJ

In [14]:
conn.commit()
cursor.close()

In [7]:
features = ['course_id', 'chapter_id', 'chapter_name', 'sequential_id', 'sequential_name', 
            'vertical_id', 'vertical_name', 'item_type', 'item_id',  'item_name']
df[features].head()

,course_id,chapter_id,chapter_name,sequential_id,sequential_name,vertical_id,vertical_name,item_type,item_id,item_name
0,hse+DATPRO+spring_2019,8411723edf2f4d8cb6b130fb8023a26d,О Вышке,b74b1284929046d2b280f3c560f0e896,Промо-ролик,e444e0cf02854ed9b516481d39f9a342,Промо-ролик,video,5b50ca94e2774d6f83da03543e39bf50,Видео о ВШЭ
1,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f27a9e3ace954d9a9838ee229f116f94,Понятие информации,video,261019e7ca7d40cabf469069dbef052c,Понятие информации
2,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,dd37a044619949568f9ae7323be7850d,Доступ к информации,video,d1efc98f8244445f9d011479366f9fb2,Доступ к информации
3,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f6b6aa6909cd40abbc74e2fd1845d549,Информационные системы,video,19a06eb43b0f461e88ac12340ed9cb54,Информационные системы
4,hse+DATPRO+spring_2019,911bd2c030644cc9841a2950b62489c0,Тема 1. Основы информационной безопасности. Ос...,da6736e32aae4ac6922f6c55129d312f,Лекция 1,f915b0bf93c24c859d741e11993375f8,Обработка информации,video,94ef7b162a3442a19eb4c3dd436afc45,Обработка информации


In [12]:
conn = psycopg2.connect(dbname='test', user='postgres', 
                        password='pg215', host='VM-AS494')
cursor = conn.cursor()

In [27]:
copy_sql = """
  copy open_edu_course (id, name, ext_id)
  from stdin with
    csv
    header
    delimiter as ';'
"""
from_csv = 'course_for_load1.csv'
with open(from_csv, 'r') as f:
  cursor.copy_expert(sql=copy_sql, file=f)
  conn.commit()
  cursor.close()

In [18]:
?cursor.copy_expert

In [11]:
conn.commit() 

In [3]:
cursor.execute("SELECT * FROM openedu_structure.course where ext_id like '%FUNDLAW%'")

In [4]:
for row in cursor:
    print(row)

(230, '(DEMO)Основы права', 'hse+FUNDLAW+DEMO', None)
(231, '(2018-2)Основы права', 'hse+FUNDLAW+fall_2018', None)
(232, 'Основы права', 'hse+FUNDLAW+fall_2019', None)
(233, 'Основы права', 'hse+FUNDLAW+fall_2019_dvfu', None)
(234, '(2019-1)Основы права', 'hse+FUNDLAW+spring_2019', None)
(235, 'Основы права', 'hse+FUNDLAW+spring_2020', None)
(236, '(2018-2)Основы права', 'hse+FUNDLAW+stud_fall_2018', None)
(237, 'Основы права', 'hse+FUNDLAW+stud_fall_2019', None)
(238, 'Основы права', 'hse+FUNDLAW+workshop_adaptive', None)


In [28]:
from psycopg2 import sql

In [29]:
columns = ('country_name_ru', 'airport_name_ru', 'city_code')
stmt = sql.SQL('SELECT {} FROM {} LIMIT 5').format(
            sql.SQL(',').join(map(sql.Identifier, columns)),
            sql.Identifier('airport')
        )

In [30]:
values = [
        (3, 'однажды в ... ;;;'),
        (4, 'хэлло, товарищи!'),
    ]

In [31]:
insert = sql.SQL('INSERT INTO test_table (id, name) VALUES {}').format(
        sql.SQL(',').join(map(sql.Literal, values))
    )
    

In [32]:
cursor.execute(insert)

UniqueViolation: ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "test_table_pkey"
DETAIL:  Ключ "(id)=(3)" уже существует.


In [22]:
insert

Composed([SQL('INSERT INTO test_table (id, name) VALUES '), Composed([Literal((3, 'однажды в ... ;;;')), SQL(','), Literal((4, 'хэлло, товарищи!'))])])

In [36]:
conn.commit()  # или надо использовать conn.autocommit = True

### На память: для перезагрузки конфига в  psql запустить select pg_reload_conf()

## Скачивание по массиву ссылок

https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_ECONOM_spring_2019.csv.gz

structure_hse_ECONOM_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_EXACTS_spring_2019.csv.gz

structure_hse_EXACTS_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FIL_spring_2019.csv.gz

structure_hse_FIL_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FORLANG_spring_2019.csv.gz

structure_hse_FORLANG_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_FUNDLAW_spring_2019.csv.gz

structure_hse_FUNDLAW_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_LABLAW_spring_2019.csv.gz

structure_hse_LABLAW_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW/SUPPORT-1137e/structure_hse_MACROEC_spring_2019.csv.gz

structure_hse_MACROEC_spring_2019.csv.gz
https://207918.selcdn.ru/jCJ5Ef1pLDebKtwW

In [79]:
import gzip

,0,1,2,3,4,5,6,7,8
0,0,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик
1,1,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии
2,2,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии
3,3,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии
4,4,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии


In [84]:
parse_structure()

In [85]:
df

,0,1,2,3,4,5,6,7,8,course_id,chapter_id,sequential_id,vertical_id,item_type,item_id,chapter_name,sequential_name,vertical_name,item_name
0,0,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик,hse+ECPSY+spring_2019,6c1d114713584f3f90f15200911361f5,6bd9c4040c384db0adc044d826f67b48,b20a29b9ace24655ac0e649d07ef8a88,video,0079332f8dba44ee8dc62bdb87e36acf,О Вышке,Промо-ролик,Промо-ролик,Промо-ролик
1,1,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,d828db02387a41eebb6f0c3ab46f736b,video,08305cf9df4b4fd5ad7fe712117c800d,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 1,Предмет экономической психологии
2,2,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,c283dadf180643899cbe5de78efdd77b,video,8eba95675ac7414fa0e965031ab07dee,Неделя 1: Экономическая психология,Лекция 1,Предмет экономической психологии. Часть 2,Предмет экономической психологии
3,3,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,94db82d471a94052a16038175c861d22,video,0bf8b4fc19df471fb9866da6f52ea426,Неделя 1: Экономическая психология,Лекция 1,История экономической психологии,История экономической психологии
4,4,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,bb5df3a8a76a479aaf1dccd594f0d566,video,4e2716bda32f4d899230f5ac17541dbc,Неделя 1: Экономическая психология,Лекция 1,Методы экономической психологии,Методы экономической психологии
5,5,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@video+bloc...,Неделя 1: Экономическая психология,Лекция 1,О курсе,О курсе,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,e84e2770efd04b09ad487024427b5ea3,f962fabc68d94cefb5ab4996228a174c,video,0c823967d4874b22b74cd0bd5d0f880e,Неделя 1: Экономическая психология,Лекция 1,О курсе,О курсе
6,6,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v1:hse+ECPSY+spring_2019+type@sequential...,block-v1:hse+ECPSY+spring_2019+type@vertical+b...,block-v1:hse+ECPSY+spring_2019+type@problem+bl...,Неделя 1: Экономическая психология,Тест 1,Тест 1,Тест к лекции №1,hse+ECPSY+spring_2019,dd595706f2db4c7ea8d7a99a8a738110,9838dc78885c490687b6dd6e2fce43a2,f63ac0e3aac943bb9f2bd5c5ce15ddfe,problem,f06d73e6a1e341119feb11d1fca76791,Неделя 1: Экономическая психология,Тест 1,Тест 1,Тест к лекции №1
7,7,block-v1:hse+ECPSY+spring_2019+type@chapter+bl...,block-v